In [204]:
import os
import pandas as pd
import numpy as np
import random
import sys
import csv

In [3]:
mimic_path = "C:/Users/Mariam/Documents/AUB/Thesis_Experiments/Data/" # Path containing MIMIC data
output_files = "C:/Users/Mariam/Documents/AUB/Thesis_Experiments/Extracted_CSV_Files/" # Path containing MIMIC data


# Patients File

In [206]:
pt=pd.read_csv(mimic_path+'PATIENTS.csv.gz', header=0, index_col=0)
pt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46520 entries, 234 to 31844
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   SUBJECT_ID   46520 non-null  int64 
 1   GENDER       46520 non-null  object
 2   DOB          46520 non-null  object
 3   DOD          15759 non-null  object
 4   DOD_HOSP     9974 non-null   object
 5   DOD_SSN      13378 non-null  object
 6   EXPIRE_FLAG  46520 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.8+ MB


In [207]:
pt['GENDER'].tolist().count('F')

20399

In [208]:
pt['GENDER'].tolist().count('M')

26121

In [209]:
len(pt['SUBJECT_ID'].tolist())

46520

In [210]:
demo=pt[['SUBJECT_ID', 'GENDER']]

In [211]:
pt.DOB = pd.to_datetime(pt.DOB)
pt.DOD = pd.to_datetime(pt.DOD)
pt.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
ROW_ID,,,,,,,
234,249,F,2075-03-13,NaT,NaN,NaN,0
235,250,F,2164-12-27,2188-11-22,2188-11-22 00:00:00,NaN,1
236,251,M,2090-03-15,NaT,NaN,NaN,0
237,252,M,2078-03-06,NaT,NaN,NaN,0
238,253,F,2089-11-26,NaT,NaN,NaN,0


# Admissions File

In [212]:
adm=pd.read_csv(mimic_path+"ADMISSIONS.csv.gz", header=0, index_col=0) 
adm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58976 entries, 21 to 58598
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   SUBJECT_ID            58976 non-null  int64 
 1   HADM_ID               58976 non-null  int64 
 2   ADMITTIME             58976 non-null  object
 3   DISCHTIME             58976 non-null  object
 4   DEATHTIME             5854 non-null   object
 5   ADMISSION_TYPE        58976 non-null  object
 6   ADMISSION_LOCATION    58976 non-null  object
 7   DISCHARGE_LOCATION    58976 non-null  object
 8   INSURANCE             58976 non-null  object
 9   LANGUAGE              33644 non-null  object
 10  RELIGION              58518 non-null  object
 11  MARITAL_STATUS        48848 non-null  object
 12  ETHNICITY             58976 non-null  object
 13  EDREGTIME             30877 non-null  object
 14  EDOUTTIME             30877 non-null  object
 15  DIAGNOSIS             58951 non-nul

In [213]:
len(adm['HAS_CHARTEVENTS_DATA'].tolist())

58976

In [214]:
len(adm['HADM_ID'].tolist())

58976

In [215]:
demo1 = adm[['SUBJECT_ID', 'HADM_ID', 'INSURANCE','LANGUAGE','DISCHARGE_LOCATION','ADMISSION_TYPE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY']]

In [216]:
def subj_merge(table1, table2):
    return table1.merge(table2, how='inner', left_on=['SUBJECT_ID'], right_on=['SUBJECT_ID'])

In [217]:
adm_pt=subj_merge(adm, pt)
adm_pt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58976 entries, 0 to 58975
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   SUBJECT_ID            58976 non-null  int64         
 1   HADM_ID               58976 non-null  int64         
 2   ADMITTIME             58976 non-null  object        
 3   DISCHTIME             58976 non-null  object        
 4   DEATHTIME             5854 non-null   object        
 5   ADMISSION_TYPE        58976 non-null  object        
 6   ADMISSION_LOCATION    58976 non-null  object        
 7   DISCHARGE_LOCATION    58976 non-null  object        
 8   INSURANCE             58976 non-null  object        
 9   LANGUAGE              33644 non-null  object        
 10  RELIGION              58518 non-null  object        
 11  MARITAL_STATUS        48848 non-null  object        
 12  ETHNICITY             58976 non-null  object        
 13  EDREGTIME       

In [218]:
import datetime as DT
adm_pt['ADMITTIME'] = pd.to_datetime(adm_pt['ADMITTIME']).dt.date
adm_pt['DOB'] = pd.to_datetime(adm_pt['DOB']).dt.date

In [219]:
adm_pt['AGE'] = adm_pt.apply(lambda e: (e['ADMITTIME'] - e['DOB']).days/365, axis=1)


In [220]:
adm_pt

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,AGE
0,22,165315,2196-04-09,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,...,BENZODIAZEPINE OVERDOSE,0,1,F,2131-05-07,NaT,NaN,NaN,0,64.969863
1,23,152223,2153-09-03,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,...,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,M,2082-07-17,NaT,NaN,NaN,0,71.178082
2,23,124321,2157-10-18,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,...,BRAIN MASS,0,1,M,2082-07-17,NaT,NaN,NaN,0,75.304110
3,24,161859,2139-06-06,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,...,INTERIOR MYOCARDIAL INFARCTION,0,1,M,2100-05-31,NaT,NaN,NaN,0,39.041096
4,25,129635,2160-11-02,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,...,ACUTE CORONARY SYNDROME,0,1,M,2101-11-21,NaT,NaN,NaN,0,58.989041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,98794,190603,2127-11-07,2127-11-11 14:30:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,...,AORTIC STENOSIS\AORTIC VALVE REPLACEMENT /SDA,0,1,M,2049-07-29,NaT,NaN,NaN,0,78.326027
58972,98797,105447,2132-12-24,2132-12-25 12:00:00,2132-12-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,...,ALTERED MENTAL STATUS,1,1,M,2044-12-27,2132-12-25,2132-12-25 00:00:00,NaN,1,88.049315
58973,98800,191113,2131-03-30,2131-04-02 15:02:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,...,TRAUMA,0,1,F,2111-11-05,NaT,NaN,NaN,0,19.410959
58974,98802,101071,2151-03-05,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,...,SAH,1,1,F,2067-09-21,2151-03-06,2151-03-06 00:00:00,NaN,1,83.506849


In [221]:
adm_pt.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN',
       'EXPIRE_FLAG', 'AGE'],
      dtype='object')

In [222]:
adm_pt.drop(['ADMITTIME','DISCHTIME','DEATHTIME','EDREGTIME','EDOUTTIME','HOSPITAL_EXPIRE_FLAG','DOB','DOD','DOD_HOSP','DOD_SSN','EXPIRE_FLAG'],inplace=True,axis=1)

In [223]:
adm_pt

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,GENDER,AGE
0,22,165315,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1,F,64.969863
1,23,152223,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,1,M,71.178082
2,23,124321,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,1,M,75.304110
3,24,161859,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1,M,39.041096
4,25,129635,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1,M,58.989041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,98794,190603,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,AORTIC STENOSIS\AORTIC VALVE REPLACEMENT /SDA,1,M,78.326027
58972,98797,105447,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,ALTERED MENTAL STATUS,1,M,88.049315
58973,98800,191113,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,TRAUMA,1,F,19.410959
58974,98802,101071,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SAH,1,F,83.506849


In [224]:
adm_pt.to_csv(output_files+'demographics.csv')

# ICU Stays File

In [225]:
icu=pd.read_csv(mimic_path+"ICUSTAYS.csv.gz", header=0, index_col=0) 
icu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61532 entries, 365 to 59810
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SUBJECT_ID      61532 non-null  int64  
 1   HADM_ID         61532 non-null  int64  
 2   ICUSTAY_ID      61532 non-null  int64  
 3   DBSOURCE        61532 non-null  object 
 4   FIRST_CAREUNIT  61532 non-null  object 
 5   LAST_CAREUNIT   61532 non-null  object 
 6   FIRST_WARDID    61532 non-null  int64  
 7   LAST_WARDID     61532 non-null  int64  
 8   INTIME          61532 non-null  object 
 9   OUTTIME         61522 non-null  object 
 10  LOS             61522 non-null  float64
dtypes: float64(1), int64(5), object(5)
memory usage: 5.6+ MB


In [226]:
icu.INTIME = pd.to_datetime(icu.INTIME)
icu.OUTTIME = pd.to_datetime(icu.OUTTIME)
icu.ICUSTAY_ID.isnull().sum()

0

In [227]:
icu=icu[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LOS']]

In [228]:
def adm_merge(table1, table2):
    return table1.merge(table2, how='left', left_on=['SUBJECT_ID','HADM_ID'], right_on=['SUBJECT_ID','HADM_ID'])

In [229]:
icu_adm_pt=adm_merge(icu, adm_pt)

In [230]:
icu_adm_pt

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,LOS,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,GENDER,AGE
0,268,110404,280836,3.2490,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,SEPARATED,HISPANIC OR LATINO,DYSPNEA,1,F,66.019178
1,269,106296,206613,3.2788,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,NaN,UNOBTAINABLE,SINGLE,WHITE,SEPSIS;PILONIDAL ABSCESS,1,M,40.126027
2,270,188028,220345,2.8939,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,JEHOVAH'S WITNESS,MARRIED,UNKNOWN/NOT SPECIFIED,CAROTID STENOSIS\CAROTID ANGIOGRAM AND STENT,1,M,80.128767
3,271,173727,249196,2.0600,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,ENGL,NOT SPECIFIED,MARRIED,PATIENT DECLINED TO ANSWER,GALLSTONE PANCREATITIS,1,F,45.715068
4,272,164716,210407,1.6202,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,NaN,UNOBTAINABLE,MARRIED,WHITE,PULMONARY EMBOLIS,1,M,67.139726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,94944,143774,201233,2.1894,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,JEWISH,MARRIED,WHITE,CHEST PAIN;RULE OUT CORONARY ARTERY DISEASE\LE...,1,M,77.161644
61528,94950,123750,283653,2.4942,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SYNCOPE;TELEMETRY,1,F,300.200000
61529,94953,196881,241585,0.9259,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,PROTESTANT QUAKER,WIDOWED,WHITE,CEREBRAL ANEURYSM/SDA,1,F,53.128767
61530,94954,118475,202802,2.3346,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Medicare,ENGL,NOT SPECIFIED,MARRIED,PATIENT DECLINED TO ANSWER,AORTIC STENOSIS\AORTIC VALVE REPLACEMENT /SDA,1,F,67.884932


In [231]:
icu_adm_pt.to_csv(output_files+'icu_stays_demographics.csv')

# CHARTEVENTS and D_ITEMS Files

In [149]:
charts_org=pd.read_csv(mimic_path + "CHARTEVENTS.csv.gz", nrows=100000000, header=0, index_col=0) 
charts_org.info

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\2647419959.py:1: DtypeWarning: Columns (8,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  charts_org=pd.read_csv(mimic_path + "CHARTEVENTS.csv.gz", nrows=100000000, header=0, index_col=0)


<bound method DataFrame.info of           SUBJECT_ID  HADM_ID  ICUSTAY_ID  ITEMID            CHARTTIME  \
ROW_ID                                                                   
788               36   165660    241249.0  223834  2134-05-12 12:00:00   
789               36   165660    241249.0  223835  2134-05-12 12:00:00   
790               36   165660    241249.0  224328  2134-05-12 12:00:00   
791               36   165660    241249.0  224329  2134-05-12 12:00:00   
792               36   165660    241249.0  224330  2134-05-12 12:00:00   
...              ...      ...         ...     ...                  ...   
99885116       12768   182007    224267.0     161  2154-03-12 22:00:00   
99885117       12768   182007    224267.0     190  2154-03-12 22:00:00   
99885118       12768   182007    224267.0     211  2154-03-12 22:00:00   
99885119       12768   182007    224267.0     212  2154-03-12 22:00:00   
99885120       12768   182007    224267.0     525  2154-03-12 22:00:00   

     

In [150]:
charts_org

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
ROW_ID,,,,,,,,,,,,,,
788,36,165660,241249.0,223834,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,15.0,15.00,L/min,0.0,0.0,NaN,NaN
789,36,165660,241249.0,223835,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,100.0,100.00,NaN,0.0,0.0,NaN,NaN
790,36,165660,241249.0,224328,2134-05-12 12:00:00,2134-05-12 12:18:00,20823.0,0.37,0.37,NaN,0.0,0.0,NaN,NaN
791,36,165660,241249.0,224329,2134-05-12 12:00:00,2134-05-12 12:19:00,20823.0,6.0,6.00,min,0.0,0.0,NaN,NaN
792,36,165660,241249.0,224330,2134-05-12 12:00:00,2134-05-12 12:19:00,20823.0,2.5,2.50,NaN,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99885116,12768,182007,224267.0,161,2154-03-12 22:00:00,2154-03-12 22:03:00,20115.0,None,NaN,NaN,NaN,NaN,NaN,NotStopd
99885117,12768,182007,224267.0,190,2154-03-12 22:00:00,2154-03-12 22:46:00,16945.0,.89999997615814209,0.90,torr,NaN,NaN,NaN,NotStopd
99885118,12768,182007,224267.0,211,2154-03-12 22:00:00,2154-03-12 22:03:00,20115.0,135,135.00,BPM,NaN,NaN,NaN,NotStopd


In [151]:
charts_org=charts_org.dropna(how='any', subset=['VALUENUM']) # This removes rows with null values for VALUENUM column (a total of 190,141 records removed)

In [152]:
d_c=pd.read_csv(mimic_path + 'D_ITEMS.csv.gz', header=0, index_col=0) 
d_c=d_c[['ITEMID', 'LABEL', 'CATEGORY']]
d_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12487 entries, 457 to 14522
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ITEMID    12487 non-null  int64 
 1   LABEL     12483 non-null  object
 2   CATEGORY  6049 non-null   object
dtypes: int64(1), object(2)
memory usage: 390.2+ KB


In [153]:
def item_merge(table1, table2):
    return table1.merge(table2, how='inner', left_on=['ITEMID'], right_on=['ITEMID'])

In [154]:
charts=item_merge(charts_org, d_c)
charts

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
0,36,165660,241249.0,223834,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,15.0,15.0,L/min,0.0,0.0,NaN,NaN,O2 Flow,Respiratory
1,23,124321,234044.0,223834,2157-10-21 20:00:00,2157-10-22 00:45:00,19050.0,3.0,3.0,L/min,0.0,0.0,NaN,NaN,O2 Flow,Respiratory
2,23,124321,234044.0,223834,2157-10-21 22:00:00,2157-10-22 00:45:00,19050.0,2.0,2.0,L/min,0.0,0.0,NaN,NaN,O2 Flow,Respiratory
3,23,124321,234044.0,223834,2157-10-22 00:00:00,2157-10-22 00:45:00,19050.0,2.0,2.0,L/min,0.0,0.0,NaN,NaN,O2 Flow,Respiratory
4,36,165660,241249.0,223834,2134-05-12 07:09:00,2134-05-12 07:09:00,17525.0,15.0,15.0,L/min,0.0,0.0,NaN,NaN,O2 Flow,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61057193,12772,118571,232874.0,5943,2163-08-26 15:00:00,2163-08-26 15:14:00,15019.0,25,25.0,NaN,NaN,NaN,NaN,NaN,sigh breath,NaN
61057194,12772,118571,232874.0,5943,2163-08-25 19:00:00,2163-08-25 19:32:00,16945.0,25,25.0,NaN,NaN,NaN,NaN,NaN,sigh breath,NaN
61057195,12772,118571,232874.0,5943,2163-08-28 15:00:00,2163-08-28 14:47:00,15019.0,25,25.0,NaN,NaN,NaN,NaN,NaN,sigh breath,NaN
61057196,12772,118571,232874.0,5943,2163-08-28 17:00:00,2163-08-28 16:55:00,15019.0,0,0.0,NaN,NaN,NaN,NaN,NaN,sigh breath,NaN


In [155]:
def unique(lst):
    lst_to_set = set(lst)
    unique_list = list(lst_to_set)
    return unique_list
chart_items=unique(d_c['LABEL'].values)

In [156]:
for i in chart_items:
    if str(i)=='nan':
        chart_items.remove(i)

In [157]:
counts={}
for col in chart_items:
    counts.update({col:len(charts[charts.LABEL.isin([col])])})

In [158]:
import operator
countslist= sorted(counts.items(), key=operator.itemgetter(1), reverse=True) 
countssortdict=dict(countslist)

In [159]:
selcols=[]
j=0
for i in countslist:
    selcols.append(i[0])
    if j>=150:
        break
    j+=1

In [160]:
l=['Temp Axillary [F]','Temp Rectal [F]',
 'Temperature Fahrenheit', 'Temperature Celsius',
 'Skin [Temperature]', 
 'Temperature C', 'Temperature C (calc)', 
 'Temperature F', 'Temperature F (calc)',
 'Central Venous Pressure  Alarm - Low',
 'Central Venous Pressure Alarm - High',
 'Tidal Volume (observed)', 'Tidal Volume (set)',
 'Tidal Volume (spontaneous)','GCS - Eye Opening',
 'GCS - Verbal Response','GCS - Motor Response','Creatinine',
 'CreatinineApacheIIScore', 'CreatinineApacheIIValue',
 'Creatinine_ApacheIV','BUN    (6-20)', 'BUN (6-20)','WBC   (4-11,000)', 
 'WBC (4-11,000)', 
 'WBC 4.0-11.0','Hematocrit (whole blood - calc)','Potassium (whole blood)', 
 'Potassium  (3.5-5.3)', 'Potassium (3.5-5.3)',
 'Sodium (serum)', 'Sodium', 'Sodium  (135-148)', 'Sodium (135-148)', 
 'Sodium (whole blood)','Non Invasive Blood Pressure diastolic',
 'Non Invasive Blood Pressure mean',
 'Non Invasive Blood Pressure systolic',
 'Non-Invasive Blood Pressure Alarm - High',
 'Non-Invasive Blood Pressure Alarm - Low','Glucose (serum)', 
 'Glucose (whole blood)', 'Glucose finger stick','Arterial O2 pressure', 
 'Arterial PaO2', 'Inspired O2 Fraction', 'FIO2', 'FIO2 Alarm [High]',
 'FIO2 Alarm [Low]', 'FIO2 Alarm-High', 'FIO2 [Meas]',
 'Pulmonary Artery Pressure Alarm - High',
 'Pulmonary Artery Pressure Alarm - Low',
 'Pulmonary Artery Pressure diastolic',
 'Pulmonary Artery Pressure mean',
 'Pulmonary Artery Pressure systolic',
 'ALT',
 'ART BP Diastolic',
 'ART BP Systolic',
 'ART BP mean',
 'AST',
 'Access Pressure',
 'Admission Weight (Kg)',
 'Admission Weight (lbs.)',
 'Albumin',
 'Alkaline Phosphate',
 'Anion gap',
 'Apnea Interval',
 'Arterial Base Excess',
 'Arterial Blood Pressure Alarm - High',
 'Arterial Blood Pressure Alarm - Low',
 'Arterial Blood Pressure mean',
 'Arterial CO2 Pressure',
 'Arterial O2 Saturation',
 'BUN',
 'Blood Flow (ml/min)',
 'Blood Temperature CCO (C)',
 'CK (CPK)',
 'CK-MB',
 'CO (Arterial)',
 'Calcium non-ionized',
 'Cardiac Output (CCO)',
 'Central Venous Pressure',
 'Cerebral Perfusion Pressure',
 'Chloride (serum)',
 'Citrate (ACD-A)',
 'Creatinine',
 'CreatinineApacheIIScore',
 'CreatinineApacheIIValue',
 'Creatinine_ApacheIV',
 'Cuff Pressure',
 'Current Goal',
 'Daily Weight',
 'Dialysate Rate',
 'Differential-Basos',
 'Differential-Eos',
 'Differential-Lymphs',
 'Differential-Monos',
 'Differential-Neuts',
 'Effluent Pressure',
 'Expiratory Ratio',
 'Filter Pressure',
 'Fingerstick Glucose',
 'Fspn High',
 'HCO3 (serum)',
 'Heart Rate',
 'Hematocrit (whole blood - calc)',
 'Hemoglobin',
 'Heparin Dose (per hour)',
 'Hourly Patient Fluid Removal',
 'INR',
 'Impaired Skin Depth #1',
 'Impaired Skin Length #1',
 'Impaired Skin Length #2',
 'Impaired Skin Length #3',
 'Impaired Skin Width #1',
 'Impaired Skin Width #2',
 'Impaired Skin Width #3',
 'Inspiratory Ratio',
 'Inspiratory Time',
 'Inspired Gas Temp.',
 'Inspired O2 Fraction',
 'Intra Cranial Pressure',
 'Ionized Calcium',
 'LDH',
 'Lactic Acid',
 'MDI #1 Puff',
 'MDI #2 Puff',
 'Magnesium',
 'Mean Airway Pressure',
 'Minute Volume',
 'Non Invasive Blood Pressure mean',
 'O2 Flow',
 'O2 Flow (additional cannula)',
 'O2 saturation pulseoxymetry',
 'PA Line cm Mark',
 'PBP (Prefilter) Replacement Rate',
 'PCA 1 hour limit',
 'PCA attempt',
 'PCA dose',
 'PCA inject',
 'PCA lockout (min)',
 'PEEP',
 'PH (Arterial)',
 'PH (dipstick)',
 'PSV Level',
 'PTT',
 'Peak Insp. Pressure',
 'Phosphorous',
 'Plateau Pressure',
 'Platelet Count',
 'Post Filter Replacement Rate',
 'Potassium (whole blood)',
 'Prothrombin time',
 'Pulmonary Artery Pressure mean',
 'Replacement Rate',
 'Respiratory Rate',
 'Return Pressure',
 'SVV (Arterial)',
 'Sodium (serum)',
 'Specific Gravity (urine)',
 'Spont RR',
 'Spont Vt',
 'SvO2',
 'TCO2 (calc) Arterial',
 'Temperature C',
 'Temperature Celsius',
 'Temporary Pacemaker Rate',
 'Temporary Pacemaker Wires Atrial',
 'Temporary Pacemaker Wires Venticular',
 'Tidal Volume (observed)',
 'Total Bilirubin',
 'Total PEEP Level',
 'Troponin-T',
 'Ultrafiltrate Output',
 'Ventilator Tank #1',
 'Ventilator Tank #2',
 'Vti High',
 'WBC'
   
  ]
for i in l:
    if i not in selcols:
        selcols.append(i)
selcols=unique(selcols)

In [161]:
selcols=selcols+[
'Motor Response',
'Glucose finger stick','Fingerstick Glucose','Glucose (serum)','Glucose (70-105)','Glucose (whole blood)',
'Present Weight  (lb)','Present Weight  (oz)','Present Weight  (kg)',
'Flowrate (cc/min)','Flowrate (l/m)',
'GCS - Motor Response', 'Motor Response',
'GCS - Verbal Response', 'Verbal Response',
'GCS - Eye Opening','Eye Opening',
'Minute Volume', 'Minute Volume Alarm - High',
'Tidal Volume (observed)', 'Tidal Volume (set)',
'Tidal Volume (spontaneous)', 'PEEP set','PEEP','PEEP Set',
'Central Venous Pressure  Alarm - Low','CVP','Central Venous Pressure Alarm - High','Central Venous Pressure',
'Sodium (serum)', 'Sodium', 'Sodium  (135-148)', 'Sodium (135-148)', 'Sodium (whole blood)',
'Temp Axillary [F]','Temp Rectal [F]',
'Temperature Fahrenheit', 'Skin [Temperature]', 
'Temperature C', 'Temperature C (calc)', 'Temperature F', 'Temperature F (calc)','Temp Skin [C]',
'Arterial O2 pressure', 'Arterial PaO2', 'Inspired O2 Fraction', 'FIO2', 'FIO2 Alarm [High]',
'FIO2 Alarm [Low]', 'FIO2 Alarm-High', 'FIO2 [Meas]',
'Paw High','Mean Airway Pressure',
'Previous WeightF','Previous Weight (kg)',
'Non Invasive Blood Pressure mean',
'Non Invasive Blood Pressure systolic',
'Non Invasive Blood Pressure diastolic',
'Heart Rate', 'Heart Rate Alarm - Low','Heart rate Alarm - High',
'Respiratory Rate (Set)',  'Respiratory Rate','Resp Alarm - High','Resp Alarm - Low',
'Respiratory Rate (Total)', 'Respiratory Rate (spontaneous)','Resp Rate','Resp Rate (Total)',
'O2 saturation pulseoxymetry','O2 Saturation Pulseoxymetry Alarm - Low', 'O2 Saturation Pulseoxymetry Alarm - High','SpO2','SpO2 Desat Limit',
'Arterial Blood Pressure systolic', 'Arterial Blood Pressure diastolic','Arterial Blood Pressure mean','Arterial BP Mean',
'BUN', 'BUN    (6-20)', 'BUN (6-20)'
]

In [162]:
selcols=unique(selcols)

In [163]:
selcols.append('Non-Invasive Blood Pressure mean')
selcols.append('High Insp. Pressure')
selcols.append('Fingerstick Glucose')
selcols.append('Present Weight  (kg)')
selcols.append('Flowrate (l/m)')
selcols=unique(selcols)
charts['LABEL'] = charts['LABEL'].astype('category')
charts['LABEL'].cat.set_categories(selcols, inplace=True) 

C:\Users\Mariam\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arrays\categorical.py:2747: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


In [164]:
charts[charts.LABEL.isin(['Peak Insp. Pressure','High Insp. Pressure'])]
charts['LABEL'].mask(charts['LABEL']== 'Peak Insp. Pressure','High Insp. Pressure', inplace=True)
charts[charts.LABEL.isin(['High Insp. Pressure'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
19495151,34,144319,290505.0,224695,2191-02-23 07:31:00,2191-02-23 07:35:00,16924.0,18.0,18.0,cmH2O,0.0,0.0,NaN,NaN,High Insp. Pressure,Respiratory
19495152,34,144319,290505.0,224695,2191-02-23 11:00:00,2191-02-23 11:04:00,14913.0,10.0,10.0,cmH2O,0.0,0.0,NaN,NaN,High Insp. Pressure,Respiratory
19495153,36,165660,241249.0,224695,2134-05-13 16:00:00,2134-05-13 16:28:00,20290.0,19.0,19.0,cmH2O,0.0,0.0,NaN,NaN,High Insp. Pressure,Respiratory
19495154,36,165660,241249.0,224695,2134-05-13 20:00:00,2134-05-13 20:28:00,20111.0,18.0,18.0,cmH2O,0.0,0.0,NaN,NaN,High Insp. Pressure,Respiratory
19495155,36,165660,241249.0,224695,2134-05-12 16:00:00,2134-05-12 16:28:00,18428.0,24.0,24.0,cmH2O,0.0,0.0,NaN,NaN,High Insp. Pressure,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56292492,12772,118571,232874.0,535,2163-08-23 19:00:00,2163-08-23 18:46:00,20803.0,22,22.0,cmH2O,NaN,NaN,NaN,NotStopd,High Insp. Pressure,NaN
56292493,12772,118571,232874.0,535,2163-08-30 22:45:00,2163-08-30 22:52:00,18999.0,25,25.0,cmH2O,NaN,NaN,NaN,NotStopd,High Insp. Pressure,NaN
56292494,12772,118571,232874.0,535,2163-09-01 00:00:00,2163-08-31 23:33:00,17443.0,26,26.0,cmH2O,NaN,NaN,NaN,NotStopd,High Insp. Pressure,NaN
56292495,12772,118571,232874.0,535,2163-08-12 23:00:00,2163-08-12 23:10:00,20111.0,36,36.0,cmH2O,NaN,NaN,NaN,NotStopd,High Insp. Pressure,NaN


In [165]:
charts[charts.LABEL.isin(['Potassium (whole blood)', 'Potassium  (3.5-5.3)', 'Potassium (3.5-5.3)', 'Potassium (serum)'])]
charts['LABEL'].mask(charts['LABEL']== 'Potassium  (3.5-5.3)', 'Potassium (whole blood)', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Potassium (3.5-5.3)', 'Potassium (whole blood)', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Potassium (serum)', 'Potassium (whole blood)', inplace=True)
#cd.VALUEUOM.fillna('mmHg', inplace=True) # this is not necessary as all have the same unit (mmHg)
charts[charts.LABEL.isin(['Potassium (whole blood)'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
29092060,23,124321,234044.0,227442,2157-10-21 14:40:00,2157-10-21 15:53:00,20889.0,3.9,3.9,mEq/L,0.0,0.0,NaN,NaN,Potassium (whole blood),Labs
29092061,23,124321,234044.0,227442,2157-10-22 03:21:00,2157-10-22 04:12:00,20889.0,3.4,3.4,mEq/L,0.0,0.0,NaN,NaN,Potassium (whole blood),Labs
29092062,36,165660,241249.0,227442,2134-05-12 08:33:00,2134-05-12 10:01:00,20889.0,4.2,4.2,mEq/L,0.0,0.0,NaN,NaN,Potassium (whole blood),Labs
29092063,34,144319,290505.0,227442,2191-02-23 10:48:00,2191-02-23 12:23:00,20889.0,4.6,4.6,mEq/L,0.0,0.0,NaN,NaN,Potassium (whole blood),Labs
29092064,34,144319,290505.0,227442,2191-02-24 04:32:00,2191-02-24 05:41:00,20889.0,3.9,3.9,mEq/L,0.0,0.0,NaN,NaN,Potassium (whole blood),Labs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59388572,12680,196168,281683.0,3792,2115-08-16 01:30:00,2115-08-16 03:06:00,15331.0,5.4,5.4,NaN,NaN,NaN,Final,NotStopd,Potassium (whole blood),Chemistry
59388573,12680,196168,281683.0,3792,2115-08-31 01:35:00,2115-08-31 02:41:00,15331.0,4.4,4.4,NaN,NaN,NaN,Final,NotStopd,Potassium (whole blood),Chemistry
59388574,12680,196168,281683.0,3792,2115-08-29 01:15:00,2115-08-29 02:59:00,15331.0,5.1,5.1,NaN,NaN,NaN,Final,NotStopd,Potassium (whole blood),Chemistry
59388575,12680,196168,281683.0,3792,2115-09-10 04:45:00,2115-09-10 06:25:00,15331.0,4.4,4.4,NaN,NaN,NaN,Final,NotStopd,Potassium (whole blood),Chemistry


In [166]:
charts[charts.LABEL.isin(['Non Invasive Blood Pressure diastolic','Non-Invasive Blood Pressure Alarm - High','Non-Invasive Blood Pressure Alarm - Low','Non Invasive Blood Pressure systolic'])]
charts['LABEL'].mask(charts['LABEL']== 'Non Invasive Blood Pressure diastolic', 'Non Invasive Blood Pressure mean',inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Non-Invasive Blood Pressure Alarm - High', 'Non-Invasive Blood Pressure mean',inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Non-Invasive Blood Pressure Alarm - Low', 'Non-Invasive Blood Pressure mean',inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Non Invasive Blood Pressure systolic','Non-Invasive Blood Pressure mean',inplace=True)
charts[charts.LABEL.isin(['Non-Invasive Blood Pressure mean'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
4432558,36,165660,241249.0,220179,2134-05-12 13:00:00,2134-05-12 13:55:00,17525.0,137.0,137.0,mmHg,0.0,0.0,NaN,NaN,Non-Invasive Blood Pressure mean,Routine Vital Signs
4432559,36,165660,241249.0,220179,2134-05-12 14:00:00,2134-05-12 14:39:00,17525.0,118.0,118.0,mmHg,0.0,0.0,NaN,NaN,Non-Invasive Blood Pressure mean,Routine Vital Signs
4432560,36,165660,241249.0,220179,2134-05-12 15:00:00,2134-05-12 16:50:00,17525.0,130.0,130.0,mmHg,0.0,0.0,NaN,NaN,Non-Invasive Blood Pressure mean,Routine Vital Signs
4432561,36,165660,241249.0,220179,2134-05-12 16:00:00,2134-05-12 16:50:00,17525.0,117.0,117.0,mmHg,0.0,0.0,NaN,NaN,Non-Invasive Blood Pressure mean,Routine Vital Signs
4432562,34,144319,290505.0,220179,2191-02-23 07:34:00,2191-02-23 07:45:00,17741.0,135.0,135.0,mmHg,0.0,0.0,NaN,NaN,Non-Invasive Blood Pressure mean,Routine Vital Signs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21863845,99230,158319,285123.0,223752,2131-01-25 19:32:00,2131-01-25 19:33:00,19169.0,90,90.0,mmHg,0.0,0.0,NaN,NaN,Non-Invasive Blood Pressure mean,Alarms
21863846,99166,117872,242196.0,223752,2154-02-27 08:47:00,2154-02-27 08:48:00,21189.0,90,90.0,mmHg,0.0,0.0,NaN,NaN,Non-Invasive Blood Pressure mean,Alarms
21863847,99260,181010,206662.0,223752,2190-12-24 08:03:00,2190-12-24 08:04:00,20088.0,90,90.0,mmHg,0.0,0.0,NaN,NaN,Non-Invasive Blood Pressure mean,Alarms
21863848,99260,181010,206662.0,223752,2190-12-27 08:41:00,2190-12-27 08:41:00,21050.0,90,90.0,mmHg,0.0,0.0,NaN,NaN,Non-Invasive Blood Pressure mean,Alarms


In [167]:
for i in ['Hematocrit (serum)','Hematocrit (whole blood - calc)','Fingerstick Glucose']:
    print(i in selcols)

True
True
True


In [168]:
charts[charts.LABEL.isin(['Hematocrit (serum)','Hematocrit (whole blood - calc)'])]
charts['LABEL'].mask(charts['LABEL']== 'Hematocrit (whole blood - calc)','Hematocrit (serum)', inplace=True)
charts[charts.LABEL.isin(['Hematocrit (serum)'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
27440284,23,124321,234044.0,220545,2157-10-21 14:40:00,2157-10-21 15:06:00,20889.0,36.9,36.9,%,1.0,0.0,NaN,NaN,Hematocrit (serum),Labs
27440285,23,124321,234044.0,220545,2157-10-22 03:21:00,2157-10-22 04:02:00,20889.0,38.0,38.0,%,1.0,0.0,NaN,NaN,Hematocrit (serum),Labs
27440286,36,165660,241249.0,220545,2134-05-12 08:33:00,2134-05-12 09:36:00,20889.0,36.3,36.3,%,1.0,0.0,NaN,NaN,Hematocrit (serum),Labs
27440287,34,144319,290505.0,220545,2191-02-23 10:48:00,2191-02-23 11:12:00,20889.0,35.9,35.9,%,1.0,0.0,NaN,NaN,Hematocrit (serum),Labs
27440288,34,144319,290505.0,220545,2191-02-24 04:32:00,2191-02-24 05:07:00,20889.0,30.7,30.7,%,1.0,0.0,NaN,NaN,Hematocrit (serum),Labs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32362328,99216,113331,275773.0,226540,2137-04-18 12:21:00,2137-04-18 12:25:00,20889.0,22,22.0,%,0.0,0.0,NaN,NaN,Hematocrit (serum),Labs
32362329,99165,199565,214080.0,226540,2190-11-08 10:52:00,2190-11-08 10:55:00,20889.0,25,25.0,%,0.0,0.0,NaN,NaN,Hematocrit (serum),Labs
32362330,99213,149621,260550.0,226540,2132-08-09 16:34:00,2132-08-09 16:38:00,20889.0,23,23.0,%,0.0,0.0,NaN,NaN,Hematocrit (serum),Labs
32362331,99166,184860,227031.0,226540,2154-02-01 07:54:00,2154-02-01 07:57:00,20889.0,30,30.0,%,0.0,0.0,NaN,NaN,Hematocrit (serum),Labs


In [169]:
charts[charts.LABEL.isin(['Glucose finger stick','Fingerstick Glucose','Glucose (serum)','Glucose (70-105)','Glucose (whole blood)'])]
charts['LABEL'].mask(charts['LABEL']== 'Glucose (serum)','Fingerstick Glucose', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Glucose finger stick','Fingerstick Glucose', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Glucose (70-105)','Fingerstick Glucose', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Glucose (whole blood)','Fingerstick Glucose', inplace=True)
charts[charts.LABEL.isin(['Fingerstick Glucose'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
21863850,23,124321,234044.0,225664,2157-10-22 10:00:00,2157-10-22 09:56:00,20009.0,188.0,188.0,NaN,0.0,0.0,NaN,NaN,Fingerstick Glucose,Labs
21863851,23,124321,234044.0,225664,2157-10-21 16:00:00,2157-10-21 16:38:00,16978.0,123.0,123.0,NaN,0.0,0.0,NaN,NaN,Fingerstick Glucose,Labs
21863852,23,124321,234044.0,225664,2157-10-22 00:00:00,2157-10-22 01:19:00,18063.0,105.0,105.0,NaN,0.0,0.0,NaN,NaN,Fingerstick Glucose,Labs
21863853,34,144319,290505.0,225664,2191-02-23 08:00:00,2191-02-23 08:12:00,19667.0,133.0,133.0,NaN,0.0,0.0,NaN,NaN,Fingerstick Glucose,Labs
21863854,36,165660,241249.0,225664,2134-05-13 12:00:00,2134-05-13 11:50:00,15225.0,92.0,92.0,NaN,0.0,0.0,NaN,NaN,Fingerstick Glucose,Labs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47926562,12761,171718,205774.0,811,2117-06-03 19:00:00,2117-06-03 19:09:00,17706.0,138,138.0,NaN,NaN,NaN,Manual,NotStopd,Fingerstick Glucose,Chemistry
47926563,12768,182007,224267.0,811,2154-03-12 21:51:00,2154-03-12 21:52:00,15331.0,140,140.0,NaN,NaN,NaN,Final,NotStopd,Fingerstick Glucose,Chemistry
47926564,12770,128996,221411.0,811,2132-01-25 03:30:00,2132-01-25 04:17:00,15331.0,143,143.0,NaN,NaN,NaN,Final,NotStopd,Fingerstick Glucose,Chemistry
47926565,12772,118571,232874.0,811,2163-09-04 16:11:00,2163-09-04 17:31:00,15331.0,153,153.0,NaN,NaN,NaN,Final,NotStopd,Fingerstick Glucose,Chemistry


In [170]:
charts[charts.LABEL.isin([ 'WBC   (4-11,000)', 'WBC (4-11,000)', 'WBC 4.0-11.0', 'WBC'])]
charts['LABEL'].mask(charts['LABEL']== 'WBC   (4-11,000)', 'WBC', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'WBC (4-11,000)', 'WBC', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'WBC 4.0-11.0', 'WBC', inplace=True)
charts[charts.LABEL.isin(['WBC'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
27606146,23,124321,234044.0,220546,2157-10-21 14:40:00,2157-10-21 15:06:00,20889.0,14.8,14.8,K/uL,1.0,0.0,NaN,NaN,WBC,Labs
27606147,23,124321,234044.0,220546,2157-10-22 03:21:00,2157-10-22 04:02:00,20889.0,16.4,16.4,K/uL,1.0,0.0,NaN,NaN,WBC,Labs
27606148,36,165660,241249.0,220546,2134-05-12 08:33:00,2134-05-12 09:36:00,20889.0,15.3,15.3,K/uL,1.0,0.0,NaN,NaN,WBC,Labs
27606149,34,144319,290505.0,220546,2191-02-23 10:48:00,2191-02-23 11:12:00,20889.0,7.1,7.1,K/uL,0.0,0.0,NaN,NaN,WBC,Labs
27606150,34,144319,290505.0,220546,2191-02-24 04:32:00,2191-02-24 05:07:00,20889.0,7.9,7.9,K/uL,0.0,0.0,NaN,NaN,WBC,Labs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60444162,12680,196168,281683.0,4200,2115-08-16 05:06:00,2115-08-16 05:56:00,15331.0,13.3,13.3,NaN,NaN,NaN,Final,NotStopd,WBC,Heme/Coag
60444163,12680,196168,281683.0,4200,2115-09-08 21:30:00,2115-09-09 01:24:00,15331.0,13.9,13.9,NaN,NaN,NaN,Final,NotStopd,WBC,Heme/Coag
60444164,12680,196168,281683.0,4200,2115-08-03 03:49:00,2115-08-03 04:04:00,15331.0,25.1,25.1,NaN,NaN,NaN,Final,NotStopd,WBC,Heme/Coag
60444165,12680,196168,281683.0,4200,2115-08-21 21:10:00,2115-08-21 22:57:00,15331.0,18.7,18.7,NaN,NaN,NaN,Final,NotStopd,WBC,Heme/Coag


In [171]:
charts[charts.LABEL.isin(['Present Weight  (lb)','Present Weight  (oz)','Present Weight  (kg)'])]
charts.loc[charts.LABEL== 'Present Weight  (oz)', 'VALUENUM'] = ((charts.VALUENUM) * 0.02834952)
charts.loc[charts.LABEL== 'Present Weight  (lb)', 'VALUENUM'] = ((charts.VALUENUM) * 0.45359237)


In [172]:
charts['LABEL'].mask(charts['LABEL']=='Present Weight  (lb)', 'Present Weight  (kg)', inplace=True)
charts['LABEL'].mask(charts['LABEL']=='Present Weight  (oz)', 'Present Weight  (kg)', inplace=True)
charts[charts.LABEL.isin(['Present Weight  (kg)'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
51163915,10,184167,288409.0,3580,2103-07-04 16:00:00,2103-07-04 16:43:00,14968.0,1.4149999618530273,1.415000,kg,NaN,NaN,NaN,NotStopd,Present Weight (kg),NaN
51163916,92,142807,291001.0,3580,2123-01-10 02:00:00,2123-01-10 01:36:00,17679.0,.9179999828338623,0.918000,kg,NaN,NaN,NaN,NotStopd,Present Weight (kg),NaN
51163917,92,142807,291001.0,3580,2123-01-11 09:00:00,2123-01-11 10:08:00,18433.0,.92500001192092896,0.925000,kg,NaN,NaN,NaN,NotStopd,Present Weight (kg),NaN
51163918,87,190659,277633.0,3580,2191-03-17 00:30:00,2191-03-17 01:24:00,20296.0,1.4700000286102295,1.470000,kg,NaN,NaN,NaN,NotStopd,Present Weight (kg),NaN
51163919,87,190659,277633.0,3580,2191-03-19 00:00:00,2191-03-19 01:51:00,15930.0,1.5499999523162842,1.550000,kg,NaN,NaN,NaN,NotStopd,Present Weight (kg),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51743651,12680,196168,281683.0,3582,2115-10-27 18:00:00,2115-10-27 18:25:00,14989.0,10.526300430297852,0.298416,cmH20,NaN,NaN,NaN,NotStopd,Present Weight (kg),NaN
51743652,12680,196168,281683.0,3582,2115-10-29 22:00:00,2115-10-29 21:42:00,17217.0,13.348199844360352,0.378415,cmH20,NaN,NaN,NaN,NotStopd,Present Weight (kg),NaN
51743653,12680,196168,281683.0,3582,2115-08-10 23:00:00,2115-08-10 22:23:00,20296.0,4.9667301177978516,0.140804,cmH20,NaN,NaN,NaN,NotStopd,Present Weight (kg),NaN
51743654,12680,196168,281683.0,3582,2115-09-15 13:00:00,2115-09-15 13:39:00,14989.0,5.6158699989318848,0.159207,cmH20,NaN,NaN,NaN,NotStopd,Present Weight (kg),NaN


In [173]:
charts[charts.LABEL.isin(['Flowrate (cc/min)','Flowrate (l/m)'])]
charts.loc[charts.LABEL== 'Flowrate (cc/min)', 'VALUENUM'] = ((charts.VALUENUM) *1000)
charts['LABEL'].mask(charts['LABEL']=='Flowrate (cc/min)','Flowrate (l/m)', inplace=True)
charts[charts.LABEL.isin(['Flowrate (l/m)'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
58309640,92,142807,291001.0,3437,2123-01-10 00:00:00,2123-01-10 00:13:00,15713.0,7,7.0,cmH20,NaN,NaN,NaN,NotStopd,Flowrate (l/m),NaN
58309641,92,142807,291001.0,3437,2123-01-10 01:00:00,2123-01-10 01:36:00,17679.0,7,7.0,cmH20,NaN,NaN,NaN,NotStopd,Flowrate (l/m),NaN
58309642,92,142807,291001.0,3437,2123-01-10 02:00:00,2123-01-10 02:34:00,15713.0,7,7.0,cmH20,NaN,NaN,NaN,NotStopd,Flowrate (l/m),NaN
58309643,92,142807,291001.0,3437,2123-01-11 09:00:00,2123-01-11 09:31:00,19412.0,6,6.0,cmH20,NaN,NaN,NaN,NotStopd,Flowrate (l/m),NaN
58309644,87,190659,277633.0,3437,2191-03-17 01:00:00,2191-03-17 01:23:00,20296.0,8,8.0,cmH20,NaN,NaN,NaN,NotStopd,Flowrate (l/m),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59381503,12680,196168,281683.0,3436,2115-10-06 11:00:00,2115-10-06 12:21:00,17894.0,600,600000.0,cc/min,NaN,NaN,NaN,NotStopd,Flowrate (l/m),NaN
59381504,12680,196168,281683.0,3436,2115-10-06 12:00:00,2115-10-06 12:43:00,17894.0,600,600000.0,cc/min,NaN,NaN,NaN,NotStopd,Flowrate (l/m),NaN
59381505,12680,196168,281683.0,3436,2115-09-17 21:00:00,2115-09-17 21:25:00,17217.0,50,50000.0,cc/min,NaN,NaN,NaN,NotStopd,Flowrate (l/m),NaN
59381506,12680,196168,281683.0,3436,2115-09-17 22:00:00,2115-09-17 22:43:00,17217.0,75,75000.0,cc/min,NaN,NaN,NaN,NotStopd,Flowrate (l/m),NaN


In [174]:
charts[charts.LABEL.isin(['O2 Flow (lpm)','O2 Flow'])]
charts['LABEL'].mask(charts['LABEL']=='O2 Flow (lpm)', 'O2 Flow', inplace=True)
charts[charts.LABEL.isin(['O2 Flow'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
0,36,165660,241249.0,223834,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,15.0,15.0,L/min,0.0,0.0,NaN,NaN,O2 Flow,Respiratory
1,23,124321,234044.0,223834,2157-10-21 20:00:00,2157-10-22 00:45:00,19050.0,3.0,3.0,L/min,0.0,0.0,NaN,NaN,O2 Flow,Respiratory
2,23,124321,234044.0,223834,2157-10-21 22:00:00,2157-10-22 00:45:00,19050.0,2.0,2.0,L/min,0.0,0.0,NaN,NaN,O2 Flow,Respiratory
3,23,124321,234044.0,223834,2157-10-22 00:00:00,2157-10-22 00:45:00,19050.0,2.0,2.0,L/min,0.0,0.0,NaN,NaN,O2 Flow,Respiratory
4,36,165660,241249.0,223834,2134-05-12 07:09:00,2134-05-12 07:09:00,17525.0,15.0,15.0,L/min,0.0,0.0,NaN,NaN,O2 Flow,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38919414,12769,158658,273623.0,470,2149-12-27 04:00:00,2149-12-27 04:21:00,18557.0,4,4.0,lpm,NaN,NaN,NaN,NotStopd,O2 Flow,NaN
38919415,12770,128996,221411.0,470,2132-01-26 03:00:00,2132-01-26 03:39:00,15627.0,2,2.0,lpm,NaN,NaN,NaN,NotStopd,O2 Flow,NaN
38919416,12763,111190,227170.0,470,2104-06-25 20:00:00,2104-06-25 20:06:00,19963.0,12,12.0,lpm,NaN,NaN,NaN,NotStopd,O2 Flow,NaN
38919417,12763,111190,227170.0,470,2104-06-27 07:00:00,2104-06-27 06:49:00,18653.0,2,2.0,lpm,NaN,NaN,NaN,NotStopd,O2 Flow,NaN


In [175]:
charts[charts.LABEL.isin(['GCS - Motor Response', 'Motor Response'])]
charts['LABEL'].mask(charts['LABEL']=='GCS - Motor Response', 'Motor Response', inplace=True)
charts[charts.LABEL.isin(['Motor Response'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
38383198,4,185777,294638.0,454,2191-03-16 08:00:00,2191-03-16 09:16:00,18692.0,6 Obeys Commands,6.0,NaN,NaN,NaN,NaN,NotStopd,Motor Response,NaN
38383199,13,143045,263738.0,454,2167-01-10 08:00:00,2167-01-10 09:08:00,20670.0,6 Obeys Commands,6.0,NaN,NaN,NaN,NaN,NotStopd,Motor Response,NaN
38383200,19,109235,273430.0,454,2108-08-06 13:00:00,2108-08-06 16:03:00,15521.0,6 Obeys Commands,6.0,NaN,NaN,NaN,NaN,NotStopd,Motor Response,NaN
38383201,3,145834,211552.0,454,2101-10-22 01:00:00,2101-10-22 01:47:00,21570.0,6 Obeys Commands,6.0,NaN,NaN,NaN,NaN,NotStopd,Motor Response,NaN
38383202,12,112213,232669.0,454,2104-08-13 19:00:00,2104-08-13 19:07:00,17609.0,5 Localizes Pain,5.0,NaN,NaN,NaN,NaN,NotStopd,Motor Response,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38768110,12763,111190,227170.0,454,2104-06-27 07:00:00,2104-06-27 06:49:00,18653.0,6 Obeys Commands,6.0,NaN,NaN,NaN,NaN,NotStopd,Motor Response,NaN
38768111,12764,172548,254003.0,454,2200-11-21 04:00:00,2200-11-21 03:40:00,16901.0,1 No Response,1.0,NaN,NaN,NaN,NaN,NotStopd,Motor Response,NaN
38768112,12772,118571,232874.0,454,2163-08-12 23:00:00,2163-08-12 23:18:00,15627.0,4 Flex-withdraws,4.0,NaN,NaN,NaN,NaN,NotStopd,Motor Response,NaN
38768113,12767,170690,205223.0,454,2189-12-27 09:00:00,2189-12-27 11:09:00,21570.0,4 Flex-withdraws,4.0,NaN,NaN,NaN,NaN,NotStopd,Motor Response,NaN


In [176]:
charts[charts.LABEL.isin(['GCS - Verbal Response', 'Verbal Response'])]
charts['LABEL'].mask(charts['LABEL']=='GCS - Verbal Response', 'Verbal Response', inplace=True)
charts[charts.LABEL.isin(['Verbal Response'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
42992816,4,185777,294638.0,723,2191-03-16 08:00:00,2191-03-16 09:16:00,18692.0,5 Oriented,5.0,NaN,NaN,NaN,NaN,NotStopd,Verbal Response,NaN
42992817,13,143045,263738.0,723,2167-01-10 08:00:00,2167-01-10 09:08:00,20670.0,5 Oriented,5.0,NaN,NaN,NaN,NaN,NotStopd,Verbal Response,NaN
42992818,28,162569,225559.0,723,2177-09-01 23:00:00,2177-09-01 23:26:00,17130.0,5 Oriented,5.0,NaN,NaN,NaN,NaN,NotStopd,Verbal Response,NaN
42992819,9,150750,220597.0,723,2149-11-11 18:00:00,2149-11-11 18:33:00,20992.0,1.0 ET/Trach,1.0,NaN,NaN,NaN,NaN,NotStopd,Verbal Response,NaN
42992820,19,109235,273430.0,723,2108-08-06 13:00:00,2108-08-06 16:03:00,15521.0,4 Confused,4.0,NaN,NaN,NaN,NaN,NotStopd,Verbal Response,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43378518,12752,107086,235949.0,723,2167-02-26 03:00:00,2167-02-26 03:12:00,16302.0,1 No Response,1.0,NaN,NaN,NaN,NaN,NotStopd,Verbal Response,NaN
43378519,12763,111190,227170.0,723,2104-06-27 07:00:00,2104-06-27 06:49:00,18653.0,5 Oriented,5.0,NaN,NaN,NaN,NaN,NotStopd,Verbal Response,NaN
43378520,12772,118571,232874.0,723,2163-09-07 23:00:00,2163-09-07 23:25:00,20034.0,1.0 ET/Trach,1.0,NaN,NaN,NaN,NaN,NotStopd,Verbal Response,NaN
43378521,12767,170690,205223.0,723,2189-12-27 08:00:00,2189-12-27 08:43:00,21570.0,1.0 ET/Trach,1.0,NaN,NaN,NaN,NaN,NotStopd,Verbal Response,NaN


In [177]:
charts[charts.LABEL.isin(['GCS - Eye Opening','Eye Opening'])]
charts['LABEL'].mask(charts['LABEL']=='GCS - Eye Opening','Eye Opening', inplace=True)
charts[charts.LABEL.isin(['Eye Opening'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
37379224,4,185777,294638.0,184,2191-03-16 08:00:00,2191-03-16 09:16:00,18692.0,4 Spontaneously,4.0,NaN,NaN,NaN,NaN,NotStopd,Eye Opening,NaN
37379225,6,107064,228232.0,184,2175-05-31 02:00:00,2175-05-31 02:16:00,15443.0,4 Spontaneously,4.0,NaN,NaN,NaN,NaN,NotStopd,Eye Opening,NaN
37379226,13,143045,263738.0,184,2167-01-11 05:00:00,2167-01-11 04:43:00,20971.0,4 Spontaneously,4.0,NaN,NaN,NaN,NaN,NotStopd,Eye Opening,NaN
37379227,9,150750,220597.0,184,2149-11-11 00:00:00,2149-11-11 00:32:00,16273.0,1 No Response,1.0,NaN,NaN,NaN,NaN,NotStopd,Eye Opening,NaN
37379228,19,109235,273430.0,184,2108-08-06 13:00:00,2108-08-06 16:03:00,15521.0,4 Spontaneously,4.0,NaN,NaN,NaN,NaN,NotStopd,Eye Opening,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37765925,12772,118571,232874.0,184,2163-08-12 23:00:00,2163-08-12 23:18:00,15627.0,2 To pain,2.0,NaN,NaN,NaN,NaN,NotStopd,Eye Opening,NaN
37765926,12767,170690,205223.0,184,2189-12-27 09:00:00,2189-12-27 10:18:00,21570.0,1 No Response,1.0,NaN,NaN,NaN,NaN,NotStopd,Eye Opening,NaN
37765927,12767,170690,205223.0,184,2189-12-28 12:00:00,2189-12-28 13:19:00,21570.0,1 No Response,1.0,NaN,NaN,NaN,NaN,NotStopd,Eye Opening,NaN
37765928,12772,118571,232874.0,184,2163-08-19 15:00:00,2163-08-19 14:54:00,20701.0,3 To speech,3.0,NaN,NaN,NaN,NaN,NotStopd,Eye Opening,NaN


In [178]:
charts[charts.LABEL.isin(['Minute Volume', 'Minute Volume Alarm - High','Minute Volume Alarm - Low','Minute Volume(Obser)'])]
charts['LABEL'].mask(charts['LABEL']=='Minute Volume(Obser)','Minute Volume', inplace=True)
charts['LABEL'].mask(charts['LABEL']=='Minute Volume Alarm - Low','Minute Volume', inplace=True)
charts['LABEL'].mask(charts['LABEL']=='Minute Volume Alarm - High','Minute Volume', inplace=True)
charts[charts.LABEL.isin(['Minute Volume'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
15432582,34,144319,290505.0,220292,2191-02-23 07:31:00,2191-02-23 07:36:00,16924.0,4.0,4.00,L/min,0.0,0.0,NaN,NaN,Minute Volume,Alarms
15432583,34,144319,290505.0,220292,2191-02-23 11:00:00,2191-02-23 11:04:00,14913.0,3.0,3.00,L/min,0.0,0.0,NaN,NaN,Minute Volume,Alarms
15432584,36,165660,241249.0,220292,2134-05-13 16:00:00,2134-05-13 16:28:00,20290.0,4.0,4.00,L/min,0.0,0.0,NaN,NaN,Minute Volume,Alarms
15432585,36,165660,241249.0,220292,2134-05-13 20:00:00,2134-05-13 20:29:00,20111.0,4.0,4.00,L/min,0.0,0.0,NaN,NaN,Minute Volume,Alarms
15432586,36,165660,241249.0,220292,2134-05-12 16:13:00,2134-05-12 16:29:00,18428.0,4.0,4.00,L/min,0.0,0.0,NaN,NaN,Minute Volume,Alarms
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61038624,2561,130654,256907.0,448,2166-09-08 14:00:00,2166-09-08 14:18:00,21570.0,8.8000001907348633,8.80,L/min,NaN,NaN,NaN,NotStopd,Minute Volume,NaN
61038625,7666,117716,231014.0,448,2154-06-04 16:00:00,2154-06-04 16:41:00,20389.0,18.75,18.75,L/min,NaN,NaN,NaN,NotStopd,Minute Volume,NaN
61038626,8832,179467,216465.0,448,2133-11-13 13:30:00,2133-11-13 13:51:00,15225.0,13.800000190734863,13.80,L/min,NaN,NaN,NaN,NotStopd,Minute Volume,NaN
61038627,10434,124064,287880.0,448,2189-04-08 07:00:00,2189-04-08 10:18:00,15805.0,6,6.00,L/min,NaN,NaN,NaN,NotStopd,Minute Volume,NaN


In [179]:
charts[charts.LABEL.isin(['Tidal Volume (observed)', 'Tidal Volume (set)',
'Tidal Volume (spontaneous)'])]
charts['LABEL'].mask(charts['LABEL']=='Tidal Volume (set)','Tidal Volume (observed)', inplace=True)
charts['LABEL'].mask(charts['LABEL']=='Tidal Volume (spontaneous)','Tidal Volume (observed)', inplace=True)
charts[charts.LABEL.isin(['Tidal Volume (observed)'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
17704735,34,144319,290505.0,224684,2191-02-23 07:31:00,2191-02-23 07:35:00,16924.0,450.0,450.0,mL,0.0,0.0,NaN,NaN,Tidal Volume (observed),Respiratory
17704736,36,165660,241249.0,224684,2134-05-13 21:00:00,2134-05-13 20:40:00,20111.0,500.0,500.0,mL,1.0,0.0,NaN,NaN,Tidal Volume (observed),Respiratory
17704737,36,165660,241249.0,224684,2134-05-12 16:00:00,2134-05-12 16:28:00,18428.0,500.0,500.0,mL,0.0,0.0,NaN,NaN,Tidal Volume (observed),Respiratory
17704738,36,165660,241249.0,224684,2134-05-12 21:00:00,2134-05-13 01:09:00,17698.0,500.0,500.0,mL,0.0,0.0,NaN,NaN,Tidal Volume (observed),Respiratory
17704739,36,165660,241249.0,224684,2134-05-13 01:00:00,2134-05-13 01:12:00,17698.0,500.0,500.0,mL,0.0,0.0,NaN,NaN,Tidal Volume (observed),Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30288240,99260,181010,206662.0,224686,2190-12-29 20:00:00,2190-12-29 20:23:00,17698.0,547,547.0,mL,0.0,0.0,NaN,NaN,Tidal Volume (observed),Respiratory
30288241,99260,181010,206662.0,224686,2191-01-01 12:00:00,2191-01-01 12:47:00,19510.0,422,422.0,mL,0.0,0.0,NaN,NaN,Tidal Volume (observed),Respiratory
30288242,99288,179982,221552.0,224686,2194-04-22 15:00:00,2194-04-22 16:12:00,17443.0,461,461.0,mL,0.0,0.0,NaN,NaN,Tidal Volume (observed),Respiratory
30288243,99186,111707,274991.0,224686,2183-10-02 20:00:00,2183-10-02 20:17:00,19747.0,367,367.0,mL,0.0,0.0,NaN,NaN,Tidal Volume (observed),Respiratory


In [180]:
charts[charts.LABEL.isin(['PEEP set','PEEP','PEEP Set'])]
charts['LABEL'].mask(charts['LABEL']=='PEEP set','PEEP', inplace=True)
charts['LABEL'].mask(charts['LABEL']=='PEEP Set','PEEP', inplace=True)
charts[charts.LABEL.isin(['PEEP'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
13714768,36,165660,241249.0,220339,2134-05-12 16:00:00,2134-05-12 16:28:00,18428.0,5.0,5.0,cmH2O,0.0,0.0,NaN,NaN,PEEP,Respiratory
13714769,34,144319,290505.0,220339,2191-02-23 07:31:00,2191-02-23 07:35:00,16924.0,5.0,5.0,cmH2O,0.0,0.0,NaN,NaN,PEEP,Respiratory
13714770,34,144319,290505.0,220339,2191-02-23 11:00:00,2191-02-23 11:04:00,14913.0,5.0,5.0,cmH2O,0.0,0.0,NaN,NaN,PEEP,Respiratory
13714771,36,165660,241249.0,220339,2134-05-13 16:00:00,2134-05-13 16:28:00,20290.0,8.0,8.0,cmH2O,0.0,0.0,NaN,NaN,PEEP,Respiratory
13714772,36,165660,241249.0,220339,2134-05-13 20:00:00,2134-05-13 20:39:00,20111.0,10.0,10.0,cmH2O,1.0,0.0,NaN,NaN,PEEP,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57680191,12680,196168,281683.0,505,2115-09-07 15:00:00,2115-09-07 15:06:00,14948.0,6,6.0,cmH20,NaN,NaN,NaN,NotStopd,PEEP,NaN
57680192,12680,196168,281683.0,505,2115-09-01 17:00:00,2115-09-01 17:22:00,16446.0,7,7.0,cmH20,NaN,NaN,NaN,NotStopd,PEEP,NaN
57680193,12680,196168,281683.0,505,2115-08-29 22:00:00,2115-08-29 21:53:00,20809.0,7,7.0,cmH20,NaN,NaN,NaN,NotStopd,PEEP,NaN
57680194,12680,196168,281683.0,505,2115-09-05 08:00:00,2115-09-05 09:40:00,16446.0,6,6.0,cmH20,NaN,NaN,NaN,NotStopd,PEEP,NaN


In [181]:
charts[charts.LABEL.isin(['Central Venous Pressure  Alarm - Low','CVP','Central Venous Pressure Alarm - High','Central Venous Pressure',])]
charts['LABEL'].mask(charts['LABEL']=='CVP','Central Venous Pressure', inplace=True)
charts['LABEL'].mask(charts['LABEL']=='Central Venous Pressure  Alarm - Low','Central Venous Pressure', inplace=True)
charts['LABEL'].mask(charts['LABEL']=='Central Venous Pressure Alarm - High','Central Venous Pressure', inplace=True)
charts[charts.LABEL.isin(['Central Venous Pressure'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
25567181,34,144319,290505.0,220074,2191-02-23 05:25:00,2191-02-23 07:45:00,17741.0,7.0,7.0,mmHg,0.0,0.0,NaN,NaN,Central Venous Pressure,Hemodynamics
25567182,111,155897,249202.0,220074,2144-07-01 11:26:00,2144-07-01 11:28:00,19562.0,8.0,8.0,mmHg,1.0,0.0,NaN,NaN,Central Venous Pressure,Hemodynamics
25567183,124,138376,256064.0,220074,2166-01-16 13:00:00,2166-01-16 13:37:00,18602.0,9.0,9.0,mmHg,0.0,0.0,NaN,NaN,Central Venous Pressure,Hemodynamics
25567184,124,138376,256064.0,220074,2166-01-16 14:00:00,2166-01-16 16:05:00,18602.0,12.0,12.0,mmHg,0.0,0.0,NaN,NaN,Central Venous Pressure,Hemodynamics
25567185,124,138376,256064.0,220074,2166-01-16 16:00:00,2166-01-16 16:05:00,18602.0,15.0,15.0,mmHg,0.0,0.0,NaN,NaN,Central Venous Pressure,Hemodynamics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47301099,12769,193014,217725.0,113,2151-01-05 09:30:00,2151-01-05 09:49:00,15767.0,4,4.0,mmHg,NaN,NaN,NaN,NotStopd,Central Venous Pressure,NaN
47301100,12772,118571,232874.0,113,2163-08-27 23:00:00,2163-08-27 23:19:00,18565.0,12,12.0,mmHg,NaN,NaN,NaN,NotStopd,Central Venous Pressure,NaN
47301101,12772,118571,232874.0,113,2163-08-28 23:00:00,2163-08-28 23:17:00,18565.0,10,10.0,mmHg,NaN,NaN,NaN,NotStopd,Central Venous Pressure,NaN
47301102,12761,171718,205774.0,113,2117-06-05 15:00:00,2117-06-05 15:20:00,20868.0,18,18.0,mmHg,NaN,NaN,NaN,NotStopd,Central Venous Pressure,NaN


In [182]:
charts[charts.LABEL.isin(['Sodium (serum)', 'Sodium', 'Sodium  (135-148)', 'Sodium (135-148)', 'Sodium (whole blood)'])]
charts['LABEL'].mask(charts['LABEL']== 'Sodium', 'Sodium (serum)', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Sodium  (135-148)', 'Sodium (serum)', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Sodium (135-148)', 'Sodium (serum)', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Sodium (whole blood)', 'Sodium (serum)', inplace=True)
charts[charts.LABEL.isin(['Sodium (serum)'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
28356212,23,124321,234044.0,220645,2157-10-21 14:40:00,2157-10-21 15:53:00,20889.0,144.0,144.0,mEq/L,0.0,0.0,NaN,NaN,Sodium (serum),Labs
28356213,23,124321,234044.0,220645,2157-10-22 03:21:00,2157-10-22 04:12:00,20889.0,144.0,144.0,mEq/L,0.0,0.0,NaN,NaN,Sodium (serum),Labs
28356214,36,165660,241249.0,220645,2134-05-12 08:33:00,2134-05-12 10:01:00,20889.0,139.0,139.0,mEq/L,0.0,0.0,NaN,NaN,Sodium (serum),Labs
28356215,34,144319,290505.0,220645,2191-02-23 10:48:00,2191-02-23 12:23:00,20889.0,139.0,139.0,mEq/L,0.0,0.0,NaN,NaN,Sodium (serum),Labs
28356216,34,144319,290505.0,220645,2191-02-24 04:32:00,2191-02-24 05:41:00,20889.0,142.0,142.0,mEq/L,0.0,0.0,NaN,NaN,Sodium (serum),Labs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395624,12680,196168,281683.0,3803,2115-08-16 01:30:00,2115-08-16 03:06:00,15331.0,134,134.0,NaN,NaN,NaN,Final,NotStopd,Sodium (serum),Chemistry
59395625,12680,196168,281683.0,3803,2115-08-31 01:35:00,2115-08-31 02:41:00,15331.0,134,134.0,NaN,NaN,NaN,Final,NotStopd,Sodium (serum),Chemistry
59395626,12680,196168,281683.0,3803,2115-08-29 01:15:00,2115-08-29 02:59:00,15331.0,129,129.0,NaN,NaN,NaN,Final,NotStopd,Sodium (serum),Chemistry
59395627,12680,196168,281683.0,3803,2115-09-10 04:45:00,2115-09-10 06:25:00,15331.0,133,133.0,NaN,NaN,NaN,Final,NotStopd,Sodium (serum),Chemistry


In [183]:
charts[charts.LABEL.isin(['Temp Axillary [F]','Temp Rectal [F]',
                          'Temperature Fahrenheit', 'Skin [Temperature]', 'Temperature Celsius',
                          'Temperature C', 'Temperature C (calc)', 'Temperature F', 'Temperature F (calc)','Temp Skin [C]'])]
charts.loc[charts.LABEL== 'Temperature Fahrenheit', 'VALUENUM'] = ((charts.VALUENUM -32) * 5./9)
charts.loc[charts.LABEL== 'Temperature F', 'VALUENUM'] = ((charts.VALUENUM -32) * 5./9)
charts.loc[charts.LABEL== 'Temperature F (calc)', 'VALUENUM'] = ((charts.VALUENUM -32) * 5./9)
charts.loc[charts.LABEL== 'Temp Axillary [F]', 'VALUENUM'] = ((charts.VALUENUM -32) * 5./9)
charts.loc[charts.LABEL== 'Temp Rectal [F]', 'VALUENUM'] = ((charts.VALUENUM -32) * 5./9)
charts['LABEL'].mask(charts['LABEL']== 'Temperature Fahrenheit', 'Temperature C', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Temperature F', 'Temperature C', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Temperature F (calc)', 'Temperature C', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Temp Skin [C]', 'Temperature C', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Temperature C (calc)', 'Temperature C', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Temp Axillary [F]', 'Temperature C', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Temp Rectal [F]', 'Temperature C', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Temperature Celsius', 'Temperature C', inplace=True)
charts[charts.LABEL.isin(['Temperature C'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
14195331,36,165660,241249.0,223761,2134-05-12 16:00:00,2134-05-12 16:51:00,17525.0,97.3,36.277778,?F,0.0,0.0,NaN,NaN,Temperature C,Routine Vital Signs
14195332,34,144319,290505.0,223761,2191-02-23 08:00:00,2191-02-23 08:23:00,17741.0,97.0,36.111111,?F,0.0,0.0,NaN,NaN,Temperature C,Routine Vital Signs
14195333,23,124321,234044.0,223761,2157-10-22 16:00:00,2157-10-22 15:53:00,19907.0,98.4,36.888889,?F,0.0,0.0,NaN,NaN,Temperature C,Routine Vital Signs
14195334,23,124321,234044.0,223761,2157-10-21 12:05:00,2157-10-21 12:05:00,16978.0,95.8,35.444444,?F,0.0,0.0,NaN,NaN,Temperature C,Routine Vital Signs
14195335,23,124321,234044.0,223761,2157-10-21 14:00:00,2157-10-21 13:56:00,16978.0,97.2,36.222222,?F,0.0,0.0,NaN,NaN,Temperature C,Routine Vital Signs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60223265,12747,121680,284292.0,3654,2143-05-05 04:00:00,2143-05-05 05:07:00,14872.0,101.19999694824219,38.444443,Deg F,NaN,NaN,NaN,NotStopd,Temperature C,NaN
60223266,12747,121680,284292.0,3654,2143-05-16 03:00:00,2143-05-16 03:19:00,15918.0,98.900001525878906,37.166668,Deg F,NaN,NaN,NaN,NotStopd,Temperature C,NaN
60223267,12680,196168,281683.0,3654,2115-10-29 10:00:00,2115-10-29 10:12:00,16133.0,98.599998474121094,36.999999,Deg F,NaN,NaN,NaN,NotStopd,Temperature C,NaN
60223268,12750,130131,296088.0,3654,2141-01-19 12:45:00,2141-01-19 15:14:00,14989.0,96.599998474121094,35.888888,Deg F,NaN,NaN,NaN,NotStopd,Temperature C,NaN


In [184]:
charts[charts.LABEL.isin(['Arterial O2 pressure', 'Arterial PaO2', 'Inspired O2 Fraction', 'FIO2', 'FIO2 Alarm [High]',
                          'FIO2 Alarm [Low]', 'FIO2 Alarm-High', 'FIO2 [Meas]'])]
charts['LABEL'].mask(charts['LABEL']== 'Arterial O2 pressure', 'Inspired O2 Fraction', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Arterial PaO2', 'Inspired O2 Fraction', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'FIO2', 'Inspired O2 Fraction', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'FIO2 Alarm [High]', 'Inspired O2 Fraction', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'FIO2 Alarm [Low]', 'Inspired O2 Fraction', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'FIO2 Alarm-High', 'Inspired O2 Fraction', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'FIO2 [Meas]', 'Inspired O2 Fraction', inplace=True)
charts[charts.LABEL.isin(['Inspired O2 Fraction'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
202279,36,165660,241249.0,223835,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,100.0,100.0,NaN,0.0,0.0,NaN,NaN,Inspired O2 Fraction,Respiratory
202280,34,144319,290505.0,223835,2191-02-23 07:31:00,2191-02-23 07:35:00,16924.0,60.0,60.0,NaN,0.0,0.0,NaN,NaN,Inspired O2 Fraction,Respiratory
202281,36,165660,241249.0,223835,2134-05-12 07:09:00,2134-05-12 07:09:00,17525.0,100.0,100.0,NaN,0.0,0.0,NaN,NaN,Inspired O2 Fraction,Respiratory
202282,34,144319,290505.0,223835,2191-02-23 11:00:00,2191-02-23 11:04:00,14913.0,60.0,60.0,NaN,0.0,0.0,NaN,NaN,Inspired O2 Fraction,Respiratory
202283,36,165660,241249.0,223835,2134-05-13 16:00:00,2134-05-13 16:27:00,20290.0,50.0,50.0,NaN,0.0,0.0,NaN,NaN,Inspired O2 Fraction,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60719700,12727,169047,224510.0,185,2142-08-22 16:00:00,2142-08-22 16:02:00,17738.0,60,60.0,decFrc,NaN,NaN,NaN,NotStopd,Inspired O2 Fraction,NaN
60719701,12727,169047,224510.0,185,2142-08-21 19:00:00,2142-08-21 19:13:00,15411.0,70,70.0,decFrc,NaN,NaN,NaN,NotStopd,Inspired O2 Fraction,NaN
60719702,12727,169047,224510.0,185,2142-08-25 11:00:00,2142-08-25 11:09:00,19585.0,100,100.0,decFrc,NaN,NaN,NaN,NotStopd,Inspired O2 Fraction,NaN
60719703,12760,162126,256979.0,185,2136-10-19 04:50:00,2136-10-19 05:10:00,19653.0,45,45.0,decFrc,NaN,NaN,NaN,NotStopd,Inspired O2 Fraction,NaN


In [185]:
charts[charts.LABEL.isin(['Paw High','Mean Airway Pressure'])]
charts['LABEL'].mask(charts['LABEL']== 'Paw High','Mean Airway Pressure', inplace=True)
charts[charts.LABEL.isin(['Mean Airway Pressure'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
16345327,34,144319,290505.0,223873,2191-02-23 07:31:00,2191-02-23 07:36:00,16924.0,35.0,35.0,cmH2O,0.0,0.0,NaN,NaN,Mean Airway Pressure,Respiratory
16345328,34,144319,290505.0,223873,2191-02-23 11:00:00,2191-02-23 11:04:00,14913.0,35.0,35.0,cmH2O,0.0,0.0,NaN,NaN,Mean Airway Pressure,Respiratory
16345329,36,165660,241249.0,223873,2134-05-13 16:00:00,2134-05-13 16:28:00,20290.0,35.0,35.0,cmH2O,0.0,0.0,NaN,NaN,Mean Airway Pressure,Respiratory
16345330,36,165660,241249.0,223873,2134-05-13 20:00:00,2134-05-13 20:29:00,20111.0,35.0,35.0,cmH2O,0.0,0.0,NaN,NaN,Mean Airway Pressure,Respiratory
16345331,36,165660,241249.0,223873,2134-05-12 16:13:00,2134-05-12 16:29:00,18428.0,40.0,40.0,cmH2O,0.0,0.0,NaN,NaN,Mean Airway Pressure,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50382490,12772,118571,232874.0,444,2163-08-30 22:45:00,2163-08-30 22:52:00,18999.0,12,12.0,cmH2O,NaN,NaN,NaN,NotStopd,Mean Airway Pressure,NaN
50382491,12772,118571,232874.0,444,2163-09-01 00:00:00,2163-08-31 23:33:00,17443.0,14,14.0,cmH2O,NaN,NaN,NaN,NotStopd,Mean Airway Pressure,NaN
50382492,12772,118571,232874.0,444,2163-08-12 23:00:00,2163-08-12 23:10:00,20111.0,15,15.0,cmH2O,NaN,NaN,NaN,NotStopd,Mean Airway Pressure,NaN
50382493,12767,170690,205223.0,444,2189-12-27 09:00:00,2189-12-27 08:59:00,15597.0,7.4000000953674316,7.4,cmH2O,NaN,NaN,NaN,NotStopd,Mean Airway Pressure,NaN


In [186]:
charts[charts.LABEL.isin(['Previous WeightF','Previous Weight (kg)'])]
charts['LABEL'].mask(charts['LABEL']== 'Previous WeightF','Previous Weight (kg)', inplace=True)
charts[charts.LABEL.isin(['Previous Weight (kg)'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
38938571,4,185777,294638.0,581,2191-03-16 08:00:00,2191-03-16 09:16:00,18692.0,53.599998474121094,53.599998,kg,NaN,NaN,NaN,NotStopd,Previous Weight (kg),NaN
38938572,9,150750,220597.0,581,2149-11-10 23:45:00,2149-11-11 00:11:00,16273.0,104,104.000000,kg,NaN,NaN,NaN,NotStopd,Previous Weight (kg),NaN
38938573,9,150750,220597.0,581,2149-11-11 18:00:00,2149-11-11 18:33:00,20992.0,100.30000305175781,100.300003,kg,NaN,NaN,NaN,NotStopd,Previous Weight (kg),NaN
38938574,9,150750,220597.0,581,2149-11-11 19:00:00,2149-11-11 18:59:00,20992.0,100.30000305175781,100.300003,kg,NaN,NaN,NaN,NotStopd,Previous Weight (kg),NaN
38938575,9,150750,220597.0,581,2149-11-11 19:45:00,2149-11-11 19:43:00,15025.0,100.30000305175781,100.300003,kg,NaN,NaN,NaN,NotStopd,Previous Weight (kg),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51930085,12680,196168,281683.0,3583,2115-10-27 18:00:00,2115-10-27 18:25:00,14989.0,2.9600000381469727,2.960000,kg,NaN,NaN,NaN,NotStopd,Previous Weight (kg),NaN
51930086,12680,196168,281683.0,3583,2115-10-29 22:00:00,2115-10-29 21:42:00,17217.0,3.0750000476837158,3.075000,kg,NaN,NaN,NaN,NotStopd,Previous Weight (kg),NaN
51930087,12680,196168,281683.0,3583,2115-08-10 23:00:00,2115-08-10 22:23:00,20296.0,1.0069999694824219,1.007000,kg,NaN,NaN,NaN,NotStopd,Previous Weight (kg),NaN
51930088,12680,196168,281683.0,3583,2115-09-15 13:00:00,2115-09-15 13:39:00,14989.0,1.5149999856948853,1.515000,kg,NaN,NaN,NaN,NotStopd,Previous Weight (kg),NaN


In [187]:
charts[charts.LABEL.isin([ 'Non Invasive Blood Pressure mean','Non Invasive Blood Pressure systolic','Non Invasive Blood Pressure diastolic'])]
charts['LABEL'].mask(charts['LABEL']==   'Non Invasive Blood Pressure diastolic', 'Non Invasive Blood Pressure mean', inplace=True)
charts['LABEL'].mask(charts['LABEL']==   'Non Invasive Blood Pressure systolic', 'Non Invasive Blood Pressure mean', inplace=True)
charts[charts.LABEL.isin(['Non Invasive Blood Pressure mean'])]
    

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
5723046,36,165660,241249.0,220180,2134-05-12 13:00:00,2134-05-12 13:55:00,17525.0,72.0,72.0,mmHg,0.0,0.0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
5723047,36,165660,241249.0,220180,2134-05-12 14:00:00,2134-05-12 14:39:00,17525.0,53.0,53.0,mmHg,0.0,0.0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
5723048,36,165660,241249.0,220180,2134-05-12 15:00:00,2134-05-12 16:50:00,17525.0,61.0,61.0,mmHg,0.0,0.0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
5723049,36,165660,241249.0,220180,2134-05-12 16:00:00,2134-05-12 16:50:00,17525.0,75.0,75.0,mmHg,0.0,0.0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
5723050,34,144319,290505.0,220180,2191-02-23 05:30:00,2191-02-23 07:45:00,17741.0,66.0,66.0,mmHg,0.0,0.0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8305842,99064,126677,263530.0,220181,2158-08-10 14:00:00,2158-08-10 14:48:00,18539.0,52,52.0,mmHg,0.0,0.0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
8305843,99064,126677,263530.0,220181,2158-08-10 15:06:00,2158-08-10 17:32:00,18539.0,80,80.0,mmHg,0.0,0.0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
8305844,99064,126677,263530.0,220181,2158-08-10 16:00:00,2158-08-10 17:33:00,18539.0,78,78.0,mmHg,0.0,0.0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
8305845,99067,130940,204903.0,220181,2164-09-20 01:45:00,2164-09-20 02:22:00,16870.0,74,74.0,mmHg,0.0,0.0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs


In [188]:
charts[charts.LABEL.isin(['Heart Rate', 'Heart Rate Alarm - Low','Heart rate Alarm - High'])]
charts['LABEL'].mask(charts['LABEL']== 'Heart Rate Alarm - Low', 'Heart Rate', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Heart rate Alarm - High', 'Heart Rate', inplace=True)
charts[charts.LABEL.isin(['Heart Rate'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
1670333,36,165660,241249.0,220045,2134-05-12 13:00:00,2134-05-12 13:55:00,17525.0,86.0,86.0,bpm,0.0,0.0,NaN,NaN,Heart Rate,Routine Vital Signs
1670334,36,165660,241249.0,220045,2134-05-12 14:00:00,2134-05-12 14:39:00,17525.0,85.0,85.0,bpm,0.0,0.0,NaN,NaN,Heart Rate,Routine Vital Signs
1670335,36,165660,241249.0,220045,2134-05-12 15:00:00,2134-05-12 16:50:00,17525.0,87.0,87.0,bpm,0.0,0.0,NaN,NaN,Heart Rate,Routine Vital Signs
1670336,36,165660,241249.0,220045,2134-05-12 16:00:00,2134-05-12 16:50:00,17525.0,91.0,91.0,bpm,0.0,0.0,NaN,NaN,Heart Rate,Routine Vital Signs
1670337,34,144319,290505.0,220045,2191-02-23 07:34:00,2191-02-23 10:53:00,17741.0,44.0,44.0,bpm,0.0,0.0,NaN,NaN,Heart Rate,Routine Vital Signs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36618025,12772,118571,232874.0,211,2163-08-18 18:00:00,2163-08-18 19:10:00,14996.0,64,64.0,BPM,NaN,NaN,NaN,NotStopd,Heart Rate,NaN
36618026,12772,118571,232874.0,211,2163-08-19 14:00:00,2163-08-19 13:58:00,20701.0,81,81.0,BPM,NaN,NaN,NaN,NotStopd,Heart Rate,NaN
36618027,12761,171718,205774.0,211,2117-06-08 04:00:00,2117-06-08 04:08:00,20780.0,81,81.0,BPM,NaN,NaN,NaN,NotStopd,Heart Rate,NaN
36618028,12761,171718,205774.0,211,2117-06-09 03:00:00,2117-06-09 03:24:00,20868.0,65,65.0,BPM,NaN,NaN,NaN,NotStopd,Heart Rate,NaN


In [189]:
charts[charts.LABEL.isin(['Respiratory Rate (Set)',  'Respiratory Rate','Resp Alarm - High','Resp Alarm - Low',
 'Respiratory Rate (Total)', 'Respiratory Rate (spontaneous)','Resp Rate','Resp Rate (Total)'])]
charts['LABEL'].mask(charts['LABEL']==  'Respiratory Rate (Set)', 'Respiratory Rate',   inplace=True)
charts['LABEL'].mask(charts['LABEL']==   'Respiratory Rate (Total)', 'Respiratory Rate', inplace=True)
charts['LABEL'].mask(charts['LABEL']==    'Respiratory Rate (spontaneous)',  'Respiratory Rate', inplace=True)
charts['LABEL'].mask(charts['LABEL']=='Resp Rate', 'Respiratory Rate', inplace=True)
charts['LABEL'].mask(charts['LABEL']=='Resp Alarm - High', 'Respiratory Rate', inplace=True)
charts['LABEL'].mask(charts['LABEL']=='Resp Alarm - Low', 'Respiratory Rate', inplace=True)
charts['LABEL'].mask(charts['LABEL']=='Resp Rate (Total)','Respiratory Rate', inplace=True)
charts[charts.LABEL.isin(['Respiratory Rate'])]
    

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
8305847,36,165660,241249.0,220210,2134-05-12 13:00:00,2134-05-12 13:55:00,17525.0,21.0,21.0,insp/min,0.0,0.0,NaN,NaN,Respiratory Rate,Respiratory
8305848,36,165660,241249.0,220210,2134-05-12 14:00:00,2134-05-12 14:39:00,17525.0,19.0,19.0,insp/min,0.0,0.0,NaN,NaN,Respiratory Rate,Respiratory
8305849,36,165660,241249.0,220210,2134-05-12 15:00:00,2134-05-12 16:51:00,17525.0,20.0,20.0,insp/min,0.0,0.0,NaN,NaN,Respiratory Rate,Respiratory
8305850,36,165660,241249.0,220210,2134-05-12 16:00:00,2134-05-12 16:51:00,17525.0,20.0,20.0,insp/min,0.0,0.0,NaN,NaN,Respiratory Rate,Respiratory
8305851,34,144319,290505.0,220210,2191-02-23 07:33:00,2191-02-23 07:45:00,17741.0,26.0,26.0,insp/min,0.0,0.0,NaN,NaN,Respiratory Rate,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56888915,12762,176023,291210.0,615,2172-07-13 07:00:00,2172-07-13 06:37:00,14381.0,13,13.0,BPM,NaN,NaN,NaN,NotStopd,Respiratory Rate,NaN
56888916,12772,118571,232874.0,615,2163-08-30 22:45:00,2163-08-30 22:52:00,18999.0,14,14.0,BPM,NaN,NaN,NaN,NotStopd,Respiratory Rate,NaN
56888917,12772,118571,232874.0,615,2163-09-01 00:00:00,2163-08-31 23:33:00,17443.0,14,14.0,BPM,NaN,NaN,NaN,NotStopd,Respiratory Rate,NaN
56888918,12772,118571,232874.0,615,2163-08-12 23:00:00,2163-08-12 23:10:00,20111.0,10,10.0,BPM,NaN,NaN,NaN,NotStopd,Respiratory Rate,NaN


In [190]:

charts[charts.LABEL.isin(['O2 saturation pulseoxymetry','O2 Saturation Pulseoxymetry Alarm - Low', 'O2 Saturation Pulseoxymetry Alarm - High','SpO2','SpO2 Desat Limit'])]
charts['LABEL'].mask(charts['LABEL']==   'O2 Saturation Pulseoxymetry Alarm - Low','O2 saturation pulseoxymetry', inplace=True)
charts['LABEL'].mask(charts['LABEL']==    'O2 Saturation Pulseoxymetry Alarm - High','O2 saturation pulseoxymetry', inplace=True)
charts['LABEL'].mask(charts['LABEL']==    'SpO2','O2 saturation pulseoxymetry', inplace=True)
charts['LABEL'].mask(charts['LABEL']==    'SpO2 Desat Limit','O2 saturation pulseoxymetry', inplace=True)

charts[charts.LABEL.isin(['O2 saturation pulseoxymetry'])]
    

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
11042952,36,165660,241249.0,220277,2134-05-12 13:00:00,2134-05-12 13:55:00,17525.0,93.0,93.0,%,0.0,0.0,NaN,NaN,O2 saturation pulseoxymetry,Respiratory
11042953,36,165660,241249.0,220277,2134-05-12 14:00:00,2134-05-12 14:39:00,17525.0,91.0,91.0,%,0.0,0.0,NaN,NaN,O2 saturation pulseoxymetry,Respiratory
11042954,36,165660,241249.0,220277,2134-05-12 15:00:00,2134-05-12 16:51:00,17525.0,91.0,91.0,%,0.0,0.0,NaN,NaN,O2 saturation pulseoxymetry,Respiratory
11042955,36,165660,241249.0,220277,2134-05-12 16:00:00,2134-05-12 16:51:00,17525.0,92.0,92.0,%,0.0,0.0,NaN,NaN,O2 saturation pulseoxymetry,Respiratory
11042956,34,144319,290505.0,220277,2191-02-23 07:44:00,2191-02-23 07:45:00,17741.0,100.0,100.0,%,0.0,0.0,NaN,NaN,O2 saturation pulseoxymetry,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42359442,12772,118571,232874.0,646,2163-08-18 18:00:00,2163-08-18 19:10:00,14996.0,96,96.0,%,NaN,NaN,NaN,NotStopd,O2 saturation pulseoxymetry,NaN
42359443,12772,118571,232874.0,646,2163-08-19 13:00:00,2163-08-19 13:17:00,20701.0,95,95.0,%,NaN,NaN,NaN,NotStopd,O2 saturation pulseoxymetry,NaN
42359444,12772,118571,232874.0,646,2163-08-19 14:00:00,2163-08-19 13:58:00,20701.0,95,95.0,%,NaN,NaN,NaN,NotStopd,O2 saturation pulseoxymetry,NaN
42359445,12761,171718,205774.0,646,2117-06-08 04:00:00,2117-06-08 04:08:00,20780.0,100,100.0,%,NaN,NaN,NaN,NotStopd,O2 saturation pulseoxymetry,NaN


In [191]:
charts.loc[charts.LABEL== 'Admission Weight (lbs.)', 'VALUENUM'] = ((charts.VALUENUM) * 0.45359237)
charts[charts.LABEL.isin(['Admission Weight (lbs.)','Admission Weight (Kg)'])]
charts['LABEL'].mask(charts['LABEL']== 'Admission Weight (lbs.)','Admission Weight (Kg)', inplace=True)

charts[charts.LABEL.isin(['Admission Weight (Kg)'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
21052868,34,144319,290505.0,226512,2191-02-23 07:44:00,2191-02-23 07:44:00,17741.0,74.5,74.500000,kg,0.0,0.0,NaN,NaN,Admission Weight (Kg),General
21052869,23,124321,234044.0,226512,2157-10-21 12:15:00,2157-10-21 12:15:00,16978.0,66.8,66.800000,kg,0.0,0.0,NaN,NaN,Admission Weight (Kg),General
21052870,36,165660,241249.0,226512,2134-05-16 10:58:00,2134-05-16 10:58:00,16223.0,106.2,106.200000,kg,0.0,0.0,NaN,NaN,Admission Weight (Kg),General
21052871,85,112077,291697.0,226512,2167-07-25 21:31:00,2167-07-25 21:31:00,21050.0,98.0,98.000000,kg,0.0,0.0,NaN,NaN,Admission Weight (Kg),General
21052872,107,182383,252542.0,226512,2121-12-01 05:54:00,2121-12-01 05:54:00,16526.0,88.6,88.600000,kg,0.0,0.0,NaN,NaN,Admission Weight (Kg),General
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21121722,99166,184860,227031.0,226531,2154-02-01 17:10:00,2154-02-01 17:12:00,14511.0,160.6,72.846935,NaN,0.0,0.0,NaN,NaN,Admission Weight (Kg),General
21121723,98555,185035,219995.0,226531,2164-05-31 22:21:00,2164-05-31 22:22:00,20220.0,193.6,87.815483,NaN,0.0,0.0,NaN,NaN,Admission Weight (Kg),General
21121724,99229,150893,259308.0,226531,2125-12-09 19:11:00,2125-12-09 19:11:00,17806.0,132,59.874193,NaN,0.0,0.0,NaN,NaN,Admission Weight (Kg),General
21121725,99065,143271,255605.0,226531,2172-03-08 14:34:00,2172-03-08 14:34:00,19062.0,217.6,98.701700,NaN,0.0,0.0,NaN,NaN,Admission Weight (Kg),General


In [192]:
charts[charts.LABEL.isin(['Arterial Blood Pressure systolic', 'Arterial Blood Pressure diastolic','Arterial Blood Pressure mean','Arterial BP Mean','ART BP Diastolic',
 'ART BP Systolic',
 'ART BP mean'])]
charts['LABEL'].mask(charts['LABEL']== 'Arterial Blood Pressure Alarm systolic', 'Arterial Blood Pressure mean', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Arterial Blood Pressure diastolic', 'Arterial Blood Pressure mean', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Arterial BP Mean', 'Arterial Blood Pressure mean', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'ART BP Diastolic', 'Arterial Blood Pressure mean', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'ART BP Systolic', 'Arterial Blood Pressure mean', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'ART BP mean', 'Arterial Blood Pressure mean', inplace=True)
charts[charts.LABEL.isin(['Arterial Blood Pressure mean'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
23261471,23,124321,234044.0,220051,2157-10-22 11:00:00,2157-10-22 11:05:00,19907.0,67.0,67.0,mmHg,0.0,0.0,NaN,NaN,Arterial Blood Pressure mean,Routine Vital Signs
23261472,23,124321,234044.0,220051,2157-10-22 12:00:00,2157-10-22 12:49:00,19907.0,72.0,72.0,mmHg,0.0,0.0,NaN,NaN,Arterial Blood Pressure mean,Routine Vital Signs
23261473,23,124321,234044.0,220051,2157-10-22 13:00:00,2157-10-22 13:35:00,19907.0,55.0,55.0,mmHg,0.0,0.0,NaN,NaN,Arterial Blood Pressure mean,Routine Vital Signs
23261474,23,124321,234044.0,220051,2157-10-22 14:00:00,2157-10-22 15:03:00,16890.0,55.0,55.0,mmHg,0.0,0.0,NaN,NaN,Arterial Blood Pressure mean,Routine Vital Signs
23261475,23,124321,234044.0,220051,2157-10-22 15:00:00,2157-10-22 15:03:00,16890.0,56.0,56.0,mmHg,0.0,0.0,NaN,NaN,Arterial Blood Pressure mean,Routine Vital Signs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46768318,12772,118571,232874.0,52,2163-08-18 17:00:00,2163-08-18 17:15:00,17693.0,79,79.0,mmHg,NaN,NaN,NaN,NotStopd,Arterial Blood Pressure mean,NaN
46768319,12772,118571,232874.0,52,2163-08-18 18:00:00,2163-08-18 19:10:00,14996.0,67,67.0,mmHg,NaN,NaN,NaN,NotStopd,Arterial Blood Pressure mean,NaN
46768320,12772,118571,232874.0,52,2163-08-19 14:00:00,2163-08-19 13:58:00,20701.0,88,88.0,mmHg,NaN,NaN,NaN,NotStopd,Arterial Blood Pressure mean,NaN
46768321,12772,118571,232874.0,52,2163-08-19 15:00:00,2163-08-19 14:48:00,20701.0,68,68.0,mmHg,NaN,NaN,NaN,NotStopd,Arterial Blood Pressure mean,NaN


In [193]:
charts[charts.LABEL.isin(['BUN', 'BUN    (6-20)', 'BUN (6-20)'])]
charts['LABEL'].mask(charts['LABEL']== 'BUN    (6-20)', 'BUN', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'BUN (6-20)', 'BUN', inplace=True)
charts[charts.LABEL.isin(['BUN'])]
     

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
28518889,23,124321,234044.0,225624,2157-10-21 14:40:00,2157-10-21 15:53:00,20889.0,14.0,14.0,mg/dL,0.0,0.0,NaN,NaN,BUN,Labs
28518890,23,124321,234044.0,225624,2157-10-22 03:21:00,2157-10-22 04:12:00,20889.0,14.0,14.0,mg/dL,0.0,0.0,NaN,NaN,BUN,Labs
28518891,36,165660,241249.0,225624,2134-05-12 08:33:00,2134-05-12 10:01:00,20889.0,19.0,19.0,mg/dL,0.0,0.0,NaN,NaN,BUN,Labs
28518892,34,144319,290505.0,225624,2191-02-23 10:48:00,2191-02-23 12:23:00,20889.0,36.0,36.0,mg/dL,1.0,0.0,NaN,NaN,BUN,Labs
28518893,34,144319,290505.0,225624,2191-02-24 04:32:00,2191-02-24 05:41:00,20889.0,30.0,30.0,mg/dL,1.0,0.0,NaN,NaN,BUN,Labs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60405162,12680,196168,281683.0,3737,2115-08-18 02:30:00,2115-08-18 06:14:00,15331.0,12,12.0,NaN,NaN,NaN,Final,NotStopd,BUN,Chemistry
60405163,12680,196168,281683.0,3737,2115-07-30 04:00:00,2115-07-30 06:17:00,15331.0,23,23.0,NaN,NaN,NaN,Final,NotStopd,BUN,Chemistry
60405164,12747,121680,284292.0,3737,2143-05-06 04:05:00,2143-05-06 05:01:00,15331.0,33,33.0,NaN,NaN,NaN,Final,NotStopd,BUN,Chemistry
60405165,12747,121680,284292.0,3737,2143-05-08 01:42:00,2143-05-08 03:35:00,15331.0,27,27.0,NaN,NaN,NaN,Final,NotStopd,BUN,Chemistry


In [194]:
cols_dict={
'Inspired Gas Temp.':'Inspired Gas Temp.',
'PAP Mean':'PAP Mean',
'PSV Level':'PSV Level',
'Braden Score':'Braden Score',
'Ionized Calcium':'Ionized Calcium',
'Humidity Temp [Meas]':'Humidity Temp [Meas]',
'Feeding Tube Size':'Feeding Tube Size',
'Platelet Count':'Platelet Count',
'OG/NG Residual':'OG/NG Residual',
'Calcium non-ionized':'Calcium non-ionized',
'INR':'INR',
'Pulmonary Artery Pressure Alarm - High':'Pulmonary Artery Pressure mean',
'Pulmonary Artery Pressure Alarm - Low':'Pulmonary Artery Pressure mean',
'Pulmonary Artery Pressure diastolic':'Pulmonary Artery Pressure mean',
'Pulmonary Artery Pressure mean':'Pulmonary Artery Pressure mean',
'Pulmonary Artery Pressure systolic':'Pulmonary Artery Pressure mean',
'NG Tube Taped at:':'NG Tube Taped at:',
'TCO2 (calc) Arterial':'TCO2 (calc) Arterial',
'Arterial CO2 Pressure': 'Arterial CO2 Pressure',
'PTT':'PTT',
'PH (Arterial)':'PH (Arterial)',
'Plateau Pressure':'Plateau Pressure',
'Anion gap':'Anion gap',
'Sensitivity-Vent':'Sensitivity-Vent',
'HCO3 (serum)':'HCO3 (serum)',
'Glucose (serum)':'Fingerstick Glucose',
'Glucose finger stick':'Fingerstick Glucose',
'Fingerstick Glucose':'Fingerstick Glucose',
'Glucose (70-105)': 'Fingerstick Glucose',
'Glucose (whole blood)':'Fingerstick Glucose',
'Non Invasive Blood Pressure diastolic': 'Non Invasive Blood Pressure mean',
'Non Invasive Blood Pressure mean': 'Non Invasive Blood Pressure mean',
'Non-Invasive Blood Pressure Alarm - High': 'Non-Invasive Blood Pressure mean',
'Non-Invasive Blood Pressure Alarm - Low': 'Non-Invasive Blood Pressure mean',
'Non Invasive Blood Pressure systolic':'Non-Invasive Blood Pressure mean',
'Flowrate (l/m)':'Flowrate (l/m)',
'Flowrate (cc/min)':'Flowrate (l/m)',
'Admission Weight (Kg)': 'Admission Weight (Kg)',
'Admission Weight (lbs.)': 'Admission Weight (Kg)', 
'High Resp. Rate':'High Resp. Rate',
'Low Exhaled Min Vol':'Low Exhaled Min Vol',
'Chloride (serum)':'Chloride (serum)',
'Total Fluids cc/kg/d':'Total Fluids cc/kg/d',
'Potassium  (3.5-5.3)': 'Potassium (whole blood)', 
'Potassium (3.5-5.3)': 'Potassium (whole blood)', 
'Potassium (serum)': 'Potassium (whole blood)',
'Potassium (whole blood)': 'Potassium (whole blood)',   
'Sodium': 'Sodium (serum)', 
'Sodium  (135-148)': 'Sodium (serum)', 
'Sodium (135-148)': 'Sodium (serum)', 
'Sodium (whole blood)': 'Sodium (serum)',
'Sodium (serum)': 'Sodium (serum)',
'Hematocrit (serum)':'Hematocrit (serum)', 
'Hematocrit (serum)':'Hematocrit (whole blood - calc)',
'Weight Change  (gms)':'Weight Change  (gms)',
'WBC   (4-11,000)': 'WBC',
'WBC (4-11,000)': 'WBC', 
'WBC 4.0-11.0': 'WBC',                                      
'WBC': 'WBC',   
'BUN    (6-20)': 'BUN',
'BUN (6-20)': 'BUN',
'BUN': 'BUN',
'Phosphorous':'Phosphorous',
'Day of Life':'Day of Life',
'Present Weight  (kg)':'Present Weight  (kg)',
'Present Weight  (lb)':'Present Weight  (kg)',
'Present Weight  (oz)':'Present Weight  (kg)',
'Fspn High':'Fspn High',
'Magnesium': 'Magnesium',
'Expiratory Ratio':'Expiratory Ratio',
'Inspiratory Ratio':'Inspiratory Ratio',
'O2 Flow (lpm)':'O2 Flow',
'O2 Flow':'O2 Flow',
'Creatinine':'Creatinine',
'Hemoglobin':'Hemoglobin',
'Arterial Base Excess':'Arterial Base Excess',
'Inspiratory Time':'Inspiratory Time',
'GCS Total':'GCS Total',
'Motor Response':'Motor Response', 
'Verbal Response':'Verbal Response',
'GCS - Motor Response':'Motor Response', 
'GCS - Verbal Response': 'Verbal Response',  
'Eye Opening':'Eye Opening',
'GCS - Eye Opening':'Eye Opening',
'Vti High':'Vti High',
'SaO2':'SaO2',
'calprevflg':'calprevflg',
'Heart Rate Alarm - Low': 'Heart Rate',
'Heart rate Alarm - High': 'Heart Rate',
'Heart Rate': 'Heart Rate',
'Resp Rate': 'Respiratory Rate',
'Respiratory Rate': 'Respiratory Rate',
'Respiratory Rate (Set)':  'Respiratory Rate',
'Respiratory Rate (Total)':  'Respiratory Rate',
'Respiratory Rate (spontaneous)':  'Respiratory Rate',   
'O2 Saturation Pulseoxymetry Alarm - High': 'O2 Saturation Pulseoxymetry pulseoxymetry',
'O2 Saturation Pulseoxymetry Alarm - Low': 'O2 Saturation Pulseoxymetry pulseoxymetry',
'O2 saturation pulseoxymetry': 'O2 saturation pulseoxymetry',  
'SpO2':'O2 saturation pulseoxymetry',
'SpO2 Desat Limit':'O2 saturation pulseoxymetry',
'Non Invasive Blood Pressure mean':'Non Invasive Blood Pressure mean',
'Non Invasive Blood Pressure systolic':'Non Invasive Blood Pressure mean',
'Non Invasive Blood Pressure diastolic':'Non Invasive Blood Pressure mean',
'Arterial Blood Pressure systolic':'Arterial Blood Pressure mean', 
'Arterial Blood Pressure diastolic':'Arterial Blood Pressure mean',
'Arterial Blood Pressure mean':'Arterial Blood Pressure mean',
'Arterial BP Mean':'Arterial Blood Pressure mean',
'Previous Weight (kg)':'Previous Weight (kg)',
'Previous WeightF':'Previous Weight (kg)',
'Mean Airway Pressure':'Mean Airway Pressure',
'Paw High':'Mean Airway Pressure',
'NBP Mean':'NBP Mean',
'Arterial O2 pressure': 'Inspired O2 Fraction', 
'Arterial PaO2': 'Inspired O2 Fraction', 
'FIO2': 'Inspired O2 Fraction',
'FIO2 Alarm [High]': 'Inspired O2 Fraction', 
'FIO2 Alarm [Low]': 'Inspired O2 Fraction', 
'FIO2 Alarm-High': 'Inspired O2 Fraction', 
'FIO2 [Meas]': 'Inspired O2 Fraction',                                   
'Inspired O2 Fraction': 'Inspired O2 Fraction',
'Peak Insp. Pressure':'Peak Insp. Pressure',
'High Insp. Pressure':'Peak Insp. Pressure',
'Central Venous Pressure':'Central Venous Pressure',
'Central Venous Pressure  Alarm - Low':'Central Venous Pressure',
'Central Venous Pressure Alarm - High':'Central Venous Pressure',
'CVP':'Central Venous Pressure',
'PEEP':'PEEP',
'PEEP set':'PEEP',
'PEEP Set':'PEEP',
'Tidal Volume (observed)':'Tidal Volume (observed)', 
'Tidal Volume (set)':'Tidal Volume (observed)',
'Tidal Volume (spontaneous)':'Tidal Volume (observed)',
'Minute Volume':'Minute Volume', 
'Minute Volume(Obser)':'Minute Volume', 
'Minute Volume Alarm - High':'Minute Volume',
'Minute Volume Alarm - Low':'Minute Volume',
'Apnea Interval':'Apnea Interval',
    
}

In [195]:
found=False
count=0
for col in selcols:
    found=False
    if col not in cols_dict:
        for key in cols_dict:
            cols_lst=cols_dict.get(key)
            if col in cols_lst:
                found=True
    else:
        found=True
    if found==False:
        cols_dict.update({col:col})
        count+=1
count

105

In [196]:
charts['LABEL'] = charts['LABEL'].map(cols_dict)

In [197]:
charts=charts.pivot_table(values='VALUENUM', index=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID'], columns='LABEL')

In [198]:
charts

LABEL                                 ALT  AST  Abdominal Girth (cm)  \
SUBJECT_ID HADM_ID ICUSTAY_ID ITEMID                                   
2          163353  243653.0   211     NaN  NaN                   NaN   
                              834     NaN  NaN                   NaN   
                              3603    NaN  NaN                   NaN   
                              3654    NaN  NaN                   NaN   
3          145834  211552.0   211     NaN  NaN                   NaN   
...                                   ...  ...                   ...   
99999      113369  246512.0   227073  NaN  NaN                   NaN   
                              227442  NaN  NaN                   NaN   
                              227443  NaN  NaN                   NaN   
                              227457  NaN  NaN                   NaN   
                              223835  NaN  NaN                   NaN   

LABEL                                 Access Pressure  Admission Weight (Kg)  \
SUBJECT_ID HADM_ID ICUSTAY_ID ITEMID                                           
2          163353  243653.0   211                 NaN                    NaN   
                              834                 NaN                    NaN   
                              3603                NaN                    NaN   
                              3654                NaN                    NaN   
3          145834  211552.0   211                 NaN                    NaN   
...                                               ...                    ...   
99999      113369  246512.0   227073              NaN                    NaN   
                              227442              NaN                    NaN   
                              227443              NaN                    NaN   
                              227457              NaN                    NaN   
                              223835              NaN                    NaN   

LABEL                                 Albumin  Alkaline Phosphate  Anion gap  \
SUBJECT_ID HADM_ID ICUSTAY_ID ITEMID                                           
2          163353  243653.0   211         NaN                 NaN        NaN   
                              834         NaN                 NaN        NaN   
                              3603        NaN                 NaN        NaN   
                              3654        NaN                 NaN        NaN   
3          145834  211552.0   211         NaN                 NaN        NaN   
...                                       ...                 ...        ...   
99999      113369  246512.0   227073      NaN                 NaN       10.0   
                              227442      NaN                 NaN        NaN   
                              227443      NaN                 NaN        NaN   
                              227457      NaN                 NaN        NaN   
                              223835      NaN                 NaN        NaN   

LABEL                                 Apnea Interval  Apnea Time Interval  \
SUBJECT_ID HADM_ID ICUSTAY_ID ITEMID                                        
2          163353  243653.0   211                NaN                  NaN   
                              834                NaN                  NaN   
                              3603               NaN                  NaN   
                              3654               NaN                  NaN   
3          145834  211552.0   211                NaN                  NaN   
...                                              ...                  ...   
99999      113369  246512.0   227073             NaN                  NaN   
                              227442             NaN                  NaN   
                              227443             NaN                  NaN   
                              227457             NaN                  NaN   
                              223835             NaN     

In [199]:
charts_=charts.drop_duplicates()

In [200]:
charts

LABEL                                 ALT  AST  Abdominal Girth (cm)  \
SUBJECT_ID HADM_ID ICUSTAY_ID ITEMID                                   
2          163353  243653.0   211     NaN  NaN                   NaN   
                              834     NaN  NaN                   NaN   
                              3603    NaN  NaN                   NaN   
                              3654    NaN  NaN                   NaN   
3          145834  211552.0   211     NaN  NaN                   NaN   
...                                   ...  ...                   ...   
99999      113369  246512.0   227073  NaN  NaN                   NaN   
                              227442  NaN  NaN                   NaN   
                              227443  NaN  NaN                   NaN   
                              227457  NaN  NaN                   NaN   
                              223835  NaN  NaN                   NaN   

LABEL                                 Access Pressure  Admission Weight (Kg)  \
SUBJECT_ID HADM_ID ICUSTAY_ID ITEMID                                           
2          163353  243653.0   211                 NaN                    NaN   
                              834                 NaN                    NaN   
                              3603                NaN                    NaN   
                              3654                NaN                    NaN   
3          145834  211552.0   211                 NaN                    NaN   
...                                               ...                    ...   
99999      113369  246512.0   227073              NaN                    NaN   
                              227442              NaN                    NaN   
                              227443              NaN                    NaN   
                              227457              NaN                    NaN   
                              223835              NaN                    NaN   

LABEL                                 Albumin  Alkaline Phosphate  Anion gap  \
SUBJECT_ID HADM_ID ICUSTAY_ID ITEMID                                           
2          163353  243653.0   211         NaN                 NaN        NaN   
                              834         NaN                 NaN        NaN   
                              3603        NaN                 NaN        NaN   
                              3654        NaN                 NaN        NaN   
3          145834  211552.0   211         NaN                 NaN        NaN   
...                                       ...                 ...        ...   
99999      113369  246512.0   227073      NaN                 NaN       10.0   
                              227442      NaN                 NaN        NaN   
                              227443      NaN                 NaN        NaN   
                              227457      NaN                 NaN        NaN   
                              223835      NaN                 NaN        NaN   

LABEL                                 Apnea Interval  Apnea Time Interval  \
SUBJECT_ID HADM_ID ICUSTAY_ID ITEMID                                        
2          163353  243653.0   211                NaN                  NaN   
                              834                NaN                  NaN   
                              3603               NaN                  NaN   
                              3654               NaN                  NaN   
3          145834  211552.0   211                NaN                  NaN   
...                                              ...                  ...   
99999      113369  246512.0   227073             NaN                  NaN   
                              227442             NaN                  NaN   
                              227443             NaN                  NaN   
                              227457             NaN                  NaN   
                              223835             NaN     

In [201]:
charts_.to_csv(output_files+'charts_record_per_admission10M.csv')

In [202]:
cols=list(charts)
cols

['ALT',
 'AST',
 'Abdominal Girth (cm)',
 'Access Pressure',
 'Admission Weight (Kg)',
 'Albumin',
 'Alkaline Phosphate',
 'Anion gap',
 'Apnea Interval',
 'Apnea Time Interval',
 'Art.pH',
 'Arterial Base Excess',
 'Arterial Blood Pressure Alarm - High',
 'Arterial Blood Pressure Alarm - Low',
 'Arterial Blood Pressure mean',
 'Arterial CO2 Pressure',
 'Arterial CO2(Calc)',
 'Arterial O2 Saturation',
 'Arterial PaCO2',
 'Arterial pH',
 'BUN',
 'Birth Weight    (kg)',
 'Blood Flow (ml/min)',
 'Blood Temperature CCO (C)',
 'Braden Score',
 'CK (CPK)',
 'CK-MB',
 'CO (Arterial)',
 'Calcium non-ionized',
 'Cardiac Output (CCO)',
 'Central Venous Pressure',
 'Cerebral Perfusion Pressure',
 'Chloride (serum)',
 'Citrate (ACD-A)',
 'Creatinine',
 'CreatinineApacheIIScore',
 'CreatinineApacheIIValue',
 'Creatinine_ApacheIV',
 'Cuff Pressure',
 'Current Goal',
 'Daily Weight',
 'Day of Life',
 'Dialysate Rate',
 'Differential-Basos',
 'Differential-Eos',
 'Differential-Lymphs',
 'Differential-

In [203]:
for col in cols:
    charts[col] = charts.groupby('HADM_ID')[col].transform('mean')
charts.drop_duplicates(inplace=True)
charts.to_csv(output_files+'charts_avg_record_per_admission10M.csv')
charts

,,,LABEL,ALT,AST,Abdominal Girth (cm),Access Pressure,Admission Weight (Kg),Albumin,Alkaline Phosphate,Anion gap,Apnea Interval,Apnea Time Interval,...,Total PEEP Level,Troponin-T,Ultrafiltrate Output,Ventilator Tank #1,Ventilator Tank #2,Verbal Response,Vti High,WBC,Weight Change (gms),calprevflg
SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,,,,,,,,,,,,,,,,,,,,,
2,163353,243653.0,211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,145834,211552.0,211,22.500000,68.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.37931,NaN,14.744444,NaN,1.0
4,185777,294638.0,211,24.000000,64.000000,NaN,NaN,NaN,2.80,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.00000,NaN,8.500000,NaN,1.0
5,178980,214757.0,211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.900000,NaN,NaN
6,107064,228232.0,211,9.000000,54.666667,NaN,NaN,NaN,3.10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.00000,NaN,7.420000,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99985,176670,279638.0,220045,56.416667,54.833333,NaN,NaN,71.224935,2.45,281.083333,11.461538,20.0,NaN,...,7.877273,NaN,NaN,2248.0,2100.0,NaN,1.75,20.475000,NaN,NaN
99991,151118,226241.0,220045,NaN,NaN,NaN,NaN,100.394637,NaN,NaN,11.200000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.100000,NaN,NaN
99992,197084,242052.0,220045,NaN,NaN,NaN,NaN,65.335971,NaN,NaN,12.500000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.333333,NaN,NaN


# OUTPUTEVENTS File

In [412]:
outputevents=pd.read_csv(mimic_path+"OUTPUTEVENTS.csv.gz")

In [413]:
outputevents

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR
0,344,21219,177991.0,225765.0,2142-09-08 10:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
1,345,21219,177991.0,225765.0,2142-09-08 12:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
2,346,21219,177991.0,225765.0,2142-09-08 13:00:00,40055,120.0,ml,2142-09-08 13:39:00,17269,NaN,NaN,NaN
3,347,21219,177991.0,225765.0,2142-09-08 14:00:00,40055,100.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
4,348,21219,177991.0,225765.0,2142-09-08 16:00:00,40055,200.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4349213,4340476,68375,123645.0,276251.0,2175-09-05 23:00:00,226588,30.0,mL,2175-09-05 22:48:00,16915,NaN,NaN,NaN
4349214,4340477,68375,123645.0,276251.0,2175-09-06 00:00:00,226588,0.0,mL,2175-09-06 00:03:00,20971,NaN,NaN,NaN
4349215,4340478,68375,123645.0,276251.0,2175-09-06 01:00:00,226588,40.0,mL,2175-09-06 01:13:00,20971,NaN,NaN,NaN
4349216,4340479,68375,123645.0,276251.0,2175-09-06 02:00:00,226588,20.0,mL,2175-09-06 02:14:00,20971,NaN,NaN,NaN


In [414]:
outputevents=item_merge(outputevents, d_c)
outputevents

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
0,344,21219,177991.0,225765.0,2142-09-08 10:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
1,345,21219,177991.0,225765.0,2142-09-08 12:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
2,346,21219,177991.0,225765.0,2142-09-08 13:00:00,40055,120.0,ml,2142-09-08 13:39:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
3,347,21219,177991.0,225765.0,2142-09-08 14:00:00,40055,100.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
4,348,21219,177991.0,225765.0,2142-09-08 16:00:00,40055,200.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4349213,3928870,42590,199643.0,291593.0,2112-08-18 15:00:00,226621,0.0,mL,2112-08-18 15:56:00,14542,NaN,NaN,NaN,Davol,Drains
4349214,3928871,42590,199643.0,291593.0,2112-08-18 20:00:00,226621,20.0,mL,2112-08-18 21:38:00,14576,NaN,NaN,NaN,Davol,Drains
4349215,3928872,42590,199643.0,291593.0,2112-08-19 00:00:00,226621,0.0,mL,2112-08-19 00:10:00,14576,NaN,NaN,NaN,Davol,Drains
4349216,3928873,42590,199643.0,291593.0,2112-08-19 04:00:00,226621,10.0,mL,2112-08-19 04:19:00,14576,NaN,NaN,NaN,Davol,Drains


In [308]:
labels=outputevents['LABEL'].tolist()

In [309]:
unique_labels=np.unique(labels)

In [310]:
def unique(lst):
    lst_to_set = set(lst)
    unique_list = list(lst_to_set)
    return unique_list
outputevents_items=unique(d_c['LABEL'].values)
for i in outputevents_items:
    if str(i)=='nan':
        outputevents_items.remove(i)
counts={}
for col in outputevents_items:
    counts.update({col:len(outputevents[outputevents.LABEL.isin([col])])})
import operator
countslist= sorted(counts.items(), key=operator.itemgetter(1), reverse=True) 
countssortdict=dict(countslist)
selcols=[]
j=0
for i in countslist:
    selcols.append(i[0])
    if j>=150:
        break
    j+=1

In [417]:
cols=[
 'Stool Out Stool',
 'TF Residual',
 'Blood Out Lab',
 'Urine Out Condom Cath',
 'OR Out PACU Urine',
 'Drain Out #1 Other',
 'Stool',
 'OR Out OR Urine',
 'Pre-Admission Output Pre-Admission Output',
 'Stool Out Ileostomy',
 'Urine Out Suprapubic',
 'OR Out EBL',
 'Ostomy (output)',
 'Pre-Admission',
 'OR Urine',
 'Condom Cath',
 'Gastric Emesis',
 'Mediastinal',
 'Stool Out Fecal Bag',
 'Urine Out IleoConduit',
 'Stool Out Colostomy',
 'Drainage Bag',
 'Stool Out Rectal Tube',
 'L Pleural #1',
 'R Pleural #1',
 'Drain Out #1 Lumbar',
 'Fecal Bag',
 'OR EBL',
 'Emesis',
 'GU Irrigant/Urine Volume Out',
 'GU Irrigant Volume In',
 'Wound Vac #1',
 'TF Residual Output',
 'Urine Out Incontinent',
 'Lumbar',
 'Repogle Tube NG',
 'Drain Out #1 Pigtail',
 'Urine Out Rt Nephrostomy',
 'Pigtail #1',
 'Urine Out Lt Nephrostomy',
 'Ileoconduit',
 'Suprapubic',
 'Drain Out #1 Wound Vac',
 'Drain Out #1 Hemovac',
 'Drain Out #3 T Tube',
 'Drain Out #2 Other',
 'Drain Out #1 Pericardial',
 'Urine Out Other',
 'Rectal Tube',
 'Drain Out #1 T Tube',
'Chest Tube #1',
'Chest Tube #2',
'Chest Tube #3',
'Chest Tube In',
'Chest Tube R #3',
'Chest Tubes CTICU CT 1',
'Chest Tubes CTICU CT 2',
'Chest Tubes Chest Tube',
'Chest Tubes Left Pleural 1',
'Chest Tubes Left Pleural 2',
'Chest Tubes Mediastinal',
'Chest Tubes Other',
'Chest Tubes Right Pleural 1',
'Chest Tubes Right Pleural 2',
'Chest tube flush',
'Chest tube irrigate',
'Chest tube pleural',
'chest tube',
'chest tube # 3',
'chest tube #3',
'chest tube BEFORE OR',
'chest tube dsg',
'chest tube in or',
'chest tube irrigant',
'chest tube site',
 'Gastric Jejunostomy Tube',
 'Hemovac #1',
 'R Nephrostomy',
 'T Tube',
 'Drain Out #2 Pigtail',
 'Stool Out (non-specific)',
 'Stool .',
 'Ostomies Ileostomy',
 'L Nephrostomy',
 'PACU Out PACU Urine',
 'Drain Out #3 Other',
 'Jackson Pratt #4',
 'Pericardial',
 'Stool Out Ostomy',
 'Cerebral Drain Subdural',
 'Cerebral Drain Other',
 'Pigtail #2',
 'Urine Out Straight Cath',
 'Repogle Tube OGT',
 'Drain Out #2 T Tube',
 'Drain Out #3 Pigtail',
 'Stool Out Other',
 'lumbar drain',
 'Drain Out #2 Wound Vac',
 'Cath Lab',
 'Drain Out #4 Other',
 'Tap',
 'PACU Urine',
 'Straight Cath',
 'L Pleural #2',
 'Drain Out #1 Sump',
 'Cerebral Subdural #1',
 'Gastric Other',
 'Sump #1',
 'Drain Out #1 Lt Nephrostomy',
 'Drain Out #3 Wound Vac',
 'Jejunostomy',
 'Drain Out #1 Tap',
 'Urine Out Ureteral Stent #1',
 'R Pleural #2',
 'Drain Out #2 Hemovac',
 'PACU Out PACU Drains',
 'Drain Out #4 Pigtail',
 'Wound Vac #2',
 'URINE CC/KG/HR',
 'PACU Out PACU NG',
 'Drain Out #1 Jejunostomy Tube',
 'Drain Out #3 Sump',
 'Drain Out #5 T Tube',
 'Drain Out #1 Rt Nephrostomy',
 'Ostomies Colostomy',
 'Drain Out #2 Lumbar',
 'Hemovac #2',
 'paracentesis',
 'drain irrigant',
 'Drain Out #5 Other',
 'PACU Drains',
 'Drain Out #5 Pigtail',
 'Cath Lab Output']

selcols=np.unique(cols+selcols)

In [418]:
outputevents['LABEL'] = outputevents['LABEL'].astype('category')
outputevents['LABEL'].cat.set_categories(selcols,inplace=True)

C:\Users\Mariam\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arrays\categorical.py:2747: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


In [419]:
outputevents=outputevents.dropna(how='any', subset=['VALUE']) # This removes rows with null values for VALUENUM column (a total of 190,141 records removed)

In [420]:
outputevents[outputevents.LABEL.isin(['Cerebral Drain Subdural'])]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2392045,11171,5675,193148.0,256803.0,2116-11-11 05:00:00,40537,60.0,ml,2116-11-11 05:10:00,16018,NaN,NaN,NaN,Cerebral Drain Subdural,NaN
2392046,11172,5675,193148.0,256803.0,2116-11-11 11:00:00,40537,55.0,ml,2116-11-11 11:21:00,16869,NaN,NaN,NaN,Cerebral Drain Subdural,NaN
2392047,11173,5675,193148.0,256803.0,2116-11-12 06:00:00,40537,6.0,ml,2116-11-12 06:17:00,17219,NaN,NaN,NaN,Cerebral Drain Subdural,NaN
2392048,11174,5675,193148.0,256803.0,2116-11-12 09:00:00,40537,0.0,NaN,2116-11-12 09:00:00,21570,NaN,NaN,NaN,Cerebral Drain Subdural,NaN
2392049,47776,9705,174007.0,204677.0,2130-06-23 10:35:00,40537,90.0,ml,2130-06-23 10:48:00,21570,NaN,NaN,NaN,Cerebral Drain Subdural,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2392871,2766644,5726,199931.0,268867.0,2199-07-27 23:00:00,40537,17.0,ml,2199-07-27 23:10:00,19963,NaN,NaN,NaN,Cerebral Drain Subdural,NaN
2392872,2762369,5806,157649.0,212882.0,2157-08-23 10:00:00,40537,20.0,ml,2157-08-23 10:27:00,14377,NaN,NaN,NaN,Cerebral Drain Subdural,NaN
2392873,2762370,5806,157649.0,212882.0,2157-08-24 02:00:00,40537,10.0,ml,2157-08-24 02:09:00,20971,NaN,NaN,NaN,Cerebral Drain Subdural,NaN
2392874,2763016,1062,105525.0,231370.0,2105-05-19 17:00:00,40537,110.0,ml,2105-05-19 17:26:00,19295,NaN,NaN,NaN,Cerebral Drain Subdural,NaN


In [421]:
file=open(output_files+'unique_labels_output_events.txt','w')
for item in unique_labels:
    file.write(item+'\n')

file.close()

In [422]:
outputevents[outputevents.LABEL.isin(['OR Out PACU Urine', 'OR Out OR Urine'])]
outputevents['LABEL'].mask(outputevents['LABEL']=='OR Out PACU Urine', 'OR Out OR Urine', inplace=True)

outputevents[outputevents.LABEL.isin(['OR Out OR Urine'])]


C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\4031557911.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']=='OR Out PACU Urine', 'OR Out OR Urine', inplace=True)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2332202,9258,13905,173520.0,287840.0,2173-11-17 21:00:00,40061,500.0,ml,2173-11-17 23:24:00,21570,NaN,NaN,NaN,OR Out OR Urine,NaN
2332203,9259,13905,173520.0,287840.0,2173-11-18 00:00:00,40061,0.0,NaN,2173-11-18 00:00:00,21570,NaN,NaN,NaN,OR Out OR Urine,NaN
2332204,7558,7413,134105.0,219750.0,2172-11-04 23:00:00,40061,406.0,ml,2172-11-05 03:07:00,21570,NaN,NaN,NaN,OR Out OR Urine,NaN
2332205,7559,7413,134105.0,219750.0,2172-11-05 03:00:00,40061,0.0,NaN,2172-11-05 03:00:00,21570,NaN,NaN,NaN,OR Out OR Urine,NaN
2332206,14200,20040,138188.0,263100.0,2128-11-29 23:00:00,40061,150.0,ml,2128-11-30 06:22:00,21570,NaN,NaN,NaN,OR Out OR Urine,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401156,2784126,16115,164952.0,252098.0,2140-02-16 23:00:00,40065,0.0,NaN,2140-02-16 23:00:00,21570,NaN,NaN,NaN,OR Out OR Urine,NaN
2401157,2785044,30607,199225.0,202551.0,2187-06-08 15:45:00,40065,330.0,ml,2187-06-08 17:09:00,21327,NaN,NaN,NaN,OR Out OR Urine,NaN
2401158,2784594,30312,127493.0,211767.0,2187-09-14 14:00:00,40065,3700.0,ml,2187-09-14 13:58:00,17249,NaN,NaN,NaN,OR Out OR Urine,NaN
2401159,2784718,28440,179766.0,279064.0,2106-01-31 04:00:00,40065,0.0,NaN,2106-01-31 04:00:00,21570,NaN,NaN,NaN,OR Out OR Urine,NaN


In [423]:
outputevents[outputevents.LABEL.isin(['Stool Out Stool',
'Stool',
'Stool Out Ileostomy',
'Stool Out Fecal Bag',
'Stool Out Colostomy',
'Stool Out Rectal Tube',
'Stool Out (non-specific)',
'Stool .',
'Stool Out Other',
'Stool Out Ostomy'])]

outputevents['LABEL'].mask(outputevents['LABEL']=='Stool Out Stool', 'Stool',inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Stool','Stool', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Stool Out Ileostomy','Stool', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Stool Out Fecal Bag','Stool', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Stool Out Colostomy', 'Stool', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Stool Out Rectal Tube', 'Stool', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Stool Out (non-specific)', 'Stool', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Stool .', 'Stool', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Stool Out Other', 'Stool', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Stool Out Ostomy', 'Stool', inplace=True)

outputevents[outputevents.LABEL.isin(['Stool'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\492554635.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']=='Stool Out Stool', 'Stool',inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\492554635.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']=='Stool','Stool', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\492554635.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
1988842,154,17785,183076.0,203063.0,2161-07-26 06:00:00,40054,0.0,NaN,2161-07-26 06:00:00,21570,NaN,NaN,NaN,Stool,NaN
1988850,1781,1207,198966.0,264608.0,2172-07-17 11:00:00,40054,100.0,ml,2172-07-17 13:59:00,15654,NaN,NaN,NaN,Stool,NaN
1988851,1782,1207,198966.0,264608.0,2172-07-17 13:00:00,40054,150.0,ml,2172-07-17 13:59:00,15654,NaN,NaN,NaN,Stool,NaN
1988867,1810,14995,102432.0,231928.0,2102-09-26 21:00:00,40054,450.0,ml,2102-09-26 21:36:00,15218,NaN,NaN,NaN,Stool,NaN
1988868,1811,14995,102432.0,231928.0,2102-09-27 09:00:00,40054,50.0,ml,2102-09-27 10:32:00,16018,NaN,NaN,NaN,Stool,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994594,4345900,44625,163762.0,208161.0,2136-10-22 03:00:00,226579,200.0,mL,2136-10-22 03:00:00,15225,NaN,NaN,NaN,Stool,Output
3994595,4345901,44625,163762.0,208161.0,2136-10-22 04:50:00,226579,300.0,mL,2136-10-22 04:50:00,15225,NaN,NaN,NaN,Stool,Output
3994596,4345902,44625,163762.0,208161.0,2136-10-22 06:00:00,226579,150.0,mL,2136-10-22 06:34:00,15225,NaN,NaN,NaN,Stool,Output
3994597,4344510,49853,179842.0,232851.0,2138-09-27 09:00:00,226579,400.0,mL,2138-09-27 15:19:00,16866,NaN,NaN,NaN,Stool,Output


In [424]:
outputevents[outputevents.LABEL.isin([
    'Gastric Emesis','Emesis',])]
    
outputevents['LABEL'].mask(outputevents['LABEL']== 'Gastric Emesis','Emesis', inplace=True)
outputevents[outputevents.LABEL.isin(['Emesis'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\437450532.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']== 'Gastric Emesis','Emesis', inplace=True)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2134992,224,834,153730.0,282461.0,2166-08-26 21:00:00,40067,0.0,NaN,2166-08-26 21:00:00,21570,NaN,NaN,NaN,Emesis,NaN
2134994,4717,1474,140051.0,226332.0,2106-09-21 16:00:00,40067,100.0,ml,2106-09-21 16:17:00,14431,NaN,NaN,NaN,Emesis,NaN
2134995,3555,19931,186802.0,200426.0,2183-08-21 16:00:00,40067,150.0,ml,2183-08-21 16:00:00,21570,NaN,NaN,NaN,Emesis,NaN
2134996,3556,19931,186802.0,200426.0,2183-08-21 21:00:00,40067,75.0,ml,2183-08-21 21:44:00,17155,NaN,NaN,NaN,Emesis,NaN
2134997,3557,24457,184834.0,205776.0,2193-09-10 19:00:00,40067,900.0,ml,2193-09-10 19:15:00,21570,NaN,NaN,NaN,Emesis,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4066838,4332629,50532,111610.0,237192.0,2195-10-16 16:45:00,226571,200.0,mL,2195-10-16 16:45:00,15652,NaN,NaN,NaN,Emesis,Output
4066839,4338478,93321,115396.0,248745.0,2128-09-16 21:01:00,226571,30.0,mL,2128-09-16 21:01:00,16037,NaN,NaN,NaN,Emesis,Output
4066840,4338479,93321,115396.0,248745.0,2128-09-17 09:00:00,226571,100.0,mL,2128-09-17 09:00:00,18576,NaN,NaN,NaN,Emesis,Output
4066841,4346850,43320,154851.0,233289.0,2128-09-08 17:35:00,226571,50.0,mL,2128-09-08 17:36:00,18462,NaN,NaN,NaN,Emesis,Output


In [425]:
outputevents[outputevents.LABEL.isin([
    'TF Residual Output','TF Residual',])]
    
outputevents['LABEL'].mask(outputevents['LABEL']== 'TF Residual Output','TF Residual', inplace=True)
outputevents[outputevents.LABEL.isin(['TF Residual'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\2164732686.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']== 'TF Residual Output','TF Residual', inplace=True)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
4081547,2813999,32725,131525.0,294087.0,2193-09-21 02:29:00,227510,0.0,mL,2193-09-21 02:29:00,20622,NaN,NaN,NaN,TF Residual,Output
4081548,2814000,32725,131525.0,294087.0,2193-09-21 21:00:00,227510,200.0,mL,2193-09-21 21:58:00,20622,NaN,NaN,NaN,TF Residual,Output
4081549,2814001,32725,131525.0,294087.0,2193-09-22 00:01:00,227510,100.0,mL,2193-09-22 01:56:00,20622,NaN,NaN,NaN,TF Residual,Output
4081550,2815957,87186,176866.0,218451.0,2190-08-30 21:08:00,227510,0.0,mL,2190-08-30 21:08:00,14986,NaN,NaN,NaN,TF Residual,Output
4081551,2815958,87186,176866.0,218451.0,2190-08-31 00:00:00,227510,0.0,mL,2190-08-31 00:27:00,14986,NaN,NaN,NaN,TF Residual,Output
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116420,4346192,10694,138159.0,294193.0,2153-08-16 13:42:00,227511,10.0,mL,2153-08-16 13:42:00,16187,NaN,NaN,NaN,TF Residual,Output
4116421,4333790,51078,168187.0,267917.0,2191-08-08 20:07:00,227511,150.0,mL,2191-08-08 20:07:00,16229,NaN,NaN,NaN,TF Residual,Output
4116422,4333791,51078,168187.0,267917.0,2191-08-09 01:00:00,227511,100.0,mL,2191-08-09 01:17:00,16526,NaN,NaN,NaN,TF Residual,Output
4116423,4333792,51078,168187.0,267917.0,2191-08-10 20:00:00,227511,325.0,mL,2191-08-10 19:59:00,15696,NaN,NaN,NaN,TF Residual,Output


In [426]:
outputevents[outputevents.LABEL.isin([
    'Gastric Nasogastric',
'Gastric Oral Gastric',
'Nasogastric',
'Oral Gastric'])]

outputevents['LABEL'].mask(outputevents['LABEL']=='Gastric Nasogastric','Oral Gastric', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Gastric Oral Gastric','Oral Gastric', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Nasogastric','Oral Gastric', inplace=True)
outputevents[outputevents.LABEL.isin(['Oral Gastric'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\4124031216.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']=='Gastric Nasogastric','Oral Gastric', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\4124031216.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']=='Gastric Oral Gastric','Oral Gastric', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\4124031216.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2080546,200,21219,177991.0,225765.0,2142-09-06 20:00:00,40052,0.0,NaN,2142-09-06 20:00:00,21570,NaN,NaN,NaN,Oral Gastric,NaN
2080547,1825,21219,177991.0,225765.0,2142-09-05 22:00:00,40052,60.0,ml,2142-09-05 22:01:00,15218,NaN,NaN,NaN,Oral Gastric,NaN
2080548,4679,20896,107559.0,204786.0,2104-09-14 11:00:00,40052,700.0,ml,2104-09-14 11:32:00,19651,NaN,NaN,NaN,Oral Gastric,NaN
2080549,4680,20896,107559.0,204786.0,2104-09-14 18:00:00,40052,300.0,ml,2104-09-14 18:30:00,19651,NaN,NaN,NaN,Oral Gastric,NaN
2080550,4681,20896,107559.0,204786.0,2104-09-14 20:00:00,40052,600.0,ml,2104-09-14 20:16:00,14442,NaN,NaN,NaN,Oral Gastric,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4081542,4348921,70698,190460.0,214189.0,2183-10-17 16:46:00,226576,250.0,mL,2183-10-17 16:46:00,20081,NaN,NaN,NaN,Oral Gastric,Output
4081543,4347391,84057,112235.0,263991.0,2119-09-06 17:57:00,226576,50.0,mL,2119-09-06 17:57:00,14891,NaN,NaN,NaN,Oral Gastric,Output
4081544,4344770,84095,131026.0,269064.0,2164-08-24 15:14:00,226576,25.0,mL,2164-08-24 15:15:00,14988,NaN,NaN,NaN,Oral Gastric,Output
4081545,4344771,84095,131026.0,269064.0,2164-08-24 16:00:00,226576,25.0,mL,2164-08-24 17:03:00,14988,NaN,NaN,NaN,Oral Gastric,Output


In [427]:
outputevents[outputevents.LABEL.isin([
 'Gastric Gastric Tube',
'Gastric Tube',
'Gastric Jejunostomy Tube',
'Gastric Other'])]

outputevents['LABEL'].mask(outputevents['LABEL']=='Gastric Gastric Tube','Gastric Tube', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Gastric Jejunostomy Tube','Gastric Tube', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Gastric Other','Gastric Tube', inplace=True)
outputevents[outputevents.LABEL.isin(['Gastric Tube'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\3482260804.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']=='Gastric Gastric Tube','Gastric Tube', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\3482260804.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']=='Gastric Jejunostomy Tube','Gastric Tube', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\3482260804.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docum

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2127367,214,834,153730.0,282461.0,2166-08-19 14:00:00,40051,190.0,ml,2166-08-19 18:03:00,15654,NaN,NaN,NaN,Gastric Tube,NaN
2127368,215,834,153730.0,282461.0,2166-08-19 17:00:00,40051,200.0,ml,2166-08-19 18:03:00,15654,NaN,NaN,NaN,Gastric Tube,NaN
2127369,216,834,153730.0,282461.0,2166-08-19 22:00:00,40051,150.0,ml,2166-08-19 22:22:00,15218,NaN,NaN,NaN,Gastric Tube,NaN
2127370,217,834,153730.0,282461.0,2166-08-20 00:00:00,40051,50.0,ml,2166-08-20 00:29:00,15023,NaN,NaN,NaN,Gastric Tube,NaN
2127371,218,834,153730.0,282461.0,2166-08-20 04:00:00,40051,0.0,ml,2166-08-20 04:18:00,15023,NaN,NaN,NaN,Gastric Tube,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975793,4346823,77617,128104.0,215763.0,2154-09-19 15:15:00,226573,30.0,mL,2154-09-19 15:15:00,15659,NaN,NaN,NaN,Gastric Tube,Output
3975794,4340800,41493,121639.0,296353.0,2175-09-05 18:09:00,226573,50.0,mL,2175-09-05 18:10:00,14638,NaN,NaN,NaN,Gastric Tube,Output
3975795,4340801,41493,121639.0,296353.0,2175-09-06 06:00:00,226573,1500.0,mL,2175-09-06 05:44:00,14435,NaN,NaN,NaN,Gastric Tube,Output
3975796,4340802,41493,121639.0,296353.0,2175-09-06 12:00:00,226573,500.0,mL,2175-09-06 11:34:00,18954,NaN,NaN,NaN,Gastric Tube,Output


In [428]:
outputevents[outputevents.LABEL.isin([
    'Cerebral Ventricular #2','Cerebral Ventricular #1',
'Cerebral Drain R Ventricular Drain',
'Cerebral Drain L Ventricular Drain',])]
    
outputevents['LABEL'].mask(outputevents['LABEL']== 'Cerebral Ventricular #2','Cerebral Ventricular #1', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Cerebral Drain R Ventricular Drain','Cerebral Ventricular #1', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Cerebral Drain L Ventricular Drain','Cerebral Ventricular #1', inplace=True)

outputevents[outputevents.LABEL.isin(['Cerebral Ventricular #1'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\3246926219.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']== 'Cerebral Ventricular #2','Cerebral Ventricular #1', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\3246926219.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']== 'Cerebral Drain R Ventricular Drain','Cerebral Ventricular #1', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\3246926219.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2233480,2453,20856,157810.0,283332.0,2195-10-07 18:30:00,40058,10.0,ml,2195-10-07 18:47:00,14431,NaN,NaN,NaN,Cerebral Ventricular #1,NaN
2233481,35565,26006,136948.0,229287.0,2171-04-23 07:00:00,40058,70.0,ml,2171-04-23 07:11:00,14442,NaN,NaN,NaN,Cerebral Ventricular #1,NaN
2233482,35566,26006,136948.0,229287.0,2171-04-23 12:00:00,40058,65.0,ml,2171-04-23 14:54:00,21570,NaN,NaN,NaN,Cerebral Ventricular #1,NaN
2233483,35567,26006,136948.0,229287.0,2171-04-23 14:00:00,40058,0.0,NaN,2171-04-23 14:00:00,21570,NaN,NaN,NaN,Cerebral Ventricular #1,NaN
2233484,52094,18260,123938.0,284394.0,2169-08-17 11:00:00,40058,10.0,ml,2169-08-17 11:43:00,19942,NaN,NaN,NaN,Cerebral Ventricular #1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4340868,4301939,52330,106589.0,212619.0,2112-07-21 08:00:00,226607,3.0,mL,2112-07-21 08:01:00,15667,NaN,NaN,NaN,Cerebral Ventricular #1,Drains
4340869,4301940,52330,106589.0,212619.0,2112-07-21 09:00:00,226607,1.0,mL,2112-07-21 10:06:00,15667,NaN,NaN,NaN,Cerebral Ventricular #1,Drains
4340870,4301941,52330,106589.0,212619.0,2112-07-21 10:00:00,226607,1.0,mL,2112-07-21 10:07:00,15667,NaN,NaN,NaN,Cerebral Ventricular #1,Drains
4340871,4301942,52330,106589.0,212619.0,2112-07-21 11:00:00,226607,0.0,mL,2112-07-21 11:18:00,15667,NaN,NaN,NaN,Cerebral Ventricular #1,Drains


In [429]:
outputevents[outputevents.LABEL.isin([
'Drain Out #1 JP Medial',
'Drain Out #1 Jackson Pratt',
'Jackson Pratt #3',
'JP Medial',
'Jackson Pratt #4',
'JP Lateral',
'Jackson Pratt #2',
'Jackson Pratt #1',
'Drain Out #2 Jackson Pratt',
'Drain Out #2 JP Lateral',
'Drain Out #3 Jackson Pratt',
'Drain Out #1 JP Lateral',
'Drain Out #4 Jackson Pratt',
'Drain Out #2 JP Medial',
'Drain Out #5 Jackson Pratt'])]

outputevents['LABEL'].mask(outputevents['LABEL']== 'Drain Out #1 JP Medial', 'Drain Out #1 Jackson Pratt', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Jackson Pratt #3','Drain Out #1 Jackson Pratt', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'JP Medial','Drain Out #1 Jackson Pratt', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Jackson Pratt #4','Drain Out #1 Jackson Pratt', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'JP Lateral','Drain Out #1 Jackson Pratt', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Jackson Pratt #2','Drain Out #1 Jackson Pratt', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Jackson Pratt #1','Drain Out #1 Jackson Pratt', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Drain Out #2 Jackson Pratt','Drain Out #1 Jackson Pratt', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Drain Out #2 JP Lateral','Drain Out #1 Jackson Pratt', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Drain Out #3 Jackson Pratt','Drain Out #1 Jackson Pratt', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Drain Out #1 JP Lateral','Drain Out #1 Jackson Pratt', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Drain Out #4 Jackson Pratt','Drain Out #1 Jackson Pratt', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Drain Out #2 JP Medial','Drain Out #1 Jackson Pratt', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Drain Out #5 Jackson Pratt','Drain Out #1 Jackson Pratt', inplace=True)
outputevents[outputevents.LABEL.isin(['Drain Out #1 Jackson Pratt'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\3099256102.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']== 'Drain Out #1 JP Medial', 'Drain Out #1 Jackson Pratt', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\3099256102.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']== 'Jackson Pratt #3','Drain Out #1 Jackson Pratt', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\3099256102.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame



,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2251174,4048,5417,170150.0,256429.0,2179-08-12 08:00:00,40072,30.0,ml,2179-08-12 08:42:00,15654,NaN,NaN,NaN,Drain Out #1 Jackson Pratt,NaN
2251175,4049,5417,170150.0,256429.0,2179-08-12 18:00:00,40072,20.0,ml,2179-08-12 17:48:00,15654,NaN,NaN,NaN,Drain Out #1 Jackson Pratt,NaN
2251176,4050,5417,170150.0,256429.0,2179-08-13 06:00:00,40072,65.0,ml,2179-08-13 05:52:00,21570,NaN,NaN,NaN,Drain Out #1 Jackson Pratt,NaN
2251177,4051,5417,170150.0,256429.0,2179-08-13 18:00:00,40072,60.0,ml,2179-08-13 17:50:00,18592,NaN,NaN,NaN,Drain Out #1 Jackson Pratt,NaN
2251178,24426,12487,182699.0,243156.0,2162-02-03 00:00:00,40072,100.0,ml,2162-02-03 01:54:00,14442,NaN,NaN,NaN,Drain Out #1 Jackson Pratt,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4332470,4307934,63201,190733.0,247444.0,2200-09-03 18:00:00,226601,20.0,mL,2200-09-03 18:20:00,21492,NaN,NaN,NaN,Drain Out #1 Jackson Pratt,Drains
4332471,4307935,63201,190733.0,247444.0,2200-09-03 23:00:00,226601,30.0,mL,2200-09-03 23:20:00,20165,NaN,NaN,NaN,Drain Out #1 Jackson Pratt,Drains
4332472,4307936,63201,190733.0,247444.0,2200-09-04 04:00:00,226601,40.0,mL,2200-09-04 06:34:00,20165,NaN,NaN,NaN,Drain Out #1 Jackson Pratt,Drains
4332473,4307937,63201,190733.0,247444.0,2200-09-04 11:03:00,226601,80.0,mL,2200-09-04 11:03:00,21106,NaN,NaN,NaN,Drain Out #1 Jackson Pratt,Drains


In [430]:
outputevents[outputevents.LABEL.isin(['Urine .','Urine Out Void'])]
outputevents['LABEL'].mask(outputevents['LABEL']== 'Urine .','Urine Out Void', inplace=True)
outputevents[outputevents.LABEL.isin(['Urine Out Void'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\4193977070.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']== 'Urine .','Urine Out Void', inplace=True)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
1917421,1,24730,143418.0,225089.0,2144-07-09 19:15:00,40069,80.0,ml,2144-07-09 19:20:00,14990,NaN,NaN,NaN,Urine Out Void,NaN
1917422,2,24730,143418.0,225089.0,2144-07-10 00:00:00,40069,100.0,ml,2144-07-09 23:51:00,20675,NaN,NaN,NaN,Urine Out Void,NaN
1917423,3,24730,143418.0,225089.0,2144-07-10 02:00:00,40069,45.0,ml,2144-07-10 01:50:00,20675,NaN,NaN,NaN,Urine Out Void,NaN
1917424,4,24730,143418.0,225089.0,2144-07-10 05:00:00,40069,45.0,ml,2144-07-10 04:58:00,20675,NaN,NaN,NaN,Urine Out Void,NaN
1917425,5,24730,143418.0,225089.0,2144-07-10 06:00:00,40069,45.0,ml,2144-07-10 06:11:00,20675,NaN,NaN,NaN,Urine Out Void,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2758347,2785335,31085,164318.0,236187.0,2117-12-24 17:30:00,43175,11.0,ml,2117-12-24 18:19:00,17001,NaN,NaN,NaN,Urine Out Void,NaN
2758348,2785336,30931,113566.0,248716.0,2197-11-12 05:00:00,43175,15.0,ml,2197-11-12 06:06:00,20318,NaN,NaN,NaN,Urine Out Void,NaN
2758349,2785337,30931,113566.0,248716.0,2197-11-13 09:00:00,43175,12.0,ml,2197-11-13 13:52:00,16433,NaN,NaN,NaN,Urine Out Void,NaN
2758350,2785341,28983,135893.0,222689.0,2177-11-06 08:00:00,43175,9.0,ml,2177-11-06 09:07:00,16571,NaN,NaN,NaN,Urine Out Void,NaN


In [431]:
outputevents[outputevents.LABEL.isin(['Foley','Urine Out Foley','Urine .','Urine Out Void'])]
outputevents['LABEL'].mask(outputevents['LABEL']== 'Foley','Urine Out Foley', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Foley','Urine Out Foley', inplace=True)
outputevents[outputevents.LABEL.isin(['Urine Out Foley'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\813459452.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']== 'Foley','Urine Out Foley', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\813459452.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']== 'Foley','Urine Out Foley', inplace=True)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
0,344,21219,177991.0,225765.0,2142-09-08 10:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
1,345,21219,177991.0,225765.0,2142-09-08 12:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
2,346,21219,177991.0,225765.0,2142-09-08 13:00:00,40055,120.0,ml,2142-09-08 13:39:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
3,347,21219,177991.0,225765.0,2142-09-08 14:00:00,40055,100.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
4,348,21219,177991.0,225765.0,2142-09-08 16:00:00,40055,200.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3972317,4340462,68375,123645.0,276251.0,2175-09-06 13:00:00,226559,25.0,mL,2175-09-06 13:03:00,16915,NaN,NaN,NaN,Urine Out Foley,Output
3972318,4340463,68375,123645.0,276251.0,2175-09-06 14:00:00,226559,5.0,mL,2175-09-06 15:27:00,15039,NaN,NaN,NaN,Urine Out Foley,Output
3972319,4340464,68375,123645.0,276251.0,2175-09-06 15:00:00,226559,200.0,mL,2175-09-06 15:07:00,15039,NaN,NaN,NaN,Urine Out Foley,Output
3972320,4340465,68375,123645.0,276251.0,2175-09-06 16:00:00,226559,250.0,mL,2175-09-06 16:12:00,15039,NaN,NaN,NaN,Urine Out Foley,Output


In [432]:
outputevents[outputevents.LABEL.isin(['Chest Tube #1',
'Chest Tube #2',
'Chest Tube #3',
'Chest Tube In',
'Chest Tube R #3',
'Chest Tubes CTICU CT 1',
'Chest Tubes CTICU CT 2',
'Chest Tubes Chest Tube',
'Chest Tubes Left Pleural 1',
'Chest Tubes Left Pleural 2',
'Chest Tubes Mediastinal',
'Chest Tubes Other',
'Chest Tubes Right Pleural 1',
'Chest Tubes Right Pleural 2',
'Chest tube flush',
'Chest tube irrigate',
'Chest tube pleural',
'chest tube',
'chest tube # 3',
'chest tube #3',
'chest tube BEFORE OR',
'chest tube dsg',
'chest tube in or',
'chest tube irrigant',
'chest tube site',])]

outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube # 3', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube #3', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube BEFORE OR', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube dsg','Chest Tubes Other',inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube in or','Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube irrigant', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube site', 'Chest Tubes Other',inplace=True)

outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tube #1', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tube #2', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tube #3', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tube In', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tube R #3', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes CTICU CT 1', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes CTICU CT 2', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes Chest Tube', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes Left Pleural 1', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes Left Pleural 2', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes Mediastinal', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes Right Pleural 2', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest tube flush', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes Right Pleural 1', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest tube irrigate', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest tube pleural', 'Chest Tubes Other', inplace=True)

outputevents[outputevents.LABEL.isin(['Chest Tubes Other'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\552950729.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube # 3', 'Chest Tubes Other', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\552950729.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube #3', 'Chest Tubes Other', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\552950729.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the document

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2214943,1774,5896,199422.0,285436.0,2188-07-24 06:00:00,40050,110.0,ml,2188-07-24 06:43:00,15029,NaN,NaN,NaN,Chest Tubes Other,NaN
2214944,19601,2355,191024.0,292695.0,2158-01-28 07:00:00,40050,70.0,ml,2158-01-28 07:12:00,14442,NaN,NaN,NaN,Chest Tubes Other,NaN
2214945,26149,10168,184129.0,NaN,2111-03-08 04:00:00,40050,20.0,ml,2111-03-08 04:50:00,21570,NaN,NaN,NaN,Chest Tubes Other,NaN
2214946,26150,10168,184129.0,NaN,2111-03-08 07:00:00,40050,30.0,ml,2111-03-08 07:13:00,21570,NaN,NaN,NaN,Chest Tubes Other,NaN
2214947,26151,10168,184129.0,NaN,2111-03-08 16:00:00,40050,60.0,ml,2111-03-08 17:20:00,21570,NaN,NaN,NaN,Chest Tubes Other,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4292402,4344731,45141,178080.0,287521.0,2113-10-15 08:00:00,226589,90.0,mL,2113-10-15 08:50:00,16593,NaN,NaN,NaN,Chest Tubes Other,Output
4292403,4344732,45141,178080.0,287521.0,2113-10-15 15:00:00,226589,0.0,mL,2113-10-15 14:59:00,16593,NaN,NaN,NaN,Chest Tubes Other,Output
4292404,4344733,45141,178080.0,287521.0,2113-10-16 05:03:00,226589,200.0,mL,2113-10-16 05:03:00,19611,NaN,NaN,NaN,Chest Tubes Other,Output
4292405,4344734,45141,178080.0,287521.0,2113-10-16 09:00:00,226589,70.0,mL,2113-10-16 10:00:00,19174,NaN,NaN,NaN,Chest Tubes Other,Output


In [434]:
outputevents['LABEL']=outputevents['LABEL'].map({
'Foley':'Urine Out Foley',
'Urine Out Foley':'Urine Out Foley',
'Urine .':'Urine Out Void',
'Urine Out Void':'Urine Out Void',
'Void':'Void',
'OR Out PACU Urine':'OR Out OR Urine',
'OR Out OR Urine':'OR Out OR Urine',
'Ultrafiltrate Ultrafiltrate':'Ultrafiltrate Ultrafiltrate',
'Drain Out #1 JP Medial':'Drain Out #1 Jackson Pratt',
'Drain Out #1 Jackson Pratt':'Drain Out #1 Jackson Pratt',
'Jackson Pratt #3':'Drain Out #1 Jackson Pratt',
'JP Medial':'Drain Out #1 Jackson Pratt',
'Jackson Pratt #4':'Drain Out #1 Jackson Pratt',
'JP Lateral':'Drain Out #1 Jackson Pratt',
'Jackson Pratt #2':'Drain Out #1 Jackson Pratt',
'Jackson Pratt #1':'Drain Out #1 Jackson Pratt',
'Drain Out #2 Jackson Pratt':'Drain Out #1 Jackson Pratt',
'Drain Out #2 JP Lateral':'Drain Out #1 Jackson Pratt',
'Drain Out #3 Jackson Pratt':'Drain Out #1 Jackson Pratt',
'Drain Out #1 JP Lateral':'Drain Out #1 Jackson Pratt',
'Drain Out #4 Jackson Pratt':'Drain Out #1 Jackson Pratt',
'Drain Out #2 JP Medial':'Drain Out #1 Jackson Pratt',
'Drain Out #5 Jackson Pratt':'Drain Out #1 Jackson Pratt',
'Cerebral Ventricular #1':'Cerebral Ventricular #1',
'Cerebral Ventricular #2':'Cerebral Ventricular #1',
'Cerebral Drain R Ventricular Drain':'Cerebral Ventricular #1',
'Cerebral Drain L Ventricular Drain':'Cerebral Ventricular #1',
'Emesis':'Emesis',
'Gastric Emesis':'Emesis',
'Stool Out Stool':'Stool',
'Stool':'Stool',
'Stool Out Ileostomy':'Stool',
'Stool Out Fecal Bag':'Stool',
'Stool Out Colostomy':'Stool',
'Stool Out Rectal Tube':'Stool',
'Stool Out (non-specific)':'Stool',
'Stool .':'Stool',
'Stool Out Other':'Stool',
'Stool Out Ostomy':'Stool',
'Blood Out Lab':'Blood Out Lab',
'Urine Out Suprapubic':'Urine Out Suprapubic',
'Urine Out Condom Cath':'Urine Out Condom Cath',
'TF Residual Output':'TF Residual',
'TF Residual':'TF Residual',
'Gastric Nasogastric':'Oral Gastric',
'Gastric Oral Gastric':'Oral Gastric',
'Nasogastric':'Oral Gastric',
'Oral Gastric':'Oral Gastric',
'Gastric Gastric Tube':'Gastric Tube',
'Gastric Tube':'Gastric Tube',
'Gastric Jejunostomy Tube':'Gastric Tube',
'Gastric Other':'Gastric Tube',




'Chest Tube #1':'Chest Tubes Other',
'Chest Tube #2':'Chest Tubes Other',
'Chest Tube #3':'Chest Tubes Other',
'Chest Tube In':'Chest Tubes Other',
'Chest Tube R #3':'Chest Tubes Other',
'Chest Tubes CTICU CT 1':'Chest Tubes Other',
'Chest Tubes CTICU CT 2':'Chest Tubes Other',
'Chest Tubes Chest Tube':'Chest Tubes Other',
'Chest Tubes Left Pleural 1':'Chest Tubes Other',
'Chest Tubes Left Pleural 2':'Chest Tubes Other',
'Chest Tubes Mediastinal':'Chest Tubes Other',
'Chest Tubes Other':'Chest Tubes Other',
'Chest Tubes Right Pleural 1':'Chest Tubes Other',
'Chest Tubes Right Pleural 2':'Chest Tubes Other',
'Chest tube flush':'Chest Tubes Other',
'Chest tube irrigate':'Chest Tubes Other',
'Chest tube pleural':'Chest Tubes Other',
'Chest Tube #1':'Chest Tubes Other',
'Chest Tube #2':'Chest Tubes Other',
'Chest Tube #3':'Chest Tubes Other',
'Chest Tube In':'Chest Tubes Other',
'Chest Tube R #3':'Chest Tubes Other',
'Chest Tubes CTICU CT 1':'Chest Tubes Other',
'Chest Tubes CTICU CT 2':'Chest Tubes Other',
'Chest Tubes Chest Tube':'Chest Tubes Other',
'Chest Tubes Left Pleural 1':'Chest Tubes Other',
'Chest Tubes Left Pleural 2':'Chest Tubes Other',
'Chest Tubes Mediastinal':'Chest Tubes Other',
'Chest Tubes Other':'Chest Tubes Other',
'Chest Tubes Right Pleural 1':'Chest Tubes Other',
'Chest Tubes Right Pleural 2':'Chest Tubes Other',
'Chest tube flush':'Chest Tubes Other',
'Chest tube irrigate':'Chest Tubes Other',
'Chest tube pleural':'Chest Tubes Other',
'chest tube':'Chest Tubes Other',
'chest tube # 3':'Chest Tubes Other',
'chest tube #3':'Chest Tubes Other',
'chest tube BEFORE OR':'Chest Tubes Other',
'chest tube dsg':'Chest Tubes Other',
'chest tube in or':'Chest Tubes Other',
'chest tube irrigant':'Chest Tubes Other',
'chest tube site':'Chest Tubes Other',

})

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\323924360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL']=outputevents['LABEL'].map({


In [435]:
outputevents=outputevents.pivot_table(values='VALUE', index=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID'], columns='LABEL')

In [436]:
outputevents_dupdrop=outputevents.drop_duplicates()
outputevents_dupdrop

LABEL                                  Blood Out Lab  Cerebral Ventricular #1  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                                           
3          145834.0 211552.0   40055             NaN                      NaN   
                               40059             NaN                      NaN   
4          185777.0 294638.0   40054             NaN                      NaN   
                               40069             NaN                      NaN   
6          107064.0 228232.0   40055             NaN                      NaN   
...                                              ...                      ...   
99982      151454.0 221194.0   226559            NaN                      NaN   
           183791.0 239827.0   226559            NaN                      NaN   
                               226560            NaN                      NaN   
99992      197084.0 242052.0   226559            NaN                      NaN   
99995      137810.0 229633.0   226559            NaN                      NaN   

LABEL                                  Chest Tubes Other  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                      
3          145834.0 211552.0   40055                 NaN   
                               40059                 NaN   
4          185777.0 294638.0   40054                 NaN   
                               40069                 NaN   
6          107064.0 228232.0   40055                 NaN   
...                                                  ...   
99982      151454.0 221194.0   226559                NaN   
           183791.0 239827.0   226559                NaN   
                               226560                NaN   
99992      197084.0 242052.0   226559                NaN   
99995      137810.0 229633.0   226559                NaN   

LABEL                                  Drain Out #1 Jackson Pratt  Emesis  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                                       
3          145834.0 211552.0   40055                          NaN     NaN   
                               40059                          NaN     NaN   
4          185777.0 294638.0   40054                          NaN     NaN   
                               40069                          NaN     NaN   
6          107064.0 228232.0   40055                          NaN     NaN   
...                                                           ...     ...   
99982      151454.0 221194.0   226559                         NaN     NaN   
           183791.0 239827.0   226559                         NaN     NaN   
                               226560                         NaN     NaN   
99992      197084.0 242052.0   226559                         NaN     NaN   
99995      137810.0 229633.0   226559                         NaN     NaN   

LABEL                                  Gastric Tube  OR Out OR Urine  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                                  
3          145834.0 211552.0   40055            NaN              NaN   
                               40059            NaN              NaN   
4          185777.0 294638.0   40054            NaN              NaN   
                               40069            NaN              NaN   
6          107064.0 228232.0   40055            NaN              NaN   
...                                             ...              ...   
99982      151454.0 221194.0   226559           NaN              NaN   
           183791.0 239827.0   226559           NaN              NaN   
                               226560           NaN              NaN   
99992      197084.0 242052.0   226559           NaN              NaN   
99995      137810.0 229633.0   226559           NaN              NaN   

LABEL                                  Oral Gastric  Stool  TF Residual  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                                     
3          145834.0 211552.0   40055            NaN    NaN          NaN   
 

In [443]:
outputevents_dupdrop.to_csv(output_files+"outputevents_.csv")

In [438]:
cols=list(outputevents)

In [440]:
for col in cols:
    outputevents[col] = outputevents.groupby('HADM_ID')[col].transform('mean')
outputevents.drop_duplicates(inplace=True)

In [441]:
outputevents.columns

Index(['Blood Out Lab', 'Cerebral Ventricular #1', 'Chest Tubes Other',
       'Drain Out #1 Jackson Pratt', 'Emesis', 'Gastric Tube',
       'OR Out OR Urine', 'Oral Gastric', 'Stool', 'TF Residual',
       'Ultrafiltrate Ultrafiltrate', 'Urine Out Condom Cath',
       'Urine Out Foley', 'Urine Out Suprapubic', 'Urine Out Void', 'Void'],
      dtype='object', name='LABEL')

In [444]:
outputevents.to_csv('outputevents_avg.csv')

In [445]:
charts

,,,LABEL,ALT,AST,Abdominal Girth (cm),Access Pressure,Admission Weight (Kg),Albumin,Alkaline Phosphate,Anion gap,Apnea Interval,Apnea Time Interval,...,Total PEEP Level,Troponin-T,Ultrafiltrate Output,Ventilator Tank #1,Ventilator Tank #2,Verbal Response,Vti High,WBC,Weight Change (gms),calprevflg
SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,,,,,,,,,,,,,,,,,,,,,
2,163353,243653.0,211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,145834,211552.0,211,22.500000,68.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.37931,NaN,14.744444,NaN,1.0
4,185777,294638.0,211,24.000000,64.000000,NaN,NaN,NaN,2.80,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.00000,NaN,8.500000,NaN,1.0
5,178980,214757.0,211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.900000,NaN,NaN
6,107064,228232.0,211,9.000000,54.666667,NaN,NaN,NaN,3.10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.00000,NaN,7.420000,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99985,176670,279638.0,220045,56.416667,54.833333,NaN,NaN,71.224935,2.45,281.083333,11.461538,20.0,NaN,...,7.877273,NaN,NaN,2248.0,2100.0,NaN,1.75,20.475000,NaN,NaN
99991,151118,226241.0,220045,NaN,NaN,NaN,NaN,100.394637,NaN,NaN,11.200000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.100000,NaN,NaN
99992,197084,242052.0,220045,NaN,NaN,NaN,NaN,65.335971,NaN,NaN,12.500000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.333333,NaN,NaN


In [446]:
def _merge(table1, table2):
    return table1.merge(table2, how='left', left_on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'], right_on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'])

In [447]:
charts_outputevents=_merge(charts,outputevents)

In [448]:
charts_outputevents

,,LABEL,ALT,AST,Abdominal Girth (cm),Access Pressure,Admission Weight (Kg),Albumin,Alkaline Phosphate,Anion gap,Apnea Interval,Apnea Time Interval,...,OR Out OR Urine,Oral Gastric,Stool,TF Residual,Ultrafiltrate Ultrafiltrate,Urine Out Condom Cath,Urine Out Foley,Urine Out Suprapubic,Urine Out Void,Void
SUBJECT_ID,HADM_ID,ICUSTAY_ID,,,,,,,,,,,,,,,,,,,,,
2,163353,243653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,145834,211552.0,22.500000,68.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25.0,NaN,NaN,NaN,NaN,142.943396,NaN,NaN,NaN
4,185777,294638.0,24.000000,64.000000,NaN,NaN,NaN,2.80,NaN,NaN,NaN,NaN,...,NaN,NaN,150.0,NaN,NaN,NaN,NaN,NaN,508.333333,NaN
5,178980,214757.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,107064,228232.0,9.000000,54.666667,NaN,NaN,NaN,3.10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,80.689655,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99985,176670,279638.0,56.416667,54.833333,NaN,NaN,71.224935,2.45,281.083333,11.461538,20.0,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,178.800000,NaN,NaN,330.0
99991,151118,226241.0,NaN,NaN,NaN,NaN,100.394637,NaN,NaN,11.200000,NaN,NaN,...,NaN,300.0,NaN,NaN,NaN,NaN,143.500000,NaN,NaN,NaN
99992,197084,242052.0,NaN,NaN,NaN,NaN,65.335971,NaN,NaN,12.500000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,134.032258,NaN,NaN,NaN


In [449]:
charts_outputevents.to_csv(output_files+'charts_outputevents.csv')

# INPUTEVENTS_CV File

In [582]:
inputevents_cv=pd.read_csv(mimic_path+"INPUTEVENTS_CV.csv.gz")

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\3628261418.py:1: DtypeWarning: Columns (7,9,17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  inputevents_cv=pd.read_csv(mimic_path+"INPUTEVENTS_CV.csv.gz")


In [583]:
inputevents_cv

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE
0,592,24457,184834.0,205776.0,2193-09-11 09:00:00,30056,100.0,ml,NaN,NaN,...,756654,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,593,24457,184834.0,205776.0,2193-09-11 12:00:00,30056,200.0,ml,NaN,NaN,...,3564075,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,594,24457,184834.0,205776.0,2193-09-11 16:00:00,30056,160.0,ml,NaN,NaN,...,422646,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,595,24457,184834.0,205776.0,2193-09-11 19:00:00,30056,240.0,ml,NaN,NaN,...,5137889,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,596,24457,184834.0,205776.0,2193-09-11 21:00:00,30056,50.0,ml,NaN,NaN,...,8343792,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527930,17422687,27946,169481.0,212466.0,2154-07-16 22:00:00,40864,82.0,ml,NaN,NaN,...,10296124,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17527931,17422688,27946,169481.0,212466.0,2154-07-16 23:00:00,40864,82.0,ml,NaN,NaN,...,10248887,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17527932,17422689,27946,169481.0,212466.0,2154-07-17 00:00:00,40864,NaN,ml,NaN,NaN,...,7418343,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17527933,17422690,30516,132490.0,260091.0,2105-08-25 06:30:00,46046,31.0,ml,NaN,NaN,...,1753414,4180996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [584]:
inputevents_cv=item_merge(inputevents_cv, d_c)
inputevents_cv

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY
0,592,24457,184834.0,205776.0,2193-09-11 09:00:00,30056,100.0,ml,NaN,NaN,...,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN
1,593,24457,184834.0,205776.0,2193-09-11 12:00:00,30056,200.0,ml,NaN,NaN,...,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN
2,594,24457,184834.0,205776.0,2193-09-11 16:00:00,30056,160.0,ml,NaN,NaN,...,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN
3,595,24457,184834.0,205776.0,2193-09-11 19:00:00,30056,240.0,ml,NaN,NaN,...,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN
4,596,24457,184834.0,205776.0,2193-09-11 21:00:00,30056,50.0,ml,NaN,NaN,...,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527930,17421052,8786,132435.0,258871.0,2152-09-13 00:00:00,45347,150.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Banana Bag,Free Form Intake
17527931,17421053,8786,132435.0,258871.0,2152-09-13 01:00:00,45347,150.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Banana Bag,Free Form Intake
17527932,17421054,8786,132435.0,258871.0,2152-09-13 02:00:00,45347,150.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Banana Bag,Free Form Intake
17527933,17421055,8786,132435.0,258871.0,2152-09-13 03:00:00,45347,0.0,NaN,NaN,NaN,...,D/C'd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Banana Bag,Free Form Intake


In [585]:
labels_incv=inputevents_cv['LABEL'].tolist()

In [586]:
labels_incv_unique=unique(labels_incv)

In [587]:
for i in labels_incv_unique:
    if str(i)=='nan':
        labels_incv_unique.remove(i)
counts={}
for col in labels_incv_unique:
    counts.update({col:len(inputevents_cv[inputevents_cv.LABEL.isin([col])])})
import operator
countslist= sorted(counts.items(), key=operator.itemgetter(1), reverse=True) 
countssortdict=dict(countslist)
selcols=[]
j=0
for i in countslist:
    selcols.append(i[0])
    if j>=150:
        break
    j+=1

In [589]:
added_cols=['Protonix',
'Protonix 80mg/250cc',
'Protonix Drip',
'Protonix GTT',
'Protonix IV',
'Protonix gtt',
'Protonix gtt 8mg/hr',
'Protonix mg/hr']
selcols=np.unique(selcols+added_cols)

In [590]:
inpat_cols_dict={
'D5W':'D5W',
'Dextrose 10%':'D5W',
'Insulin':'Insulin',
'Regular Insulin':'Insulin',
'Neosynephrine-k':'Neosynephrine',
'Neosynephrine':'Neosynephrine',
'Heparin':'Heparin',
'Fentanyl':'Fentanyl',
'Fentanyl (Conc)':'Fentanyl',
'Fentanyl Base':'Fentanyl',
'Fentanyl Drip':'Fentanyl',
'Fentanyl bolus:':'Fentanyl',
'Midazolam':'Midazolam',
'Levophed-k':'Levophed',
'Levophed':'Levophed',
'KCL':'KCL',
'KCL 10 MEQ':'KCL',
'KCL 10meq/100ml':'KCL',
'KCL 10mmol in 100sw':'KCL',
'KCL 20mEq bolus':'KCL',
'KCL 40 MEQ IN 500MLS':'KCL',
'KCL CVVHDF':'KCL',
'KCL for CVVH':'KCL',
'Amiodarone':'Amiodarone',
'Nitroglycerine-k':'Nitroglycerine',
'Nitroglycerine':'Nitroglycerine',
'Ultracal':'Ultracal',
'TPN':'TPN',
'TPN D10.0':'TPN',
'TPN w/Lipids':'TPN',
'TPN D11.0':'TPN',
'TPN D12.5':'TPN',
'TPN D12.0':'TPN',
'TPN D15.0':'TPN',
'TPN D13.0':'TPN',
'TPN D9.0':'TPN',
'TPN D14':'TPN',
'TPN D8':'TPN',
'TPN D5.0':'TPN',
'TPN D7.5':'TPN',
'TPN D7.0':'TPN',
'TPN D6':'TPN',
'TPN D16':'TPN',
'TPN D20.0':'TPN',
'D5/.45NS':'D5/.45NS',
'Lipids 20%':'Lipids 20%', 
}

In [591]:
inputevents_cv[inputevents_cv.LABEL.isin(['TPN',
 'TPN D10.0',
 'TPN w/Lipids',
 'TPN D11.0',
 'TPN D12.5',
 'TPN D12.0',
 'TPN D15.0',
 'TPN D13.0',
 'TPN D9.0',
 'TPN D14',
 'TPN D8',
 'TPN D5.0',
 'TPN D7.5',
 'TPN D7.0',
 'TPN D6',
 'TPN D16',
 'TPN D20.0'])]


inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN','TPN', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D10.0','TPN', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN w/Lipids','TPN', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D11.0','TPN',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D12.5','TPN',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D12.0','TPN',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D15.0','TPN',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D13.0','TPN',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D9.0','TPN',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D14','TPN',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D8','TPN',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D5.0','TPN',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D7.5','TPN',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D7.0','TPN',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D6','TPN',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D16','TPN',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'TPN D20.0','TPN',inplace=True)


inputevents_cv[inputevents_cv.LABEL.isin(['TPN'])]


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY
6593381,4508963,18248,143950.0,266884.0,2174-08-05 03:00:00,30032,40.0,ml,NaN,NaN,...,NaN,NaN,1000.0,ml,Intravenous Push,NaN,NaN,NaN,TPN,NaN
6593382,4508964,18248,143950.0,266884.0,2174-08-05 04:00:00,30032,40.0,ml,NaN,NaN,...,NaN,NaN,1000.0,ml,Intravenous Push,NaN,NaN,NaN,TPN,NaN
6593383,4508965,18248,143950.0,266884.0,2174-08-05 05:00:00,30032,40.0,ml,NaN,NaN,...,NaN,NaN,1000.0,ml,Intravenous Push,NaN,NaN,NaN,TPN,NaN
6593384,4508966,18248,143950.0,266884.0,2174-08-05 06:00:00,30032,40.0,ml,NaN,NaN,...,NaN,NaN,1000.0,ml,Intravenous Push,NaN,NaN,NaN,TPN,NaN
6593385,4508967,18248,143950.0,266884.0,2174-08-05 07:00:00,30032,40.0,ml,NaN,NaN,...,NaN,NaN,1000.0,ml,Intravenous Push,NaN,NaN,NaN,TPN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17170166,8707367,15114,192621.0,267302.0,2133-05-01 10:00:00,30301,2.9,ml,NaN,NaN,...,NaN,NaN,500.0,cc,Intravenous,NaN,NaN,NaN,TPN,NaN
17170167,8707368,15114,192621.0,267302.0,2133-05-01 11:00:00,30301,4.6,ml,NaN,NaN,...,NaN,NaN,500.0,cc,Intravenous,NaN,NaN,NaN,TPN,NaN
17170168,8707369,15114,192621.0,267302.0,2133-05-01 12:00:00,30301,5.2,ml,NaN,NaN,...,NaN,NaN,500.0,cc,Intravenous,NaN,NaN,NaN,TPN,NaN
17170169,8707370,15114,192621.0,267302.0,2133-05-01 13:00:00,30301,4.1,ml,NaN,NaN,...,NaN,NaN,500.0,cc,Intravenous,NaN,NaN,NaN,TPN,NaN


In [592]:
for col in selcols:
    if col not in inpat_cols_dict:
        inpat_cols_dict.update({col:col})

In [594]:
inputevents_cv[inputevents_cv.LABEL.isin(['Dopamine',])]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY
10106754,8925649,7605,107327.0,276732.0,2147-01-19 17:00:00,30043,49.500000,mg,NaN,NaN,...,NaN,NaN,400.0,mg,Intravenous Push,NaN,NaN,NaN,Dopamine,NaN
10106755,8926242,590,172260.0,258414.0,2124-11-14 04:00:00,30043,53.100000,mg,NaN,NaN,...,NaN,NaN,400.0,mg,Intravenous Push,11.0625,ml/hr,Right Int. Jugular,Dopamine,NaN
10106756,8926243,12744,116766.0,270724.0,2166-12-09 04:00:00,30043,34.200000,mg,NaN,NaN,...,NaN,NaN,400.0,mg,Intravenous Push,NaN,NaN,NaN,Dopamine,NaN
10106757,8926244,2128,152495.0,208642.0,2166-11-25 04:00:00,30043,58.559991,mg,NaN,NaN,...,NaN,NaN,400.0,mg,Intravenous Push,NaN,NaN,NaN,Dopamine,NaN
10106758,8926245,20866,147338.0,208231.0,2198-12-25 04:00:00,30043,8.025000,mg,NaN,NaN,...,NaN,NaN,400.0,mg,Intravenous Push,NaN,NaN,NaN,Dopamine,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10280494,17459828,29601,188620.0,241690.0,2135-09-16 10:00:00,30043,NaN,NaN,5.0,mcgkgmin,...,NaN,NaN,NaN,NaN,IV Drip,NaN,NaN,NaN,Dopamine,NaN
10280495,17460660,30286,193809.0,263130.0,2181-08-19 02:00:00,30043,NaN,NaN,8.0,mcgkgmin,...,NaN,NaN,NaN,NaN,IV Drip,NaN,NaN,NaN,Dopamine,NaN
10280496,17460667,30286,193809.0,263130.0,2181-08-19 03:00:00,30043,NaN,NaN,8.0,mcgkgmin,...,NaN,NaN,NaN,NaN,IV Drip,NaN,NaN,NaN,Dopamine,NaN
10280497,17460678,30286,193809.0,263130.0,2181-08-19 04:00:00,30043,NaN,NaN,8.0,mcgkgmin,...,NaN,NaN,NaN,NaN,IV Drip,NaN,NaN,NaN,Dopamine,NaN


In [595]:
file=open(output_files+'unique_labels_input_CV.txt','w')
for item in labels_incv_unique:
    file.write(item+'\n')

file.close()

In [596]:
inputevents_cv['LABEL'] = inputevents_cv['LABEL'].astype('category')
inputevents_cv['LABEL'].cat.set_categories(selcols,inplace=True)
inputevents_cv=inputevents_cv.dropna(how='any', subset=['AMOUNT']) # This removes rows with null values for VALUENUM column (a total of 190,141 records removed)

C:\Users\Mariam\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arrays\categorical.py:2747: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


In [597]:

inputevents_cv[inputevents_cv.LABEL.isin(['Dextrose 10%','D5W'])]
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='Dextrose 10%','D5W', inplace=True)
inputevents_cv[inputevents_cv.LABEL.isin(['D5W'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\3526124679.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='Dextrose 10%','D5W', inplace=True)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY
188107,188542,21776,197813.0,208688.0,2133-11-16 04:00:00,30013,50.0,ml,NaN,NaN,...,NaN,NaN,100.0,ml,Intravenous Push,NaN,NaN,NaN,D5W,NaN
188108,188543,21776,197813.0,208688.0,2133-11-16 06:00:00,30013,50.0,ml,NaN,NaN,...,NaN,NaN,100.0,ml,Intravenous Push,NaN,NaN,NaN,D5W,NaN
188109,188544,21776,197813.0,208688.0,2133-11-16 08:00:00,30013,100.0,ml,NaN,NaN,...,NaN,NaN,100.0,ml,Intravenous Push,NaN,NaN,NaN,D5W,NaN
188110,188545,21776,197813.0,208688.0,2133-11-16 20:00:00,30013,150.0,ml,NaN,NaN,...,NaN,NaN,100.0,ml,Intravenous Push,NaN,NaN,NaN,D5W,NaN
188111,188546,21776,197813.0,208688.0,2133-11-17 03:00:00,30013,150.0,ml,NaN,NaN,...,NaN,NaN,100.0,ml,Intravenous Push,NaN,NaN,NaN,D5W,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6207673,9007137,27532,129997.0,243721.0,2149-08-26 22:00:00,30016,150.0,ml,NaN,NaN,...,NaN,NaN,2000.0,ml,Intravenous Push,150.0,ml/hr,NaN,D5W,NaN
6207674,9007138,27532,129997.0,243721.0,2149-08-26 23:00:00,30016,150.0,ml,NaN,NaN,...,NaN,NaN,2000.0,ml,Intravenous Push,150.0,ml/hr,NaN,D5W,NaN
6207675,9007139,27532,129997.0,243721.0,2149-08-27 00:00:00,30016,150.0,ml,NaN,NaN,...,NaN,NaN,2000.0,ml,Intravenous Push,150.0,ml/hr,NaN,D5W,NaN
6207676,9009799,27532,129997.0,243721.0,2149-08-27 01:00:00,30016,150.0,ml,NaN,NaN,...,NaN,NaN,2000.0,ml,Intravenous Push,150.0,ml/hr,NaN,D5W,NaN


In [598]:
inputevents_cv[inputevents_cv.LABEL.isin(['Levophed-k','Levophed'])]
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='Levophed-k','Levophed', inplace=True)
inputevents_cv[inputevents_cv.LABEL.isin(['Levophed'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\1898702062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='Levophed-k','Levophed', inplace=True)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY
12909636,8925663,14076,146527.0,255856.0,2146-01-14 03:00:00,30120,0.77748,mg,NaN,NaN,...,NaN,NaN,8.0,mg,Intravenous Push,22.087502,ml/hr,NaN,Levophed,NaN
12909637,8925668,2103,158138.0,216788.0,2177-01-03 03:00:00,30120,0.92340,mg,NaN,NaN,...,NaN,NaN,8.0,mg,Intravenous Push,40.078125,ml/hr,NaN,Levophed,NaN
12909638,8925112,5872,131850.0,261477.0,2110-12-06 18:00:00,30120,1.79200,mg,NaN,NaN,...,NaN,NaN,8.0,mg,Intravenous Push,31.125000,ml/hr,NaN,Levophed,NaN
12909639,8925118,4602,189601.0,214838.0,2189-11-14 18:00:00,30120,1.02850,mg,NaN,NaN,...,NaN,NaN,16.0,mg,Intravenous Push,20.400000,ml/hr,NaN,Levophed,NaN
12909640,8925128,5872,131850.0,261477.0,2110-12-07 06:00:00,30120,0.74700,mg,NaN,NaN,...,NaN,NaN,8.0,mg,Intravenous Push,31.125000,ml/hr,NaN,Levophed,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13905809,9348138,773,171563.0,297845.0,2103-07-22 12:00:00,30047,2.06250,mg,NaN,NaN,...,NaN,NaN,16.0,mg,Intravenous Push,28.125000,ml/hr,NaN,Levophed,NaN
13905810,9348147,24711,143442.0,218302.0,2175-05-07 12:00:00,30047,6.00000,mg,NaN,NaN,...,NaN,NaN,16.0,mg,Intravenous Push,56.250000,ml/hr,NaN,Levophed,NaN
13905811,9328978,24711,143442.0,218302.0,2175-05-04 11:15:00,30047,1.20000,mg,NaN,NaN,...,NaN,NaN,4.0,mg,Intravenous Push,52.500000,ml/hr,NaN,Levophed,NaN
13905812,9345345,24711,143442.0,218302.0,2175-05-05 19:00:00,30047,0.12000,mg,NaN,NaN,...,NaN,NaN,8.0,mg,Intravenous Push,48.750000,ml/hr,NaN,Levophed,NaN


In [599]:
inputevents_cv[inputevents_cv.LABEL.isin(['Neosynephrine-k','Neosynephrine'])]
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='Neosynephrine-k','Neosynephrine', inplace=True)
inputevents_cv[inputevents_cv.LABEL.isin(['Neosynephrine'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\1386385433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='Neosynephrine-k','Neosynephrine', inplace=True)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY
12355054,8925661,6104,167627.0,258768.0,2190-12-03 03:00:00,30128,16.055999,mg,NaN,NaN,...,NaN,NaN,120.0,mg,Intravenous Push,16.724998,ml/hr,NaN,Neosynephrine,NaN
12355055,8926223,15871,187119.0,273828.0,2153-01-23 04:00:00,30128,3.952500,mg,NaN,NaN,...,NaN,NaN,60.0,mg,Intravenous Push,NaN,NaN,NaN,Neosynephrine,NaN
12355056,8943891,24835,123529.0,261786.0,2131-01-22 02:00:00,30128,1.176000,mg,NaN,NaN,...,NaN,NaN,60.0,mg,Intravenous Push,9.875000,ml/hr,NaN,Neosynephrine,NaN
12355057,8927378,6668,107239.0,268587.0,2124-01-03 04:00:00,30128,2.970000,mg,NaN,NaN,...,NaN,NaN,60.0,mg,Intravenous Push,NaN,NaN,NaN,Neosynephrine,NaN
12355058,8927408,10168,153885.0,278075.0,2110-12-08 05:00:00,30128,2.466000,mg,NaN,NaN,...,NaN,NaN,120.0,mg,Intravenous Push,38.017498,ml/hr,NaN,Neosynephrine,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15959738,9332810,17167,185509.0,230366.0,2138-05-18 11:30:00,30127,0.540000,mg,NaN,NaN,...,NaN,NaN,20.0,mg,Intravenous Push,15.000000,ml/hr,NaN,Neosynephrine,NaN
15959739,9330043,17167,185509.0,230366.0,2138-05-17 22:00:00,30127,0.800000,mg,NaN,NaN,...,NaN,NaN,20.0,mg,Intravenous Push,15.000000,ml/hr,NaN,Neosynephrine,NaN
15959740,9330054,17167,185509.0,230366.0,2138-05-18 10:00:00,30127,0.720000,mg,NaN,NaN,...,NaN,NaN,20.0,mg,Intravenous Push,15.000000,ml/hr,NaN,Neosynephrine,NaN
15959741,9348112,1615,143801.0,289394.0,2125-06-10 09:00:00,30127,2.400000,mg,NaN,NaN,...,NaN,NaN,60.0,mg,Intravenous Push,50.000000,ml/hr,NaN,Neosynephrine,NaN


In [600]:
inputevents_cv[inputevents_cv.LABEL.isin(['Nitroglycerine-k','Nitroglycerine'])]
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='Nitroglycerine-k','Nitroglycerine', inplace=True)
inputevents_cv[inputevents_cv.LABEL.isin(['Nitroglycerine'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\1869639634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='Nitroglycerine-k','Nitroglycerine', inplace=True)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY
12115786,8925660,2103,158138.0,216788.0,2177-01-02 18:00:00,30121,0.513000,mg,NaN,NaN,...,NaN,NaN,100.0,mg,Intravenous Push,NaN,NaN,NaN,Nitroglycerine,NaN
12115787,8926800,3049,158811.0,204936.0,2160-01-14 16:00:00,30121,27.600000,mg,NaN,NaN,...,NaN,NaN,200.0,mg,Intravenous Push,34.5,ml/hr,NaN,Nitroglycerine,NaN
12115788,8943332,8163,158480.0,286231.0,2116-11-14 10:00:00,30121,16.560001,mg,NaN,NaN,...,NaN,NaN,100.0,mg,Intravenous Push,NaN,NaN,NaN,Nitroglycerine,NaN
12115789,8926810,3049,158811.0,204936.0,2160-01-14 17:00:00,30121,27.600000,mg,NaN,NaN,...,NaN,NaN,200.0,mg,Intravenous Push,34.5,ml/hr,NaN,Nitroglycerine,NaN
12115790,8926833,20260,102548.0,217457.0,2125-01-22 20:00:00,30121,4.764000,mg,NaN,NaN,...,NaN,NaN,100.0,mg,Intravenous Push,NaN,NaN,NaN,Nitroglycerine,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15077614,9348662,16425,154070.0,257039.0,2199-05-28 20:00:00,30049,0.000000,mg,NaN,NaN,...,NaN,1.0,100.0,mg,Intravenous Push,NaN,NaN,NaN,Nitroglycerine,NaN
15077615,9329488,11318,130910.0,251468.0,2120-06-24 13:00:00,30049,4.000000,mg,NaN,NaN,...,NaN,NaN,100.0,mg,Intravenous Push,NaN,NaN,NaN,Nitroglycerine,NaN
15077616,9348116,740,184078.0,265199.0,2169-06-25 09:00:00,30049,36.000000,mg,NaN,NaN,...,NaN,NaN,100.0,mg,Intravenous Push,30.0,ml/hr,NaN,Nitroglycerine,NaN
15077617,9329506,11318,130910.0,251468.0,2120-06-24 22:00:00,30049,40.500000,mg,NaN,NaN,...,NaN,NaN,100.0,mg,Intravenous Push,NaN,NaN,NaN,Nitroglycerine,NaN


In [601]:
inputevents_cv[inputevents_cv.LABEL.isin(['Protonix',
'Protonix 80mg/250cc',
'Protonix Drip',
'Protonix GTT',
'Protonix IV',
'Protonix gtt',
'Protonix gtt 8mg/hr',
'Protonix mg/hr'])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix 80mg/250cc', 'Protonix', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix Drip', 'Protonix', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix GTT', 'Protonix', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix IV', 'Protonix', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix gtt', 'Protonix', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix gtt 8mg/hr', 'Protonix', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix mg/hr', 'Protonix', inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['Protonix'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\928401066.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix 80mg/250cc', 'Protonix', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\928401066.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix Drip', 'Protonix', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\928401066.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY
17441240,17431226,28868,145654.0,229203.0,2141-09-26 02:00:00,42204,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake
17441241,17431227,28868,145654.0,229203.0,2141-09-26 03:00:00,42204,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake
17441242,17431228,28868,145654.0,229203.0,2141-09-26 04:00:00,42204,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake
17441243,17431229,28868,145654.0,229203.0,2141-09-26 05:00:00,42204,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake
17441244,17431230,28868,145654.0,229203.0,2141-09-26 06:00:00,42204,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527662,17458900,389,134048.0,220654.0,2104-04-18 22:00:00,45024,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake
17527663,17458901,389,134048.0,220654.0,2104-04-18 23:00:00,45024,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake
17527664,17458902,389,134048.0,220654.0,2104-04-19 00:00:00,45024,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake
17527665,17458903,389,134048.0,220654.0,2104-04-19 02:00:00,45024,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake


In [602]:
inputevents_cv[inputevents_cv.LABEL.isin([
'Fentanyl',
'Fentanyl (Conc)',
'Fentanyl Base',
'Fentanyl Drip',
'Fentanyl bolus:',])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Fentanyl (Conc)', 'Fentanyl', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Fentanyl Base', 'Fentanyl', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Fentanyl Drip', 'Fentanyl', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Fentanyl bolus:', 'Fentanyl', inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['Fentanyl'])]

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\1719266916.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Fentanyl (Conc)', 'Fentanyl', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\1719266916.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Fentanyl Base', 'Fentanyl', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\1719266916.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: h

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY
7281469,6010210,28972,134622.0,229355.0,2127-11-29 16:00:00,30150,2.0,ml,NaN,NaN,...,NaN,NaN,50.0,ml,Intravenous Push,2.000,ml/hr,NaN,Fentanyl,NaN
7281470,6010211,28972,134622.0,229355.0,2127-11-29 17:00:00,30150,2.0,ml,NaN,NaN,...,NaN,NaN,50.0,ml,Intravenous Push,2.000,ml/hr,NaN,Fentanyl,NaN
7281471,6010212,28972,134622.0,229355.0,2127-11-29 18:00:00,30150,2.0,ml,NaN,NaN,...,NaN,NaN,50.0,ml,Intravenous Push,2.000,ml/hr,NaN,Fentanyl,NaN
7281472,6010213,28972,134622.0,229355.0,2127-11-29 19:00:00,30150,2.0,ml,NaN,NaN,...,NaN,NaN,50.0,ml,Intravenous Push,2.000,ml/hr,NaN,Fentanyl,NaN
7281473,6010214,28972,134622.0,229355.0,2127-11-29 20:00:00,30150,2.0,ml,NaN,NaN,...,NaN,NaN,50.0,ml,Intravenous Push,2.000,ml/hr,NaN,Fentanyl,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16874950,13060736,27321,165983.0,274907.0,2159-12-09 22:00:00,30308,2.5,mcg,NaN,NaN,...,NaN,NaN,500.0,mcg,Intravenous,0.231,ml/hr,Percutaneous Cath,Fentanyl,NaN
16874951,13060846,31892,163351.0,240409.0,2154-02-04 23:00:00,30308,1.7,mcg,NaN,NaN,...,NaN,NaN,500.0,mcg,Intravenous,NaN,NaN,NaN,Fentanyl,NaN
16874952,13060850,27321,165983.0,274907.0,2159-12-10 23:00:00,30308,3.5,mcg,NaN,NaN,...,NaN,NaN,500.0,mcg,Intravenous,0.231,ml/hr,Percutaneous Cath,Fentanyl,NaN
16874953,13060897,31892,163351.0,240409.0,2154-02-05 07:00:00,30308,1.4,mcg,NaN,NaN,...,NaN,NaN,500.0,mcg,Intravenous,NaN,NaN,NaN,Fentanyl,NaN


In [603]:
inputevents_cv[inputevents_cv.LABEL.isin(['KCL',
'KCL 10 MEQ',
'KCL 10meq/100ml',
'KCL 10mmol in 100sw',
'KCL 20mEq bolus',
'KCL 40 MEQ IN 500MLS',
'KCL CVVHDF',
'KCL for CVVH',])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL 10 MEQ', 'KCL', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL 10meq/100ml', 'KCL', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL 10mmol in 100sw', 'KCL', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL 20mEq bolus', 'KCL', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL 40 MEQ IN 500MLS', 'KCL', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL CVVHDF', 'KCL', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL for CVVH', 'KCL', inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['KCL'])]


C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\2259238019.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL 10 MEQ', 'KCL', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\2259238019.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL 10meq/100ml', 'KCL', inplace=True)
C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\2259238019.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pand

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY
9815085,8925648,21842,174955.0,250436.0,2140-11-24 17:00:00,30026,0.000000,mEq,NaN,NaN,...,NaN,NaN,40.0,mEq,Intravenous Push,NaN,NaN,NaN,KCL,NaN
9815086,8925659,21857,173189.0,269052.0,2116-11-05 18:00:00,30026,8.346456,mEq,NaN,NaN,...,NaN,NaN,100.0,mEq,Intravenous Push,63.0,ml/hr,NaN,KCL,NaN
9815087,8943326,9175,148433.0,218471.0,2194-12-20 10:00:00,30026,40.000000,mEq,NaN,NaN,...,NaN,NaN,40.0,mEq,Intravenous Push,NaN,NaN,NaN,KCL,NaN
9815088,8943328,18605,173380.0,264352.0,2188-12-13 10:00:00,30026,4.000000,mEq,NaN,NaN,...,NaN,NaN,20.0,mEq,Intravenous Push,80.0,ml/hr,NaN,KCL,NaN
9815089,8943345,6960,178048.0,298934.0,2120-11-24 00:00:00,30026,40.000000,mEq,NaN,NaN,...,NaN,NaN,40.0,mEq,Intravenous Push,NaN,NaN,NaN,KCL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10106749,17438104,9773,136342.0,273354.0,2102-04-07 18:00:00,30026,20.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KCL,NaN
10106750,17438105,9773,136342.0,273354.0,2102-04-07 19:00:00,30026,20.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KCL,NaN
10106751,17438106,9773,136342.0,273354.0,2102-04-07 20:00:00,30026,0.000000,NaN,NaN,NaN,...,D/C'd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KCL,NaN
10106752,17455357,9773,136342.0,273354.0,2102-04-10 01:00:00,30026,20.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KCL,NaN


In [604]:
inputevents_cv[inputevents_cv.LABEL.isin([
'Insulin',
'Regular Insulin'])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Regular Insulin','Insulin', inplace=True)
inputevents_cv[inputevents_cv.LABEL.isin(['Insulin'])]


C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\1830979993.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Regular Insulin','Insulin', inplace=True)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY
13750741,8926212,21857,173189.0,269052.0,2116-11-10 04:00:00,30100,0.427822,U,NaN,NaN,...,NaN,NaN,10.0,U,Intravenous Push,63.0,ml/hr,NaN,Insulin,NaN
13750742,8926229,8449,102933.0,222892.0,2105-12-29 04:00:00,30100,1.080000,U,NaN,NaN,...,NaN,NaN,20.0,U,Intravenous Push,54.0,ml/hr,NaN,Insulin,NaN
13750743,8943897,8163,158480.0,286231.0,2116-11-23 02:00:00,30100,7.560000,U,NaN,NaN,...,NaN,NaN,60.0,U,Intravenous Push,NaN,NaN,NaN,Insulin,NaN
13750744,8925107,21857,173189.0,269052.0,2116-10-31 18:00:00,30100,0.413386,U,NaN,NaN,...,NaN,1.0,10.0,U,Intravenous Push,63.0,ml/hr,NaN,Insulin,NaN
13750745,8925126,21857,173189.0,269052.0,2116-11-01 06:00:00,30100,0.413386,U,NaN,NaN,...,NaN,NaN,10.0,U,Intravenous Push,63.0,ml/hr,NaN,Insulin,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14412844,13381126,32403,104048.0,290132.0,2129-10-12 02:00:00,30045,5.000000,U,NaN,NaN,...,NaN,NaN,100.0,U,Intravenous Push,NaN,NaN,NaN,Insulin,NaN
14412845,13381127,32545,194185.0,278072.0,2153-09-12 02:00:00,30045,1.000000,U,NaN,NaN,...,NaN,NaN,100.0,U,Intravenous Push,NaN,NaN,NaN,Insulin,NaN
14412846,13381128,32403,104048.0,290132.0,2129-10-13 00:00:00,30045,6.000000,U,NaN,NaN,...,NaN,NaN,100.0,U,Intravenous Push,NaN,NaN,NaN,Insulin,NaN
14412847,13381129,31232,185793.0,209957.0,2179-09-09 00:00:00,30045,4.000000,U,NaN,NaN,...,NaN,NaN,100.0,U,Intravenous Push,NaN,NaN,NaN,Insulin,NaN


In [605]:
inputevents_cv['LABEL'] = inputevents_cv['LABEL'].map(inpat_cols_dict)

C:\Users\Mariam\AppData\Local\Temp\ipykernel_10372\4046842139.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_cv['LABEL'] = inputevents_cv['LABEL'].map(inpat_cols_dict)


In [606]:
inputevents_cv=inputevents_cv.pivot_table(values='AMOUNT', index=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID'], columns='LABEL')

In [607]:
inputevents_cv

LABEL                                  .45% Normal Saline  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                       
3          145834.0 211552.0   30001                  NaN   
                               30013                  NaN   
                               30018                  NaN   
                               30020                100.0   
                               30025                  NaN   
...                                                   ...   
32810      189751.0 204923.0   30021                  NaN   
                               30054                  NaN   
                               30059                  NaN   
                               30143                  NaN   
                               30088                  NaN   

LABEL                                  .45NS + .5:1 Heparin  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                         
3          145834.0 211552.0   30001                    NaN   
                               30013                    NaN   
                               30018                    NaN   
                               30020                    NaN   
                               30025                    NaN   
...                                                     ...   
32810      189751.0 204923.0   30021                    NaN   
                               30054                    NaN   
                               30059                    NaN   
                               30143                    NaN   
                               30088                    NaN   

LABEL                                  .45NS + 1:1 Heparin  .9% Normal Saline  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                                           
3          145834.0 211552.0   30001                   NaN                NaN   
                               30013                   NaN                NaN   
                               30018                   NaN          80.472973   
                               30020                   NaN                NaN   
                               30025                   NaN                NaN   
...                                                    ...                ...   
32810      189751.0 204923.0   30021                   NaN                NaN   
                               30054                   NaN                NaN   
                               30059                   NaN                NaN   
                               30143                   NaN                NaN   
                               30088                   NaN                NaN   

LABEL                                  3% Normal Saline  Aggrastat  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                                
3          145834.0 211552.0   30001                NaN        NaN   
                               30013                NaN        NaN   
                               30018                NaN        NaN   
                               30020                NaN        NaN   
                               30025                NaN        NaN   
...                                                 ...        ...   
32810      189751.0 204923.0   30021                NaN        NaN   
                               30054                NaN        NaN   
                               30059                NaN        NaN   
                               30143          22.222222        NaN   
                               30088                NaN        NaN   

LABEL                                  Albumin 25%  Albumin 5%  Amiodarone  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                                        
3          145834.0 211552.0   30001           NaN         NaN         NaN   
                               30013           NaN         NaN         NaN   
                               30018           NaN         NaN         NaN   
                               

In [608]:
inputevents_cv_dupdrp=inputevents_cv.drop_duplicates()
inputevents_cv_dupdrp

LABEL                                  .45% Normal Saline  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                       
3          145834.0 211552.0   30001                  NaN   
                               30013                  NaN   
                               30018                  NaN   
                               30020                100.0   
                               30025                  NaN   
...                                                   ...   
32810      189751.0 204923.0   30178                  NaN   
                               30021                  NaN   
                               30059                  NaN   
                               30143                  NaN   
                               30088                  NaN   

LABEL                                  .45NS + .5:1 Heparin  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                         
3          145834.0 211552.0   30001                    NaN   
                               30013                    NaN   
                               30018                    NaN   
                               30020                    NaN   
                               30025                    NaN   
...                                                     ...   
32810      189751.0 204923.0   30178                    NaN   
                               30021                    NaN   
                               30059                    NaN   
                               30143                    NaN   
                               30088                    NaN   

LABEL                                  .45NS + 1:1 Heparin  .9% Normal Saline  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                                           
3          145834.0 211552.0   30001                   NaN                NaN   
                               30013                   NaN                NaN   
                               30018                   NaN          80.472973   
                               30020                   NaN                NaN   
                               30025                   NaN                NaN   
...                                                    ...                ...   
32810      189751.0 204923.0   30178                   NaN                NaN   
                               30021                   NaN                NaN   
                               30059                   NaN                NaN   
                               30143                   NaN                NaN   
                               30088                   NaN                NaN   

LABEL                                  3% Normal Saline  Aggrastat  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                                
3          145834.0 211552.0   30001                NaN        NaN   
                               30013                NaN        NaN   
                               30018                NaN        NaN   
                               30020                NaN        NaN   
                               30025                NaN        NaN   
...                                                 ...        ...   
32810      189751.0 204923.0   30178                NaN        NaN   
                               30021                NaN        NaN   
                               30059                NaN        NaN   
                               30143          22.222222        NaN   
                               30088                NaN        NaN   

LABEL                                  Albumin 25%  Albumin 5%  Amiodarone  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                                        
3          145834.0 211552.0   30001           NaN         NaN         NaN   
                               30013           NaN         NaN         NaN   
                               30018           NaN         NaN         NaN   
                               

In [609]:
cols=inputevents_cv.columns

In [610]:
for col in cols:
    inputevents_cv[col] = inputevents_cv.groupby('HADM_ID')[col].transform('mean')

In [611]:
inputevents_cv

LABEL                                  .45% Normal Saline  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                       
3          145834.0 211552.0   30001                100.0   
                               30013                100.0   
                               30018                100.0   
                               30020                100.0   
                               30025                100.0   
...                                                   ...   
32810      189751.0 204923.0   30021                  NaN   
                               30054                  NaN   
                               30059                  NaN   
                               30143                  NaN   
                               30088                  NaN   

LABEL                                  .45NS + .5:1 Heparin  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                         
3          145834.0 211552.0   30001                    NaN   
                               30013                    NaN   
                               30018                    NaN   
                               30020                    NaN   
                               30025                    NaN   
...                                                     ...   
32810      189751.0 204923.0   30021                    NaN   
                               30054                    NaN   
                               30059                    NaN   
                               30143                    NaN   
                               30088                    NaN   

LABEL                                  .45NS + 1:1 Heparin  .9% Normal Saline  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                                           
3          145834.0 211552.0   30001                   NaN          80.472973   
                               30013                   NaN          80.472973   
                               30018                   NaN          80.472973   
                               30020                   NaN          80.472973   
                               30025                   NaN          80.472973   
...                                                    ...                ...   
32810      189751.0 204923.0   30021                   NaN          14.975366   
                               30054                   NaN          14.975366   
                               30059                   NaN          14.975366   
                               30143                   NaN          14.975366   
                               30088                   NaN          14.975366   

LABEL                                  3% Normal Saline  Aggrastat  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                                
3          145834.0 211552.0   30001                NaN        NaN   
                               30013                NaN        NaN   
                               30018                NaN        NaN   
                               30020                NaN        NaN   
                               30025                NaN        NaN   
...                                                 ...        ...   
32810      189751.0 204923.0   30021          22.222222        NaN   
                               30054          22.222222        NaN   
                               30059          22.222222        NaN   
                               30143          22.222222        NaN   
                               30088          22.222222        NaN   

LABEL                                  Albumin 25%  Albumin 5%  Amiodarone  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID                                        
3          145834.0 211552.0   30001           NaN         NaN   44.374997   
                               30013           NaN         NaN   44.374997   
                               30018           NaN         NaN   44.374997   
                               

In [612]:
inputevents_cv.drop_duplicates(inplace=True)

In [613]:
charts_outputevents_inputevents=_merge(charts_outputevents,inputevents_cv)

In [579]:
charts_outputevents_inputevents.to_csv(output_files+"charts_outputevents_inputevents_cv_misdrp.csv")

In [575]:
#todrop=[]
#for col in cols:
#    mis_perc=inputevents_cv[col].isnull().sum()/inputevents_cv.shape[0]
#    if mis_perc>0.70:
#        todrop.append(col)
#inputevents_cv.drop(todrop,inplace=True,axis=1)

In [577]:
inputevents_cv.to_csv(output_files+"inputevents_cv.csv")

# INPUTEVENTS_MV File

In [614]:
inputevents_mv=pd.read_csv(mimic_path+"INPUTEVENTS_MV.csv.gz")

In [615]:
inputevents_mv

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,TOTALAMOUNTUOM,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE
0,241,27063,139787,223259.0,2133-02-05 06:29:00,2133-02-05 08:45:00,225166,6.774532,mEq,NaN,...,ml,0,0,1,Rewritten,NaN,RN,2133-02-05 12:52:00,10.000000,0.050000
1,242,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 06:30:00,225944,28.132997,ml,30.142497,...,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,28.132998,30.255817
2,243,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 06:30:00,225166,2.813300,mEq,NaN,...,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,2.813300,0.050426
3,244,27063,139787,223259.0,2133-02-03 12:00:00,2133-02-03 12:01:00,225893,1.000000,dose,NaN,...,ml,0,0,2,Rewritten,RN,NaN,2133-02-03 17:06:00,1.000000,1.000000
4,245,27063,139787,223259.0,2133-02-03 12:00:00,2133-02-03 12:01:00,220949,100.000000,ml,NaN,...,ml,0,0,2,Rewritten,RN,NaN,2133-02-03 17:06:00,100.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3618986,3568968,90959,136680,240741.0,2147-08-28 12:00:00,2147-08-28 12:01:00,221744,99.999997,mcg,NaN,...,NaN,0,0,0,FinishedRunning,NaN,NaN,NaN,100.000000,100.000000
3618987,3568969,90959,136680,240741.0,2147-08-29 12:16:00,2147-08-29 15:04:00,225942,0.842267,mg,300.809532,...,ml,0,0,0,Paused,NaN,NaN,NaN,2.500000,300.000000
3618988,3568970,90959,136680,240741.0,2147-08-29 12:16:00,2147-08-29 15:04:00,225943,16.845331,ml,6.016190,...,ml,0,0,0,Paused,NaN,NaN,NaN,50.000000,6.000000
3618989,3568971,90959,136680,240741.0,2147-08-29 02:30:00,2147-08-29 02:31:00,221744,99.999997,mcg,NaN,...,NaN,0,0,0,FinishedRunning,NaN,NaN,NaN,100.000000,100.000000


In [616]:
inputevents_mv=item_merge(inputevents_mv, d_c)
inputevents_mv

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE,LABEL,CATEGORY
0,241,27063,139787,223259.0,2133-02-05 06:29:00,2133-02-05 08:45:00,225166,6.774532,mEq,NaN,...,0,1,Rewritten,NaN,RN,2133-02-05 12:52:00,10.000000,0.050000,Potassium Chloride,Medications
1,243,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 06:30:00,225166,2.813300,mEq,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,2.813300,0.050426,Potassium Chloride,Medications
2,247,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 07:03:00,225166,4.433333,mEq,NaN,...,0,0,Rewritten,NaN,NaN,NaN,4.433333,0.050000,Potassium Chloride,Medications
3,249,27063,139787,223259.0,2133-02-05 09:43:00,2133-02-05 12:30:00,225166,8.360021,mEq,NaN,...,0,0,Changed,NaN,NaN,NaN,10.000001,0.050248,Potassium Chloride,Medications
4,251,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 05:35:00,225166,0.050060,mEq,NaN,...,0,0,Rewritten,NaN,NaN,NaN,2.753300,0.050248,Potassium Chloride,Medications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3618986,3588947,60353,111636,248214.0,2137-09-08 10:30:00,2137-09-08 17:00:00,228141,65.000001,ml,10.000000,...,0,0,Changed,NaN,NaN,NaN,500.000000,10.000000,Dextrose 30%,Fluids/Intake
3618987,3589021,60353,111636,248214.0,2137-09-08 20:00:00,2137-09-09 02:36:00,228141,59.414343,ml,9.002173,...,0,0,Stopped,NaN,NaN,NaN,408.998720,9.000000,Dextrose 30%,Fluids/Intake
3618988,3588965,60353,111636,248214.0,2137-09-08 19:00:00,2137-09-08 20:00:00,228141,8.001279,ml,8.001279,...,0,0,Changed,NaN,NaN,NaN,417.000000,8.000000,Dextrose 30%,Fluids/Intake
3618989,3588921,60353,111636,248214.0,2137-09-08 17:00:00,2137-09-08 19:00:00,228141,18.000001,ml,9.000001,...,0,0,Changed,NaN,NaN,NaN,435.000000,9.000000,Dextrose 30%,Fluids/Intake


In [617]:
labels_inmv_unique=np.unique(inputevents_mv['LABEL'].tolist())

for i in labels_inmv_unique:
    if str(i)=='nan':
        labels_inmv_unique.remove(i)
counts={}
for col in labels_incv_unique:
    counts.update({col:len(inputevents_mv[inputevents_mv.LABEL.isin([col])])})
import operator
countslist= sorted(counts.items(), key=operator.itemgetter(1), reverse=True) 
countssortdict=dict(countslist)
selcols=[]
j=0
for i in countslist:
    selcols.append(i[0])
    if j>=18:
        break
    j+=1

In [629]:
selected_from_selcols=[]
for i in countslist:
    selected_from_selcols.append(i[0])

In [ ]:
c=[
'Insulin - 70/30',
'Insulin - Glargine',
'Insulin - Humalog',
'Insulin - Humalog 75/25',
'Insulin - NPH',
'Insulin - Regular',
'Potassium Chloride',
'KCL (Bolus)',
'KCl (CRRT)',
'Fentanyl',
'Fentanyl (Concentrate)',
    
    
]
selected_from_selcols=unique(selected_from_selcols+c)

In [623]:

inputevents_mv[inputevents_mv.LABEL.isin(['Albumin 25%',
 'Albumin 5%',])]
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']=='Albumin 25%',
 'Albumin 5%', inplace=True)
inputevents_mv[inputevents_mv.LABEL.isin(['Albumin 5%'])]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE,LABEL,CATEGORY
2268568,353,30541,125746,271245.0,2112-02-23 18:35:00,2112-02-23 21:15:00,220862,200.000000,ml,75.000000,...,0,0,FinishedRunning,NaN,NaN,NaN,200.0,75.0,Albumin 5%,Blood Products/Colloids
2268569,8796,29080,181664,280895.0,2163-02-22 23:07:00,2163-02-23 00:07:00,220862,249.999990,ml,249.999990,...,0,0,FinishedRunning,NaN,NaN,NaN,250.0,250.0,Albumin 5%,Blood Products/Colloids
2268570,15051,31123,154148,209103.0,2114-03-06 13:45:00,2114-03-06 14:15:00,220862,49.999998,ml,99.999996,...,0,0,FinishedRunning,NaN,NaN,NaN,50.0,100.0,Albumin 5%,Blood Products/Colloids
2268571,15094,31123,154148,209103.0,2114-03-06 13:00:00,2114-03-06 13:30:00,220862,49.999998,ml,99.999996,...,0,0,FinishedRunning,NaN,NaN,NaN,50.0,100.0,Albumin 5%,Blood Products/Colloids
2268572,13844,31123,154148,209103.0,2114-03-08 20:00:00,2114-03-08 21:00:00,220862,49.999999,ml,49.999999,...,0,0,FinishedRunning,NaN,NaN,NaN,50.0,50.0,Albumin 5%,Blood Products/Colloids
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3299113,3573447,55639,139969,226799.0,2153-08-08 21:00:00,2153-08-08 23:00:00,220864,499.999980,ml,249.999990,...,0,0,FinishedRunning,NaN,NaN,NaN,500.0,250.0,Albumin 5%,Blood Products/Colloids
3299114,3573762,55639,139969,226799.0,2153-07-31 17:30:00,2153-07-31 20:00:00,220864,499.999995,ml,199.999998,...,0,0,FinishedRunning,NaN,NaN,NaN,500.0,200.0,Albumin 5%,Blood Products/Colloids
3299115,3579780,40548,168670,260223.0,2156-08-06 03:30:00,2156-08-06 04:00:00,220864,249.999990,ml,499.999980,...,0,0,FinishedRunning,NaN,NaN,NaN,250.0,500.0,Albumin 5%,Blood Products/Colloids
3299116,3581028,53919,154799,256370.0,2141-08-04 16:50:00,2141-08-04 17:50:00,220864,249.999990,ml,249.999990,...,0,0,FinishedRunning,NaN,NaN,NaN,250.0,250.0,Albumin 5%,Blood Products/Colloids


In [625]:
inputevents_mv[inputevents_mv.LABEL.isin(['sodium phosphate','Na Phos',])]
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']=='sodium phosphate','Na Phos', inplace=True)
inputevents_mv[inputevents_mv.LABEL.isin(['Na Phos'])]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE,LABEL,CATEGORY
3022204,3091,13401,195082,277693.0,2189-01-23 00:00:00,2189-01-23 06:01:00,225835,30.000001,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,30.0,0.083102,Na Phos,Medications
3022205,6172,29076,193948,224742.0,2151-02-07 23:00:00,2151-02-08 03:10:00,225835,15.000001,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,15.0,0.060000,Na Phos,Medications
3022206,6409,29076,193948,224742.0,2151-02-06 21:04:00,2151-02-07 03:09:00,225835,15.000001,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,15.0,0.041096,Na Phos,Medications
3022207,13134,28475,172559,204005.0,2115-02-07 22:55:00,2115-02-08 04:47:00,225835,30.000002,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,30.0,0.085227,Na Phos,Medications
3022208,15782,31123,154148,209103.0,2114-02-16 15:35:00,2114-02-16 21:32:00,225835,15.000000,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,15.0,0.042017,Na Phos,Medications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3023793,3565285,84310,100165,228494.0,2151-10-01 13:55:00,2151-10-01 19:43:00,225835,15.000001,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,15.0,0.043103,Na Phos,Medications
3023794,3578478,61195,197483,245009.0,2112-08-16 05:35:00,2112-08-16 11:50:00,225835,15.000001,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,15.0,0.040000,Na Phos,Medications
3023795,3566280,54043,143633,247460.0,2133-08-20 17:55:00,2133-08-21 00:10:00,225835,15.000001,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,15.0,0.040000,Na Phos,Medications
3023796,3582428,82434,172501,279673.0,2194-09-28 10:30:00,2194-09-28 14:30:00,225835,15.000001,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,15.0,0.062500,Na Phos,Medications


In [626]:
'KPhos 15mmol/250D5W','K Phos','potassium phos 30mml','k phos over 6hrs'
inputevents_mv[inputevents_mv.LABEL.isin(['KPhos 15mmol/250D5W','K Phos','potassium phos 30mml','k phos over 6hrs'])]
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']=='KPhos 15mmol/250D5W','K Phos', inplace=True)
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']=='potassium phos 30mml','K Phos', inplace=True)
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']=='k phos over 6hrs','K Phos', inplace=True)
inputevents_mv[inputevents_mv.LABEL.isin(['K Phos'])]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE,LABEL,CATEGORY
2328956,361,18082,181163,267692.0,2156-02-29 15:00:00,2156-02-29 20:52:00,225834,33.000000,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,33.000,0.093750,K Phos,Medications
2328957,503,18082,181163,267692.0,2156-03-04 10:27:00,2156-03-04 16:42:00,225834,15.000001,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,15.000,0.000000,K Phos,Medications
2328958,551,18082,181163,267692.0,2156-03-04 10:27:00,2156-03-04 13:34:00,225834,15.000000,mmol,NaN,...,0,2,Rewritten,RN,NaN,2156-03-04 12:15:00,15.000,0.080214,K Phos,Medications
2328959,477,18082,181163,267692.0,2156-03-03 03:00:00,2156-03-03 09:01:00,225834,30.000001,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,30.000,0.083102,K Phos,Medications
2328960,707,29056,163573,228281.0,2124-03-01 12:00:00,2124-03-01 18:01:00,225834,15.000001,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,15.000,0.041551,K Phos,Medications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2334569,3571649,91694,182538,298696.0,2114-07-26 06:01:00,2114-07-26 11:01:00,225834,15.000001,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,15.000,0.050000,K Phos,Medications
2334570,3569891,66105,184566,238502.0,2177-10-12 05:00:00,2177-10-12 08:20:00,225834,15.000001,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,15.000,0.075000,K Phos,Medications
2334571,3567247,84875,173821,201004.0,2193-08-04 04:56:00,2193-08-04 12:04:00,225834,30.000002,mmol,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,30.000,0.070093,K Phos,Medications
2334572,3570300,66105,184566,238502.0,2177-10-25 08:01:00,2177-10-25 08:26:00,225834,1.027397,mmol,NaN,...,0,0,Paused,NaN,NaN,NaN,11.392,0.041096,K Phos,Medications


In [627]:
inputevents_mv[inputevents_mv.LABEL.isin(['Nicardipine',
 'Nicardipine (mg/h)',])]
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']=='Nicardipine (mg/h)','Nicardipine', inplace=True)
inputevents_mv[inputevents_mv.LABEL.isin(['Nicardipine'])]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE,LABEL,CATEGORY
3538326,69604,27382,161290,283160.0,2119-08-21 19:57:00,2119-08-22 08:58:00,222042,125.000003,mg,2.000640,...,0,2,Rewritten,RN,NaN,2119-08-21 20:00:00,125.000000,2.000000,Nicardipine,Medications
3538327,69610,27382,161290,283160.0,2119-08-23 03:45:00,2119-08-23 06:03:00,222042,11.043534,mg,1.000320,...,0,0,Changed,NaN,NaN,NaN,125.000000,1.000000,Nicardipine,Medications
3538328,69614,27382,161290,283160.0,2119-08-24 02:12:00,2119-08-24 12:37:00,222042,124.999994,mg,2.500000,...,0,1,Rewritten,NaN,RN,2119-08-24 02:14:00,125.000000,2.500000,Nicardipine,Medications
3538329,69617,27382,161290,283160.0,2119-08-22 19:30:00,2119-08-22 19:50:00,222042,1.601597,mg,1.000998,...,0,0,Changed,NaN,NaN,NaN,29.967419,1.000000,Nicardipine,Medications
3538330,69434,27382,161290,283160.0,2119-08-22 11:10:00,2119-08-22 12:10:00,222042,4.804643,mg,1.000967,...,0,0,Changed,NaN,NaN,NaN,75.595360,1.000000,Nicardipine,Medications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3551932,3583438,97545,193655,264905.0,2147-07-22 09:07:00,2147-07-22 09:08:00,222042,0.153002,mg,1.981895,...,0,0,Rewritten,NaN,NaN,NaN,9.027133,2.000000,Nicardipine,Medications
3551933,3583442,97545,193655,264905.0,2147-07-22 09:07:00,2147-07-22 09:40:00,222042,9.027002,mg,3.543336,...,0,0,FinishedRunning,NaN,NaN,NaN,9.027000,3.575705,Nicardipine,Medications
3551934,3583445,97545,193655,264905.0,2147-07-21 16:58:00,2147-07-21 18:54:00,222042,8.953184,mg,0.999775,...,0,0,Changed,NaN,NaN,NaN,116.700120,1.000000,Nicardipine,Medications
3551935,3583451,97545,193655,264905.0,2147-07-22 09:40:00,2147-07-22 11:59:00,222042,21.267314,mg,1.981895,...,0,0,Changed,NaN,NaN,NaN,125.000000,2.000000,Nicardipine,Medications


In [632]:
inputevents_mv['LABEL'] = inputevents_mv['LABEL'].astype('category')
inputevents_mv['LABEL'].cat.set_categories(selcols,inplace=True)
inputevents_mv=inputevents_mv.dropna(how='any', subset=['AMOUNT']) # This removes rows with null values for VALUENUM column (a total of 190,141 records removed)

C:\Users\Mariam\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arrays\categorical.py:2747: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


In [633]:
inputevents_mv[inputevents_mv.LABEL.isin([
'Insulin - 70/30',
'Insulin - Glargine',
'Insulin - Humalog',
'Insulin - Humalog 75/25',
'Insulin - NPH',
'Insulin - Regular'])]

inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'Insulin - 70/30', 'Insulin - Regular', inplace=True)
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'Insulin - Glargine', 'Insulin - Regular', inplace=True)
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'Insulin - Humalog', 'Insulin - Regular', inplace=True)
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'Insulin - Humalog 75/25', 'Insulin - Regular', inplace=True)
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'Insulin - NPH', 'Insulin - Regular', inplace=True)

inputevents_mv[inputevents_mv.LABEL.isin(['Insulin - Regular'])]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE,LABEL,CATEGORY
1269067,108,29969,129547,248410.0,2181-02-20 21:00:00,2181-02-20 22:00:00,223258,5.000000,units,5.000000,...,0,0,Changed,NaN,NaN,NaN,100.000000,5.000000,Insulin - Regular,Medications
1269068,112,29969,129547,248410.0,2181-02-21 04:00:00,2181-02-21 07:24:00,223258,34.984998,units,10.289705,...,0,0,FinishedRunning,NaN,NaN,NaN,34.985001,10.294024,Insulin - Regular,Medications
1269069,116,29969,129547,248410.0,2181-02-21 04:00:00,2181-02-21 04:01:00,223258,0.166597,units,9.995806,...,0,0,Rewritten,NaN,NaN,NaN,34.985321,10.000000,Insulin - Regular,Medications
1269070,101,30354,120396,243526.0,2154-12-25 12:00:00,2154-12-25 12:01:00,223258,5.000000,units,NaN,...,0,2,Rewritten,RN,NaN,2154-12-25 18:12:00,5.000000,5.000000,Insulin - Regular,Medications
1269071,47,30354,120396,243526.0,2154-12-25 12:00:00,2154-12-25 12:01:00,223258,5.000000,units,NaN,...,0,1,Rewritten,NaN,RN,2154-12-25 18:13:00,5.000000,0.000000,Insulin - Regular,Medications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3288156,3611006,48292,199250,207195.0,2115-10-18 23:47:00,2115-10-18 23:48:00,223257,14.000000,units,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,14.000000,14.000000,Insulin - Regular,Medications
3288157,3611014,48292,199250,207195.0,2115-10-19 10:04:00,2115-10-19 10:05:00,223257,14.000000,units,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,14.000000,14.000000,Insulin - Regular,Medications
3288158,3581742,41702,120860,298837.0,2153-08-31 08:25:00,2153-08-31 08:26:00,223257,25.000000,units,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,25.000000,25.000000,Insulin - Regular,Medications
3288159,3581743,41702,120860,298837.0,2153-09-01 09:00:00,2153-09-01 09:01:00,223257,25.000000,units,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,25.000000,25.000000,Insulin - Regular,Medications


In [634]:
inputevents_mv[inputevents_mv.LABEL.isin([
'Potassium Chloride',
'KCL (Bolus)',
'KCl (CRRT)'])]

inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'KCL (Bolus)','KCl (CRRT)',inplace=True)
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'Potassium Chloride','KCl (CRRT)',inplace=True)
inputevents_mv[inputevents_mv.LABEL.isin(['KCl (CRRT)'])]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE,LABEL,CATEGORY
0,241,27063,139787,223259.0,2133-02-05 06:29:00,2133-02-05 08:45:00,225166,6.774532,mEq,NaN,...,0,1,Rewritten,NaN,RN,2133-02-05 12:52:00,10.000000,0.050000,KCl (CRRT),Medications
1,243,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 06:30:00,225166,2.813300,mEq,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,2.813300,0.050426,KCl (CRRT),Medications
2,247,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 07:03:00,225166,4.433333,mEq,NaN,...,0,0,Rewritten,NaN,NaN,NaN,4.433333,0.050000,KCl (CRRT),Medications
3,249,27063,139787,223259.0,2133-02-05 09:43:00,2133-02-05 12:30:00,225166,8.360021,mEq,NaN,...,0,0,Changed,NaN,NaN,NaN,10.000001,0.050248,KCl (CRRT),Medications
4,251,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 05:35:00,225166,0.050060,mEq,NaN,...,0,0,Rewritten,NaN,NaN,NaN,2.753300,0.050248,KCl (CRRT),Medications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3462085,3577403,51045,144997,221224.0,2119-09-08 21:53:00,2119-09-08 22:53:00,227522,49.999999,ml,49.999999,...,0,0,FinishedRunning,NaN,NaN,NaN,50.000000,50.000000,KCl (CRRT),Medications
3462086,3567929,78966,108022,211812.0,2202-08-03 05:15:00,2202-08-03 07:45:00,227522,249.999990,ml,99.999996,...,0,0,FinishedRunning,NaN,NaN,NaN,250.000000,100.000000,KCl (CRRT),Medications
3462087,3574374,91831,177630,284106.0,2187-09-12 03:27:00,2187-09-12 04:27:00,227522,49.999999,ml,49.999999,...,0,0,FinishedRunning,NaN,NaN,NaN,50.000000,50.000000,KCl (CRRT),Medications
3462088,3574377,91831,177630,284106.0,2187-09-11 14:44:00,2187-09-11 15:44:00,227522,49.999999,ml,49.999999,...,0,0,FinishedRunning,NaN,NaN,NaN,50.000000,50.000000,KCl (CRRT),Medications


In [635]:
inputevents_mv[inputevents_mv.LABEL.isin([
'Fentanyl',
'Fentanyl (Concentrate)'])]

inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']=='Fentanyl (Concentrate)', 'Fentanyl',inplace=True)
inputevents_mv[inputevents_mv.LABEL.isin(['Fentanyl'])]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE,LABEL,CATEGORY
1608530,3,30354,120396,243526.0,2154-12-24 12:00:00,2154-12-24 14:00:00,221744,0.050000,mg,25.000000,...,0,0,FinishedRunning,NaN,NaN,NaN,0.050000,25.00000,Fentanyl,Medications
1608531,11,30354,120396,243526.0,2154-12-24 12:41:00,2154-12-28 16:41:00,221744,2.500000,mg,25.000001,...,0,2,Rewritten,RN,NaN,2154-12-24 14:04:00,2.500000,25.00000,Fentanyl,Medications
1608532,31,30354,120396,243526.0,2154-12-24 15:40:00,2154-12-24 15:41:00,221744,500.000020,mcg,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,500.000000,500.00003,Fentanyl,Medications
1608533,38,30354,120396,243526.0,2154-12-24 15:35:00,2154-12-25 05:01:00,221744,0.671780,mg,50.008468,...,0,0,Stopped,NaN,NaN,NaN,2.462917,50.00000,Fentanyl,Medications
1608534,44,30354,120396,243526.0,2154-12-24 12:00:00,2154-12-28 16:00:00,221744,2.500000,mg,25.000001,...,0,2,Rewritten,RN,NaN,2154-12-24 14:05:00,2.500000,25.00000,Fentanyl,Medications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2589100,3571937,92137,103726,218089.0,2184-09-28 03:31:00,2184-09-28 05:00:00,225942,0.111447,mg,75.132996,...,0,0,Changed,NaN,NaN,NaN,2.499999,75.00000,Fentanyl,Medications
2589101,3575039,5171,125124,222399.0,2171-10-18 01:30:00,2171-10-18 04:21:00,225942,0.212385,mg,74.520972,...,0,0,Changed,NaN,NaN,NaN,0.773776,75.00000,Fentanyl,Medications
2589102,3584094,55781,186666,208138.0,2193-09-02 23:57:00,2193-09-03 12:21:00,225942,2.480000,mg,199.999998,...,0,0,FinishedRunning,NaN,NaN,NaN,2.480000,199.99997,Fentanyl,Medications
2589103,3572005,92137,103726,218089.0,2184-09-27 20:25:00,2184-09-27 20:26:00,225942,0.001252,mg,75.132996,...,0,0,Rewritten,NaN,NaN,NaN,0.537201,75.00000,Fentanyl,Medications


In [636]:
inpuputevents_mv_dict={
'Fentanyl':'Fentanyl',
'Fentanyl (Concentrate)':'Fentanyl',
'Insulin - 70/30':'Insulin - Regular',
'Insulin - Glargine':'Insulin - Regular',
'Insulin - Humalog':'Insulin - Regular',
'Insulin - Humalog 75/25':'Insulin - Regular',
'Insulin - NPH':'Insulin - Regular',
'Insulin - Regular':'Insulin - Regular',
'Pantoprazole (Protonix)':'Pantoprazole (Protonix)',
'Na Phos':'Na Phos',
'K Phos':'K Phos',
'Potassium Chloride':'KCl (CRRT)', 
'KCL (Bolus)':'KCl (CRRT)',   
'KCl (CRRT)':'KCl (CRRT)',   
'Free Water':'Free Water'}

In [637]:
for i in selected_from_selcols:
    if i not in inpuputevents_mv_dict:
        inpuputevents_mv_dict.update({i:i})

In [638]:
inputevents_mv['LABEL'] = inputevents_mv['LABEL'].map(inpuputevents_mv_dict)

In [639]:
inputevents_mv=inputevents_mv.pivot_table(values='AMOUNT', index=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID'], columns='LABEL')

In [641]:
inputevents_mv_dupdrp=inputevents_mv.drop_duplicates()

In [642]:
inputevents_mv_dupdrp.to_csv(output_files+'inputevents_mv_dupdrp.csv')

In [643]:
cols=list(inputevents_mv)

In [644]:
for col in cols:
    inputevents_mv[col] = inputevents_mv.groupby('HADM_ID')[col].transform('mean')

In [645]:
inputevents_mv.drop_duplicates(inplace=True)

In [646]:
inputevents_mv

,,,LABEL,Amiodarone,Calcium Gluconate,Cisatracurium,Citrate,Diltiazem,Dopamine,Fentanyl,Free Water,Fresh Frozen Plasma,GT Flush,...,Insulin - Regular,K Phos,KCl (CRRT),LR,Magnesium Sulfate,Morphine Sulfate,Na Phos,Nicardipine,Propofol,Sterile Water
SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,,,,,,,,,,,,,,,,,,,,,
23,124321,234044.0,222011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,2.0,NaN,NaN,6.375373,NaN,NaN
34,144319,290505.0,220970,NaN,NaN,NaN,NaN,NaN,NaN,0.037500,NaN,281.999988,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,165660,241249.0,225942,NaN,2.0,NaN,NaN,NaN,NaN,28.589013,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
85,112077,291697.0,222011,NaN,NaN,NaN,NaN,10.000001,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
107,174162,264253.0,225942,NaN,NaN,NaN,NaN,NaN,NaN,11.396136,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.861946,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99982,183791,239827.0,222011,NaN,NaN,NaN,NaN,NaN,255.806576,NaN,NaN,NaN,NaN,...,2.0,NaN,34.830000,NaN,2.0,NaN,NaN,NaN,NaN,NaN
99985,176670,279638.0,221744,NaN,NaN,NaN,NaN,NaN,NaN,23.521707,NaN,NaN,36.428571,...,NaN,NaN,34.999999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99991,151118,226241.0,222011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,15.0,20.000000,NaN,2.0,NaN,NaN,NaN,NaN,NaN


In [570]:
#todrop=[]
#for col in cols:
#    mis_perc=inputevents_mv[col].isnull().sum()/inputevents_mv.shape[0]
#    if mis_perc>0.70:
#        todrop.append(col)
#inputevents_mv.drop(todrop,inplace=True,axis=1)

In [647]:
inputevents_mv.to_csv(output_files+"inputevents_mv_allcols.csv")

# Merge Tables

In [653]:
inputevents_mv.rename(columns = {'Insulin - Regular':'Insulin'}, inplace = True)
inputevents_mv.rename(columns = {'K Phos':'Kphos'}, inplace = True)
inputevents_mv.rename(columns = {'Amiodarone':'Amiodarone'}, inplace = True)
inputevents_mv.rename(columns = {'Citrate':'Criticare HN'}, inplace = True)
inputevents_mv.rename(columns = {'Free Water':'Free Water Bolus'}, inplace = True)
inputevents_mv.rename(columns = {'KCl (CRRT)':'KCL'}, inplace = True)
inputevents_mv.rename(columns = {'Na Phos':'Sodium Phosphate'}, inplace = True)
inputevents_mv.rename(columns = {'Pantoprazole (Protonix)':'Protonix'}, inplace = True)

In [654]:
inputevents_mv

,,,LABEL,Amiodarone,Calcium Gluconate,Cisatracurium,Criticare HN,Diltiazem,Dopamine,Fentanyl,Free Water Bolus,Fresh Frozen Plasma,GT Flush,...,Insulin,Kphos,KCL,LR,Magnesium Sulfate,Morphine Sulfate,Sodium Phosphate,Nicardipine,Propofol,Sterile Water
SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,,,,,,,,,,,,,,,,,,,,,
23,124321,234044.0,222011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,2.0,NaN,NaN,6.375373,NaN,NaN
34,144319,290505.0,220970,NaN,NaN,NaN,NaN,NaN,NaN,0.037500,NaN,281.999988,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,165660,241249.0,225942,NaN,2.0,NaN,NaN,NaN,NaN,28.589013,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
85,112077,291697.0,222011,NaN,NaN,NaN,NaN,10.000001,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
107,174162,264253.0,225942,NaN,NaN,NaN,NaN,NaN,NaN,11.396136,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.861946,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99982,183791,239827.0,222011,NaN,NaN,NaN,NaN,NaN,255.806576,NaN,NaN,NaN,NaN,...,2.0,NaN,34.830000,NaN,2.0,NaN,NaN,NaN,NaN,NaN
99985,176670,279638.0,221744,NaN,NaN,NaN,NaN,NaN,NaN,23.521707,NaN,NaN,36.428571,...,NaN,NaN,34.999999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99991,151118,226241.0,222011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,15.0,20.000000,NaN,2.0,NaN,NaN,NaN,NaN,NaN


In [656]:
inputevents_cv

,,,LABEL,.45% Normal Saline,.45NS + .5:1 Heparin,.45NS + 1:1 Heparin,.9% Normal Saline,3% Normal Saline,Aggrastat,Albumin 25%,Albumin 5%,Amiodarone,Argatroban,...,Sterile Water,TF Residual,TPN,Thiamine,Tube Feeding,Ultracal,Vasopressin,Vecuronium,Vivonex,Xigris
SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,,,,,,,,,,,,,,,,,,,,,
3,145834.0,211552.0,30001,100.0,NaN,NaN,80.472973,NaN,NaN,NaN,NaN,44.374997,NaN,...,NaN,85.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185777.0,294638.0,30013,NaN,NaN,NaN,139.285714,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,107064.0,228232.0,30001,63.5,NaN,NaN,17.941176,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,159514.0,262299.0,30187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,150750.0,220597.0,30013,NaN,NaN,NaN,10.990000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,57.478261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32805,169344.0,251001.0,30013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32806,104049.0,262007.0,30187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32807,140728.0,265845.0,30013,NaN,NaN,NaN,500.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [657]:
def _merge(table1, table2):
    return table1.merge(table2, how='left', left_on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'], right_on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'])

In [658]:
charts_outputevents

,,LABEL,ALT,AST,Abdominal Girth (cm),Access Pressure,Admission Weight (Kg),Albumin,Alkaline Phosphate,Anion gap,Apnea Interval,Apnea Time Interval,...,OR Out OR Urine,Oral Gastric,Stool,TF Residual,Ultrafiltrate Ultrafiltrate,Urine Out Condom Cath,Urine Out Foley,Urine Out Suprapubic,Urine Out Void,Void
SUBJECT_ID,HADM_ID,ICUSTAY_ID,,,,,,,,,,,,,,,,,,,,,
2,163353,243653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,145834,211552.0,22.500000,68.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25.0,NaN,NaN,NaN,NaN,142.943396,NaN,NaN,NaN
4,185777,294638.0,24.000000,64.000000,NaN,NaN,NaN,2.80,NaN,NaN,NaN,NaN,...,NaN,NaN,150.0,NaN,NaN,NaN,NaN,NaN,508.333333,NaN
5,178980,214757.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,107064,228232.0,9.000000,54.666667,NaN,NaN,NaN,3.10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,80.689655,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99985,176670,279638.0,56.416667,54.833333,NaN,NaN,71.224935,2.45,281.083333,11.461538,20.0,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,178.800000,NaN,NaN,330.0
99991,151118,226241.0,NaN,NaN,NaN,NaN,100.394637,NaN,NaN,11.200000,NaN,NaN,...,NaN,300.0,NaN,NaN,NaN,NaN,143.500000,NaN,NaN,NaN
99992,197084,242052.0,NaN,NaN,NaN,NaN,65.335971,NaN,NaN,12.500000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,134.032258,NaN,NaN,NaN


In [659]:
df = inputevents_mv.combine_first(inputevents_cv)

In [660]:
df

,,,LABEL,.45% Normal Saline,.45NS + .5:1 Heparin,.45NS + 1:1 Heparin,.9% Normal Saline,3% Normal Saline,Aggrastat,Albumin 25%,Albumin 5%,Amiodarone,Argatroban,...,Sterile Water,TF Residual,TPN,Thiamine,Tube Feeding,Ultracal,Vasopressin,Vecuronium,Vivonex,Xigris
SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,,,,,,,,,,,,,,,,,,,,,
3,145834.0,211552.0,30001,100.0,NaN,NaN,80.472973,NaN,NaN,NaN,NaN,44.374997,NaN,...,NaN,85.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185777.0,294638.0,30013,NaN,NaN,NaN,139.285714,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,107064.0,228232.0,30001,63.5,NaN,NaN,17.941176,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,159514.0,262299.0,30187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,150750.0,220597.0,30013,NaN,NaN,NaN,10.990000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,57.478261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99982,183791.0,239827.0,222011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99985,176670.0,279638.0,221744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99991,151118.0,226241.0,222011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [661]:
merged_df_mvcv_charts=_merge(charts_outputevents, df)

In [662]:
merged_df_mvcv_charts

,,LABEL,ALT,AST,Abdominal Girth (cm),Access Pressure,Admission Weight (Kg),Albumin,Alkaline Phosphate,Anion gap,Apnea Interval,Apnea Time Interval,...,Sterile Water,TF Residual_y,TPN,Thiamine,Tube Feeding,Ultracal,Vasopressin,Vecuronium,Vivonex,Xigris
SUBJECT_ID,HADM_ID,ICUSTAY_ID,,,,,,,,,,,,,,,,,,,,,
2,163353,243653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,145834,211552.0,22.500000,68.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185777,294638.0,24.000000,64.000000,NaN,NaN,NaN,2.80,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,178980,214757.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,107064,228232.0,9.000000,54.666667,NaN,NaN,NaN,3.10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99985,176670,279638.0,56.416667,54.833333,NaN,NaN,71.224935,2.45,281.083333,11.461538,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99991,151118,226241.0,NaN,NaN,NaN,NaN,100.394637,NaN,NaN,11.200000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99992,197084,242052.0,NaN,NaN,NaN,NaN,65.335971,NaN,NaN,12.500000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [663]:
merged_df_mvcv_charts.to_csv(output_files+'merged_df_mvcv_charts.csv')

In [664]:
adm_data=pd.read_csv(output_files+'icu_stays_demographics.csv')

In [665]:
adm_data.columns

Index(['Unnamed: 0', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LOS',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'DIAGNOSIS', 'HAS_CHARTEVENTS_DATA', 'GENDER', 'AGE'],
      dtype='object')

In [666]:
merged_df_mvcv_charts

,,LABEL,ALT,AST,Abdominal Girth (cm),Access Pressure,Admission Weight (Kg),Albumin,Alkaline Phosphate,Anion gap,Apnea Interval,Apnea Time Interval,...,Sterile Water,TF Residual_y,TPN,Thiamine,Tube Feeding,Ultracal,Vasopressin,Vecuronium,Vivonex,Xigris
SUBJECT_ID,HADM_ID,ICUSTAY_ID,,,,,,,,,,,,,,,,,,,,,
2,163353,243653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,145834,211552.0,22.500000,68.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185777,294638.0,24.000000,64.000000,NaN,NaN,NaN,2.80,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,178980,214757.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,107064,228232.0,9.000000,54.666667,NaN,NaN,NaN,3.10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99985,176670,279638.0,56.416667,54.833333,NaN,NaN,71.224935,2.45,281.083333,11.461538,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99991,151118,226241.0,NaN,NaN,NaN,NaN,100.394637,NaN,NaN,11.200000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99992,197084,242052.0,NaN,NaN,NaN,NaN,65.335971,NaN,NaN,12.500000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [667]:
adm_data.drop(['Unnamed: 0','HAS_CHARTEVENTS_DATA'], inplace=True, axis=1)

In [668]:
merged_df_mvcv_charts_adm=_merge(merged_df_mvcv_charts, adm_data)

In [669]:
merged_df_mvcv_charts_adm.to_csv(output_files+'merged_df_mvcv_charts_adm.csv')

In [670]:
merged_df_mvcv_charts_adm

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ALT,AST,Abdominal Girth (cm),Access Pressure,Admission Weight (Kg),Albumin,Alkaline Phosphate,...,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,GENDER,AGE
0,2,163353,243653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN,NEWBORN,M,0.000000
1,3,145834,211552.0,22.500000,68.000000,NaN,NaN,NaN,NaN,NaN,...,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,HYPOTENSION,M,76.575342
2,4,185777,294638.0,24.000000,64.000000,NaN,NaN,NaN,2.80,NaN,...,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,"FEVER,DEHYDRATION,FAILURE TO THRIVE",F,47.876712
3,5,178980,214757.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,BUDDHIST,NaN,ASIAN,NEWBORN,M,0.000000
4,6,107064,228232.0,9.000000,54.666667,NaN,NaN,NaN,3.10,NaN,...,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CHRONIC RENAL FAILURE/SDA,F,65.983562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38521,99985,176670,279638.0,56.416667,54.833333,NaN,NaN,71.224935,2.45,281.083333,...,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,ENGL,JEWISH,MARRIED,WHITE,FEVER,M,53.843836
38522,99991,151118,226241.0,NaN,NaN,NaN,NaN,100.394637,NaN,NaN,...,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,DIVERTICULITIS/SDA,M,47.747945
38523,99992,197084,242052.0,NaN,NaN,NaN,NaN,65.335971,NaN,NaN,...,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,RETROPERITONEAL HEMORRHAGE,F,65.813699
38524,99995,137810,229633.0,35.000000,75.000000,NaN,NaN,67.928709,NaN,89.000000,...,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,ABDOMINAL AORTIC ANEURYSM/SDA,F,88.756164


# Merge with Diagnosis Table

In [671]:
diag=pd.read_csv(mimic_path+'DIAGNOSES_ICD.csv.gz')

In [673]:
diag_titles=pd.read_csv(mimic_path+'D_ICD_DIAGNOSES.csv.gz')

In [674]:
diag

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [675]:
diag_titles

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."
...,...,...,...,...
14562,14432,V7399,Scrn unspcf viral dis,Special screening examination for unspecified ...
14563,14433,V740,Screening for cholera,Screening examination for cholera
14564,14434,V741,Screening-pulmonary TB,Screening examination for pulmonary tuberculosis
14565,14435,V742,Screening for leprosy,Screening examination for leprosy (Hansen's di...


In [676]:
def diag_merge(table1, table2):
    return table1.merge(table2, how='left', left_on=['ICD9_CODE'], right_on=['ICD9_CODE'])

In [677]:
diag_titled=diag_merge(diag,diag_titles)

In [678]:
diag_titled.columns

Index(['ROW_ID_x', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE', 'ROW_ID_y',
       'SHORT_TITLE', 'LONG_TITLE'],
      dtype='object')

In [679]:
diag_titled.drop(['ROW_ID_x','ROW_ID_y','SEQ_NUM'],axis=1, inplace=True)

In [680]:
diag_titled

,SUBJECT_ID,HADM_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,109,172335,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
1,109,172335,486,"Pneumonia, organism NOS","Pneumonia, organism unspecified"
2,109,172335,58281,Chr nephritis in oth dis,Chronic glomerulonephritis in diseases classif...
3,109,172335,5855,Chron kidney dis stage V,"Chronic kidney disease, Stage V"
4,109,172335,4254,Prim cardiomyopathy NEC,Other primary cardiomyopathies
...,...,...,...,...,...
651042,97503,188195,20280,Oth lymp unsp xtrndl org,"Other malignant lymphomas, unspecified site, e..."
651043,97503,188195,V5869,Long-term use meds NEC,Long-term (current) use of other medications
651044,97503,188195,V1279,Prsnl hst ot spf dgst ds,Personal history of other diseases of digestiv...
651045,97503,188195,5275,Sialolithiasis,Sialolithiasis


In [681]:
def diag_charts_merge(table1, table2):
    return table1.merge(table2, how='left', left_on=['SUBJECT_ID','HADM_ID'], right_on=['SUBJECT_ID','HADM_ID'])

In [682]:
merged_df_mvcv_charts_adm_diag=diag_charts_merge(merged_df_mvcv_charts_adm, diag_titled)

In [683]:
merged_df_mvcv_charts_adm_diag

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ALT,AST,Abdominal Girth (cm),Access Pressure,Admission Weight (Kg),Albumin,Alkaline Phosphate,...,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,GENDER,AGE,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,2,163353,243653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NOT SPECIFIED,NaN,ASIAN,NEWBORN,M,0.000000,V3001,Single lb in-hosp w cs,"Single liveborn, born in hospital, delivered b..."
1,2,163353,243653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NOT SPECIFIED,NaN,ASIAN,NEWBORN,M,0.000000,V053,Need prphyl vc vrl hepat,Need for prophylactic vaccination and inoculat...
2,2,163353,243653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NOT SPECIFIED,NaN,ASIAN,NEWBORN,M,0.000000,V290,NB obsrv suspct infect,Observation for suspected infectious condition
3,3,145834,211552.0,22.5,68.0,NaN,NaN,NaN,NaN,NaN,...,NaN,CATHOLIC,MARRIED,WHITE,HYPOTENSION,M,76.575342,0389,Septicemia NOS,Unspecified septicemia
4,3,145834,211552.0,22.5,68.0,NaN,NaN,NaN,NaN,NaN,...,NaN,CATHOLIC,MARRIED,WHITE,HYPOTENSION,M,76.575342,78559,Shock w/o trauma NEC,Other shock without mention of trauma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473995,99999,113369,246512.0,NaN,NaN,NaN,NaN,96.39883,NaN,NaN,...,SPAN,JEHOVAH'S WITNESS,SEPARATED,HISPANIC OR LATINO,SPONDYLOLISTHESIS/SDA,F,64.021918,75612,Spondylolisthesis,Spondylolisthesis
473996,99999,113369,246512.0,NaN,NaN,NaN,NaN,96.39883,NaN,NaN,...,SPAN,JEHOVAH'S WITNESS,SEPARATED,HISPANIC OR LATINO,SPONDYLOLISTHESIS/SDA,F,64.021918,7861,Stridor,Stridor
473997,99999,113369,246512.0,NaN,NaN,NaN,NaN,96.39883,NaN,NaN,...,SPAN,JEHOVAH'S WITNESS,SEPARATED,HISPANIC OR LATINO,SPONDYLOLISTHESIS/SDA,F,64.021918,4019,Hypertension NOS,Unspecified essential hypertension
473998,99999,113369,246512.0,NaN,NaN,NaN,NaN,96.39883,NaN,NaN,...,SPAN,JEHOVAH'S WITNESS,SEPARATED,HISPANIC OR LATINO,SPONDYLOLISTHESIS/SDA,F,64.021918,25000,DMII wo cmp nt st uncntr,Diabetes mellitus without mention of complicat...


In [684]:
merged_df_mvcv_charts_adm_diag.to_csv(output_files+'merged_df_mvcv_charts_adm_diag.csv')

In [686]:
list(merged_df_mvcv_charts_adm_diag)

['SUBJECT_ID',
 'HADM_ID',
 'ICUSTAY_ID',
 'ALT',
 'AST',
 'Abdominal Girth (cm)',
 'Access Pressure',
 'Admission Weight (Kg)',
 'Albumin',
 'Alkaline Phosphate',
 'Anion gap',
 'Apnea Interval',
 'Apnea Time Interval',
 'Art.pH',
 'Arterial Base Excess',
 'Arterial Blood Pressure Alarm - High',
 'Arterial Blood Pressure Alarm - Low',
 'Arterial Blood Pressure mean',
 'Arterial CO2 Pressure',
 'Arterial CO2(Calc)',
 'Arterial O2 Saturation',
 'Arterial PaCO2',
 'Arterial pH',
 'BUN',
 'Birth Weight    (kg)',
 'Blood Flow (ml/min)',
 'Blood Temperature CCO (C)',
 'Braden Score',
 'CK (CPK)',
 'CK-MB',
 'CO (Arterial)',
 'Calcium non-ionized',
 'Cardiac Output (CCO)',
 'Central Venous Pressure',
 'Cerebral Perfusion Pressure',
 'Chloride (serum)',
 'Citrate (ACD-A)',
 'Creatinine',
 'CreatinineApacheIIScore',
 'CreatinineApacheIIValue',
 'Creatinine_ApacheIV',
 'Cuff Pressure',
 'Current Goal',
 'Daily Weight',
 'Day of Life',
 'Dialysate Rate',
 'Differential-Basos',
 'Differential-Eos

# Identifying subjects with delirium diagnosis

In [1]:
del_codes=['2930','2903','29281','2931','29041','29011','2910']

In [4]:
import pandas as pd 

merged_df_mvcv_charts_adm_diag=pd.read_csv(output_files+'merged_df_mvcv_charts_adm_diag.csv')

In [5]:
diag_codes=merged_df_mvcv_charts_adm_diag['ICD9_CODE'].tolist()
subject_ids=merged_df_mvcv_charts_adm_diag['SUBJECT_ID'].tolist()
delirious_subjects=[]
k=0
delirium=[]
for i in range(0,len(diag_codes)):
    if diag_codes[i] in del_codes:
        k+=1
        delirious_subjects.append(subject_ids[i])
        delirium.append(1)
    else:
        delirium.append(0)

In [6]:
short_titles=merged_df_mvcv_charts_adm_diag['SHORT_TITLE'].tolist()
long_titles=merged_df_mvcv_charts_adm_diag['LONG_TITLE'].tolist()

In [7]:
j=0
delirium_labels=[]
for i in range(0,len(diag_codes)):
    if diag_codes[i] in del_codes: #or subject_ids[i] in delirious_subjects:
        j+=1
        delirium_labels.append(1)
    else:
        delirium_labels.append(0)

In [8]:
j

1763

In [9]:
merged_df_mvcv_charts_adm_diag['Delirium_']=delirium_labels

In [10]:
merged_df_mvcv_charts_adm_diag.to_csv(output_files+"merged_df_mvcv_charts_adm_2.csv")

In [11]:
df=pd.read_csv(output_files+"merged_df_mvcv_charts_adm_2.csv")

In [18]:
icd9_codes=df['ICD9_CODE'].values

In [19]:
icd9_codes_dict={}
for code in icd9_codes:
    if code not in icd9_codes_dict:
        icd9_codes_dict.update({code:1})
    else:
        count=icd9_codes_dict.get(code)
        count+=1
        icd9_codes_dict.update({code:count})

In [740]:
import operator
countslist= sorted(icd9_codes_dict.items(), key=operator.itemgetter(1), reverse=True) 
countssortdict=dict(countslist)

In [741]:
countslist

[('4019', 14357),
 ('42731', 9128),
 ('4280', 9064),
 ('41401', 8456),
 ('2724', 7380),
 ('5849', 6846),
 ('25000', 6328),
 ('51881', 5473),
 ('53081', 4777),
 ('5990', 4695),
 ('2859', 4123),
 ('2449', 3683),
 ('2720', 3662),
 ('2762', 3493),
 ('486', 3490),
 ('2851', 3427),
 ('V5861', 3116),
 ('496', 3072),
 ('99592', 3011),
 ('40390', 2985),
 ('311', 2812),
 ('5859', 2758),
 ('0389', 2731),
 ('3051', 2708),
 ('5070', 2601),
 ('2875', 2452),
 ('2761', 2434),
 ('412', 2431),
 ('V1582', 2323),
 ('V053', 2301),
 ('V290', 2283),
 ('V4581', 2149),
 ('V4582', 2135),
 ('V5867', 2099),
 ('32723', 2072),
 ('5119', 2027),
 ('78552', 2016),
 ('41071', 1973),
 ('4240', 1970),
 ('42789', 1905),
 ('4241', 1903),
 ('2760', 1789),
 ('4168', 1750),
 ('45829', 1731),
 ('5845', 1726),
 ('2767', 1717),
 ('40391', 1714),
 ('49390', 1637),
 ('5180', 1618),
 ('2749', 1583),
 ('4589', 1551),
 ('5856', 1548),
 ('73300', 1544),
 ('9971', 1495),
 ('V3000', 1392),
 ('V1251', 1355),
 ('30000', 1324),
 ('E8798', 

In [724]:
df.drop(['SHORT_TITLE','LONG_TITLE','Delirium'],inplace=True,axis=1)
df.drop_duplicates(inplace=True)
df.to_csv(output_files+'merged_df_mvcv_charts_adm_los.csv')

In [725]:
df.drop(['ICD9_CODE','DIAGNOSIS'],axis=1,inplace=True)
df.drop_duplicates(inplace=True)
df=df[df.AGE>18]
df.to_csv(output_files+"los.csv")

In [23]:
merged_df_mvcv_charts_adm_diag.drop(['SHORT_TITLE','LONG_TITLE','DIAGNOSIS'],inplace=True,axis=1)
merged_df_mvcv_charts_adm_diag.drop_duplicates(inplace=True)
merged_df_mvcv_charts_adm_diag.to_csv(output_files+'merged_df_mvcv_charts_adm_diag_delirium_labels_2.csv')

In [720]:
file=open(output_files+'delirium_extracted_dataset_features.txt','w')
features=list(merged_df_mvcv_charts_adm_diag)
for item in features:
    file.write(item+'\n')
file.close()

In [20]:
merged_df_mvcv_charts_adm_diag.drop('ICD9_CODE',inplace=True,axis=1)

In [21]:
merged_df_mvcv_charts_adm_diag.drop_duplicates(inplace=True)

In [25]:
merged_df_mvcv_charts_adm_diag.drop('Unnamed: 0',axis=1,inplace=True)
merged_df_mvcv_charts_adm_diag.drop_duplicates(inplace=True)
merged_df_mvcv_charts_adm_diag

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ALT,AST,Abdominal Girth (cm),Access Pressure,Admission Weight (Kg),Albumin,Alkaline Phosphate,...,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,GENDER,AGE,Delirium_
0,2,163353,243653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN,M,0.000000,0
3,3,145834,211552.0,22.500000,68.000000,NaN,NaN,NaN,NaN,NaN,...,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,M,76.575342,0
12,4,185777,294638.0,24.000000,64.000000,NaN,NaN,NaN,2.80,NaN,...,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,F,47.876712,0
21,5,178980,214757.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,BUDDHIST,NaN,ASIAN,M,0.000000,0
24,6,107064,228232.0,9.000000,54.666667,NaN,NaN,NaN,3.10,NaN,...,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,F,65.983562,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473936,99985,176670,279638.0,56.416667,54.833333,NaN,NaN,71.224935,2.45,281.083333,...,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,ENGL,JEWISH,MARRIED,WHITE,M,53.843836,0
473949,99991,151118,226241.0,NaN,NaN,NaN,NaN,100.394637,NaN,NaN,...,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,M,47.747945,0
473966,99992,197084,242052.0,NaN,NaN,NaN,NaN,65.335971,NaN,NaN,...,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,F,65.813699,0
473978,99995,137810,229633.0,35.000000,75.000000,NaN,NaN,67.928709,NaN,89.000000,...,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,F,88.756164,0


In [26]:
merged_df_mvcv_charts_adm_diag=merged_df_mvcv_charts_adm_diag[merged_df_mvcv_charts_adm_diag['AGE']>=18]

In [28]:
merged_df_mvcv_charts_adm_diag.to_csv('extracted_icu_delirium_mimic.csv')

In [29]:
df_pos=merged_df_mvcv_charts_adm_diag[merged_df_mvcv_charts_adm_diag['Delirium_']==1]
print(df_pos.shape)
df_neg=merged_df_mvcv_charts_adm_diag[merged_df_mvcv_charts_adm_diag['Delirium_']==0]
df_final=pd.concat([df_pos,df_neg[0:len(df_pos)*2]])
df_final

(1714, 322)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ALT,AST,Abdominal Girth (cm),Access Pressure,Admission Weight (Kg),Albumin,Alkaline Phosphate,...,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,GENDER,AGE,Delirium_
1941,156,199280,259092.0,25.666667,77.000000,NaN,NaN,NaN,2.950000,NaN,...,PHYS REFERRAL/NORMAL DELI,REHAB/DISTINCT PART HOSP,Private,ENGL,EPISCOPALIAN,MARRIED,WHITE,M,63.178082,1
2604,214,197273,200066.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,EMERGENCY ROOM ADMIT,HOME,Private,NaN,NOT SPECIFIED,MARRIED,WHITE,M,63.482192,1
2741,223,105694,231395.0,47.000000,172.500000,NaN,NaN,NaN,2.700000,NaN,...,EMERGENCY ROOM ADMIT,HOME,Medicare,NaN,NOT SPECIFIED,NaN,WHITE,M,67.553425,1
3442,294,152578,222074.0,NaN,NaN,NaN,NaN,NaN,3.300000,NaN,...,TRANSFER FROM HOSP/EXTRAM,SNF,Medicare,NaN,OTHER,MARRIED,UNKNOWN/NOT SPECIFIED,M,78.712329,1
3685,321,192097,254444.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PHYS REFERRAL/NORMAL DELI,REHAB/DISTINCT PART HOSP,Medicare,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,F,76.265753,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40420,3764,144705,298933.0,94.214286,55.571429,NaN,NaN,NaN,3.411111,NaN,...,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicaid,NaN,UNOBTAINABLE,SINGLE,UNKNOWN/NOT SPECIFIED,M,26.178082,0
40429,3765,138299,206664.0,103.500000,100.000000,NaN,NaN,NaN,NaN,NaN,...,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicaid,NaN,CATHOLIC,MARRIED,HISPANIC OR LATINO,F,74.076712,0
40436,3765,182415,298787.0,25.000000,46.000000,NaN,NaN,NaN,3.000000,NaN,...,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicaid,NaN,CATHOLIC,MARRIED,HISPANIC OR LATINO,F,77.005479,0
40456,3767,103136,231124.0,69.000000,56.000000,NaN,NaN,NaN,NaN,NaN,...,PHYS REFERRAL/NORMAL DELI,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,MARRIED,WHITE,M,66.931507,0


In [77]:
print(100*(len(test[test['Delirium_']==1])/len(test)))
print(100*(len(test[test['Delirium_']==0])/len(test)))

33.33333333333333
66.66666666666666


In [78]:
print(100*(len(train[train['Delirium_']==1])/len(train)))
print(100*(len(train[train['Delirium_']==0])/len(train)))

33.33333333333333
66.66666666666666


In [74]:
print(100*(len(test[test['MARITAL_STATUS']=='SINGLE'])/len(test)))
print(100*(len(test[test['MARITAL_STATUS']=='MARRIED'])/len(test)))
print(100*(len(test[test['MARITAL_STATUS']=='WIDOWED'])/len(test)))
print(100*(len(test[test['MARITAL_STATUS']=='DIVORCED'])/len(test)))
print(100*(len(test[test['MARITAL_STATUS']=='SEPARATED'])/len(test)))
print(100*(len(test[test['MARITAL_STATUS']=='UNKNOWN (DEFAULT)'])/len(test)))

22.060252672497572
46.7444120505345
16.71525753158406
7.482993197278912
1.1661807580174928
0.6802721088435374


In [79]:
print(100*(len(train[train['MARITAL_STATUS']=='SINGLE'])/len(train)))
print(100*(len(train[train['MARITAL_STATUS']=='MARRIED'])/len(train)))
print(100*(len(train[train['MARITAL_STATUS']=='WIDOWED'])/len(train)))
print(100*(len(train[train['MARITAL_STATUS']=='DIVORCED'])/len(train)))
print(100*(len(train[train['MARITAL_STATUS']=='SEPARATED'])/len(train)))
print(100*(len(train[train['MARITAL_STATUS']=='UNKNOWN (DEFAULT)'])/len(train)))

24.580598103574033
47.5079017748602
15.26866034524678
5.835156819839534
1.2399708242159009
0.48626306831996113


In [67]:
print(len(test[test['GENDER']=='F'])/len(test))
print(len(test[test['GENDER']=='M'])/len(test))

0.423712342079689
0.576287657920311


In [66]:
print(len(train[train['GENDER']=='F'])/len(train))
print(len(train[train['GENDER']=='M'])/len(train))

0.424507658643326
0.5754923413566739


In [53]:
np.std(df_pos['AGE'])

67.4478432489211

In [50]:
np.average(df_neg['AGE'])

71.82137993318521

In [54]:
np.std(df_neg['AGE'])

50.592134480857005

In [55]:
np.average(df_final['AGE'])

76.60244827714817

In [57]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(df_final,stratify=df_final['Delirium_'],test_size=0.2,shuffle=True,random_state=11)

In [58]:
np.average(train['AGE'])

75.70127127817246

In [64]:
q3, q2=np.percentile(train['AGE'], [75,25])
q3-q2

25.682191780821917

In [59]:
np.average(test['AGE'])

80.20452893486161

In [63]:
q3, q2=np.percentile(test['AGE'], [75,25])
q3-q2

25.55342465753425

In [47]:
df_neg=df_final[df_final['Delirium_']==0]

In [30]:
print(df_neg.shape)
df_final.to_csv('extracted_downsampled_icu_delirium_mimic.csv')

(35142, 322)


In [32]:
import numpy as np
len(np.unique(df_pos['SUBJECT_ID']))

1648

In [33]:
import numpy as np
len(np.unique(df_pos['HADM_ID']))

1709

In [34]:
import numpy as np
len(np.unique(df_pos['ICUSTAY_ID']))

1709

In [43]:
df_final.shape

(5142, 322)

In [17]:
delirium_labels=merged_df_mvcv_charts_adm_diag['Delirium_'].values
delirium_count=0
for label in delirium_labels:
    if label==1:
        delirium_count+=1
delirium_count

1763

In [721]:
tracking_missing_vals=pd.DataFrame()
tracking_missing_vals['Feature']=features
missing_percents=[]
for col in features:
    mispercent=(merged_df_mvcv_charts_adm_diag[col].isnull().sum()/merged_df_mvcv_charts_adm_diag.shape[0])*100
    missing_percents.append(mispercent)
tracking_missing_vals['Missing Values %']=missing_percents
tracking_missing_vals.to_csv(output_files+"tracking_missing_vals_delirium_extracted_dataset.csv")